<a href="https://colab.research.google.com/github/rafaravazio/systematic-review-analysis/blob/main/Systematic_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing

In [ ]:
# Loading Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Setting working directory (change as needed)
import os

# working directory ADD PATH TO FOLDER
os.chdir("/content/drive/MyDrive/[PATH TO YOUR FOLDER]")

# Installing and downloading useful modules
!pip install xlsxwriter # Module to write and organize output tables
import xlsxwriter # Module to write and organize output tables
import pandas as pd # Data analysis and manipulation tool - https://pandas.pydata.org/
import seaborn as sns # Graphical representations - https://seaborn.pydata.org/
import matplotlib.pyplot as plt # Graphical representations - https://matplotlib.org/
import numpy as np # Scientific computing - https://numpy.org/
import scipy.stats as stats # Probability distributions and statistical functions- https://docs.scipy.org/doc/scipy/reference/stats.html
import scipy as sp # Probability distributions and statistical functions- https://docs.scipy.org/doc/scipy/reference/stats.html
import statsmodels # Statistical models and tests - https://www.statsmodels.org/stable/index.html
import statsmodels.stats.api as smsapi # Statistical models and tests - https://www.statsmodels.org/stable/index.html
import statsmodels.stats.weightstats as sms # Statistical models and tests - https://www.statsmodels.org/stable/index.html
import statsmodels.formula.api as smf # Statistical models and tests - https://www.statsmodels.org/stable/index.html
import statsmodels.api as sm # Statistical models and tests - https://www.statsmodels.org/stable/index.html
import math # Math!

# Some warning are persistent
import warnings
warnings.simplefilter("ignore", UserWarning)

# More data displayed in pandas columns
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 80)
pd.set_option('display.max_colwidth', 25) # Maximum width of each column
pd.options.display.float_format = "{:,.3f}".format # Show to at most 2 decimals values

# Print what is in the loaded directory
os.listdir()

# Functions

## General Functions

In [ ]:
# Somando colunas e inserindo as novas ao lado da última
def somarcolunas(df,somar,excluir=False):
  # Criando o nome da nova coluna
  for contagem, coluna in enumerate(somar):
    name = []
    if contagem == 0:
      nome = coluna
    else:
      nome = nome + '_' + coluna

  # Calculando e inserindo a nova coluna
  for contagem, coluna in enumerate(somar):
    if contagem == 0: # Primeiro valor
      soma = df[coluna] # Deixando a soma igual ao primeiro elemento
    else:
      soma = soma + df[coluna] # Somar elementos
      if coluna == somar[-1]: # Identificando se estamos no último elemento da lista
        if nome not in df.columns: # Vendo se já não existe coluna com esse nome
          df.insert(loc = (df.columns.get_loc(somar[-1])+1), column = nome, value = soma)
          name.append(nome)
          if excluir == True:
            df.drop(coluna, axis=1, inplace=True)

  truedf = df.copy()
  return truedf, name

def quantis(df, dividir, divisoes = 3):
  name = []
  quantilvar = True
  for variavel in dividir: # Variáveis que serão divididas
    quantis = []
    for tamanho in range(divisoes):
      quantis.append((tamanho+1)/(divisoes+1))
    quartis_bons = []
    lista = list(df[variavel].quantile(quantis))
    for contagem in range(divisoes):
      if (quantilvar == True) and (divisoes == 3):
        quantilvar = False
        nome = variavel + ' Quartile Division'
        temp = ['1' if x <= lista[0] else '2' if x <= lista[1] else '3' if x <= lista[2] else '4' for x in df[variavel]]
        if nome not in df.columns: # Vendo se já não existe coluna com esse nome
          df.insert(loc = (df.columns.get_loc(variavel)+1), column = nome, value = temp)
      nome = variavel + ' More Than ' + str(int((contagem+1)/(divisoes+1)*(100))) + '% (' + str(lista[contagem]) +')'
      temp = ['Não' if x <= lista[contagem] else 'Sim' for x in df[variavel]]
      if nome not in df.columns: # Vendo se já não existe coluna com esse nome
        df.insert(loc = (df.columns.get_loc(variavel)+1), column = nome, value = temp)
      quartis_bons.append(nome)
      name.append(nome)

  return df, name

def quantis_single_column(df, dividir, divisoes=3, show_ranges=False):
    name = []
    for variavel in dividir:  # Variables to be divided
        # Calculate the quantile values
        quantis = [(i + 1) / (divisoes + 1) for i in range(divisoes)]
        lista = list(df[variavel].quantile(quantis))

        # Determine the label for the new column
        nome = variavel + ' Quantile Division'

        # Depending on the show_ranges flag, format the output differently
        if show_ranges:
            # Create labels that show the range of each quantile
            edges = [df[variavel].min()] + lista + [df[variavel].max()]
            labels = [f"{int(edges[i])} - {int(edges[i+1])}" for i in range(len(edges)-1)]
            temp = [labels[sum(x > y for y in lista)] for x in df[variavel]]
        else:
            # Create a single column showing the quantile division as numbers
            temp = [sum(x > y for y in lista) + 1 for x in df[variavel]]

        # Check if the column already exists and add it if it doesn't
        if nome not in df.columns:
            df[nome] = temp

        name.append(nome)

    return df, name

def somarduplas(df,duplas):
  names = []
  for x in duplas:
    for y in duplas:
      if x != y:
        name1 = x + ' + ' + y
        name2 = y + ' + ' + x
        print(name1,name2)
        if (name1 not in names) and (name2 not in names):
          print('both names not in names')
          temp = df[x] + df[y]
          if name1 not in df.columns: # Vendo se já não existe coluna com esse nome
            df.insert(loc = (df.columns.get_loc(x)+1), column = name1, value = temp)
            df[name1].replace(1,0,inplace=True)
            df[name1].replace(2,1,inplace=True)
            names.append(name1)
            names.append(name2)

  truedf = df.copy()
  return truedf

def somartrios(df,trios):
  names = []
  for x in trios:
    for y in trios:
      for z in trios:
        if (x != y) and (x != z) and (y != z):
          name1 = x + ' + ' + y + ' + ' + z
          name2 = x + ' + ' + z + ' + ' + y
          name3 = y + ' + ' + z + ' + ' + x
          name4 = y + ' + ' + x + ' + ' + z
          name5 = z + ' + ' + x + ' + ' + y
          name6 = z + ' + ' + y + ' + ' + x
          if (name1 not in names) and (name2 not in names) and (name3 not in names) and (name4 not in names) and (name5 not in names) and (name6 not in names):
            temp = df[x] + df[y] + df[z]
            if name1 not in df.columns: # Vendo se já não existe coluna com esse nome
              df.insert(loc = (df.columns.get_loc(x)+1), column = name1, value = temp)
              df[name1].replace(1,0,inplace=True)
              df[name1].replace(2,1,inplace=True)
              names.append(name1)
              names.append(name2)
              names.append(name3)
              names.append(name4)
              names.append(name5)
              names.append(name6)

  truedf = df.copy()
  return truedf

Unicos e Faltantes

In [ ]:
# Criar tabela que mostra informações importantes de cada variável
# Criando a tabela com valores falfantes e quantidade de cada valor

def unicosefaltantes(df, organizar = 'Valores Faltantes'):

  '''Produza tabelas com tipos de variáveis, quantidade de valores faltantes e únicos para suas tabelas
  Diga nos argumentos qual é o banco de dados (primeiro argumento)
  Se quiser sortear a tabela por tipo, faltantes ou únicos em ordem descendente,
  No segundo argumento, coloque um dos três como string: Tipo de Variável, Valores Faltantes e Valores Únicos
  O padrão é sortear por Valores Faltantes'''

  variaveis = []
  for contagem, col in enumerate(df.columns):
    variaveis.append([col,df.dtypes[contagem],df[col].isnull().sum(),len(df[col].value_counts())])

  # Organizando a tabela - criar e sortear
  variaveis = pd.DataFrame(variaveis,columns=['Variável','Tipo de Variável','Valores Faltantes','Valores Únicos'])
  variaveis.sort_values(by=[organizar],ascending=False,inplace=True) # Sorteia por valores faltantes e únicos

  return variaveis

## Divisor Function

In [ ]:
'''Criador de novas colunas baseado em resultados de uma única coluna
Tens uma coluna onde, dentro de uma pergunta, você quer discriminar se alguém respondeu uma determinada coisa ou não?
Exemplo: qual droga você usa? Álcool, Cigarro ou Cocaína? Quer saber se uma pessoa usa álcool sim/não, cigarro sim/não etc?
Este código faz isso. Após, será criado um código para "corrigir" o que já foi inserido com o código inicial baseado em outras colunas
Exemplo: a coluna 1 faz essas perguntas mas também existe as colunas 2 e 3, que também as fazem. É possível alguém marcar umma opção em cada uma delas
Agora bastaria só mudar o Sim / Não das colunas já criadas para estar resolvido'''

from itertools import chain
import pandas as pd

def extract_unique_values(dataframe, column_name, separator = ','):
    """
    Extracts unique values from a specified column in a DataFrame where values are comma-separated.

    Parameters:
        dataframe (pd.DataFrame): The DataFrame containing the column.
        column_name (str): The name of the column to process.

    Returns:
        list: A list of unique values found in the column.
    """
    # Ensure the column exists in the DataFrame
    if column_name not in dataframe.columns:
        raise ValueError(f"The column '{column_name}' does not exist in the DataFrame.")

    # Split the values in the column and flatten them
    split_values = dataframe[column_name].str.split(separator)
    flattened_values = list(chain.from_iterable(split_values))

    # Use set to find unique values and convert it back to a list
    unique_values = list(set(flattened_values))

    return unique_values

def divisor(df, colunaprincipal, siglas, significado, colunasextras=None, apagar=False, numerical=True):
    """
    Create new columns in a DataFrame based on the responses in a specific column.

    Parameters:
        df (pd.DataFrame): The DataFrame to process.
        colunaprincipal (str): The main column containing comma-separated values.
        siglas (list): Abbreviations or keywords to look for within the column.
        significado (list): Corresponding meaningful names for new columns to create.
        colunasextras (list, optional): Additional columns to consider for adjusting responses. Defaults to None.
        apagar (bool, optional): Whether to delete the original columns after processing. Defaults to False.
        numerical (bool, optional): Whether to represent responses as numerical ('1', '0') or textual ('Yes', 'No'). Defaults to True.

    Returns:
        pd.DataFrame: The modified DataFrame with new columns added.
    """
    if colunasextras is None:
        colunasextras = []

    # Part 1: Creating new columns based on the main column
    for contagem, valor in enumerate(siglas):
        df[significado[contagem]] = df[colunaprincipal].apply(lambda x: 1 if valor in str(x) else 0 if numerical else 'Yes' if valor in str(x) else 'No')

    # Part 2: Refining these new columns using additional columns
    for coluna in colunasextras:
        for idx, val in df[coluna].iteritems():
            if val in siglas:
                col_idx = siglas.index(val)
                df.at[idx, significado[col_idx]] = 1 if numerical else 'Yes'

    # Part 3: Optionally removing the original columns
    if apagar:
        df.drop(columns=[colunaprincipal] + colunasextras, inplace=True)

    tempdf = df.copy() # For fragmentation

    return tempdf

Characterize Samples

In [ ]:
def calculate_percentage_distribution(df, columns, max_unique_values=10, use_counts=False):
    """
    Calculates and returns a DataFrame containing the distribution (either counts or percentages)
    of each value, including NaN, for specified columns in the input DataFrame, excluding columns
    with more than 'max_unique_values' unique values.

    Parameters:
    - df (pandas.DataFrame): The DataFrame containing the data.
    - columns (list of str): List of column names to analyze in the DataFrame.
    - max_unique_values (int): Maximum number of unique values a column can have to be analyzed.
    - use_counts (bool): If True, returns counts instead of percentages. Defaults to False.

    Returns:
    - pandas.DataFrame: A DataFrame where each column corresponds to one of the specified columns
      in the input DataFrame and each row corresponds to the distribution of a particular value,
      formatted as strings with percentage signs if use_counts is False.
    """
    distribution = {}
    skipped_columns = []

    for col in columns:
        if pd.unique(df[col]).size > max_unique_values:
            skipped_columns.append(col)
        else:
            # Calculate the distribution of each value (including NaN) in the column
            if use_counts:
                distribution[col] = df[col].value_counts(dropna=False)
            else:
                distribution[col] = df[col].value_counts(dropna=False, normalize=True) * 100

    # Create a DataFrame from the dictionary
    distribution_df = pd.DataFrame(distribution)

    # Optionally, format the DataFrame for better readability
    if not use_counts:
        distribution_df = distribution_df.applymap(lambda x: f"{x:.2f}%" if pd.notna(x) else "")

    # Print columns that were skipped
    if skipped_columns:
        print("Skipped columns (more than", max_unique_values, "unique values):", skipped_columns)

    return distribution_df


Parse description

In [ ]:
# Function to count if all elements are there
def parse_description(desc, all=True):
  if all == True:
    print('Mode of execution: all need to be present')
  else:
    print('Mode of execution: at least one needs to be present')
  countlist = []
  for row in df[colpos]:
    count = 0
    for element in templist:
      if element in row:
        count += 1
    if all == True:
      if count == len(templist):
        countlist.append(1)
      else:
        countlist.append(0)
    else:
      if count > 0:
        countlist.append(1)
      else:
        countlist.append(0)

  return countlist

## Multiple Cat Comparison

In [ ]:
# @title
from scipy.stats import chi2_contingency, f_oneway, kruskal, shapiro, ttest_ind, mannwhitneyu

def analyze_data_corrected_v8(df, qualitative_variable, numerical_threshold=5,
                              p_significance=4, decimal_places=1,
                              descending=True, exclude_vars=None, printsteps=False,
                              hidetotal=False):
    """
    Adjusted function to include statistical test type and effect size at the end,
    format categorical variable output as n (%), and allow configuration for p significance
    and decimal places for rounding.

    :param df: DataFrame with the data.
    :param qualitative_variable: Name of the qualitative variable for analysis.
    :param numerical_threshold: Threshold for considering a variable as numerical based on the number of distinct values.
    :param p_significance: Number of decimal places for the p-value (default 4).
    :param decimal_places: Number of decimal places for rounding results (default 1).
    :return: DataFrame formatted with the analysis results.
    """

    if exclude_vars is None:
        exclude_vars = []

    # Ensure that numerical columns are correctly interpreted
    for col in df.columns:
        if col not in exclude_vars and col != qualitative_variable:
            df.loc[:, col] = pd.to_numeric(df[col].replace('<NA>', np.nan), errors='ignore')

    # Get counts of each category and sort them if needed
    if descending:
        category_counts = df[qualitative_variable].astype(str).value_counts().sort_index()
        categories = category_counts.index.tolist()
        output_columns = ['Variables'] + [f'{cat} (n = {category_counts[cat]})' for cat in categories]
    else:
        categories = df[qualitative_variable].unique()
        output_columns = ['Variables'] + [f'{cat} (n = {len(df[df[qualitative_variable] == cat])})' for cat in categories]

    total_observations = len(df)
    results = []

    # Statistics for the entire dataset for categorical variables
    total_dataset_stats = []
    for column in df.columns:
        if column in exclude_vars or column == qualitative_variable:
            continue
        if printsteps:
            print('Producing output for variable:', column)
        try:
            if pd.api.types.is_numeric_dtype(df[column]) and df[column].nunique() >= numerical_threshold:
                numeric_data = df[column].dropna().astype(float)  # Ensure conversion to float
                mean = round(numeric_data.mean(), decimal_places)
                std_dev = round(numeric_data.std(), decimal_places)
                median = round(numeric_data.median(), decimal_places)
                iqr = round(numeric_data.quantile(0.75) - numeric_data.quantile(0.25), decimal_places)
                stats_str = f"mean: {mean} ± {std_dev}, median: {median} (IQR: {iqr})"
            else:
                category_percent = df[column].dropna().value_counts(normalize=True) * 100
                percent_str = ""
                for sub_cat, percent in category_percent.items():
                    n_sub_cat = df[column].dropna().value_counts()[sub_cat]
                    percent = round(percent, decimal_places)
                    percent_str += f"{sub_cat}: {n_sub_cat} ({percent}%), "
                stats_str = percent_str.strip(', ')
            total_dataset_stats.append(stats_str)
            if printsteps:
                print('Stats:', stats_str)
        except Exception as e:
            print(e, 'during data organization in:', column)
            pass

    for column in df.columns:
        if column in exclude_vars or column == qualitative_variable:
            continue

        # Initialize new_row with all possible category keys using the full dataset counts
        new_row = {'Variables': column}
        for cat in categories:
            new_row[f'{cat} (n = {category_counts[cat]})'] = "No data"
        p_value = None
        test_type = ''

        # Drop NaNs from the non-missing data for this column
        non_missing_data = df[column].dropna()
        percent_non_missing = len(non_missing_data) / total_observations * 100

        if pd.api.types.is_numeric_dtype(df[column]) and df[column].nunique() >= numerical_threshold:
            if printsteps:
                print('Statistical analysis for numerical variable:', column)
            # Numeric variable
            try:
                # Drop NaNs from each group
                groups = [group_data[column].dropna().astype(float) for _, group_data in df.groupby(qualitative_variable)]
                if len(categories) == 2:  # Binary outcome
                    if all(len(group) >= 3 for group in groups):
                        if all(shapiro(group.dropna())[1] > 0.05 for group in groups):
                            p_value = ttest_ind(groups[0], groups[1], nan_policy='omit')[1]
                            test_type = 't-test'
                        else:
                            p_value = mannwhitneyu(groups[0], groups[1], nan_policy='omit')[1]
                            test_type = 'Mann-Whitney'
                else:  # 3 or more outcomes
                    if all(len(group) >= 3 for group in groups):
                        if all(shapiro(group.dropna())[1] > 0.05 for group in groups):
                            p_value = f_oneway(*groups)[1]
                            test_type = 'ANOVA'
                        else:
                            p_value = kruskal(*groups)[1]
                            test_type = 'Kruskal-Wallis'

                for cat in categories:
                    group = df[df[qualitative_variable] == cat][column].dropna().astype(float)
                    if not group.empty:  # Check if the group is not empty
                        mean = round(group.mean(), decimal_places)
                        std_dev = round(group.std(), decimal_places)
                        median = round(group.median(), decimal_places)
                        iqr = round(group.quantile(0.75) - group.quantile(0.25), decimal_places)
                        new_row[f'{cat} (n = {len(df[df[qualitative_variable] == cat])})'] = f"mean: {mean} ± {std_dev}, median: {median} (IQR: {iqr})"
            except Exception as e:
                print(e, 'during numerical statistical analysis in:', column)
                pass

        else:
            if printsteps:
                print('Statistical analysis for categorical variable:', column)

            # Categorical variable
            try:
                contingency_table = pd.crosstab(df[column].dropna(), df[qualitative_variable].dropna())
                if not contingency_table.empty:
                    chi2, p_value, _, _ = chi2_contingency(contingency_table)
                    test_type = 'Chi-square'
                    for cat in categories:
                        total_group = df[df[qualitative_variable] == cat].shape[0]
                        category_percent = contingency_table[cat] / total_group
                        percent_str = ""
                        for sub_cat, val in category_percent.items():
                            n_sub_cat = round(val * total_group)
                            percent = round(val * 100, decimal_places)
                            percent_str += f"{sub_cat}: {n_sub_cat} ({percent}%), "
                        new_row[f'{cat} (n = {total_group})'] = percent_str.strip(', ')
            except Exception as e:
                print(e, column)
                pass

        new_row['p-value'] = round(p_value, p_significance) if p_value is not None else 99
        new_row['Test Type'] = test_type
        new_row['% Complete Data'] = f"{round(percent_non_missing, decimal_places)}%"
        results.append(new_row)
        if printsteps:
            print(new_row)

    results_df = pd.DataFrame(results, columns=output_columns + ['p-value', 'Test Type', '% Complete Data'])
    if not hidetotal:
        results_df.insert(1, f'Total (n = {len(df)})', total_dataset_stats)

    return results_df


Organizing the output

In [ ]:
def detailed_output_analysis_condensed(df_output):
    """
    Function to process the output of the analysis function and detail each subcategory of categorical variables
    in a single line, condensing the data of each column.

    :param df_output: DataFrame resulting from the analysis function.
    :return: Detailed DataFrame with subcategories in condensed lines.
    """
    detailed_results = []

    for index, row in df_output.iterrows():
        variable = row['Variables']
        new_base_line = row.to_dict()

        # Agora começando da coluna 'Total Dataset'
        if any(':' in str(new_base_line[col]) for col in df_output.columns[1:-3]):
            subcat_dict = {}
            for col in df_output.columns[1:-3]:  # Ajuste aqui também
                subcategories = str(new_base_line[col]).split(', ')
                for subcat in subcategories:
                    if ':' in subcat:
                        parts = subcat.split(': ')
                        subcat_name = parts[0].strip()
                        values = ': '.join(parts[1:])
                        subcat_dict.setdefault(subcat_name, new_base_line.copy())
                        subcat_dict[subcat_name]['Variables'] = f"{variable}, {subcat_name}"
                        subcat_dict[subcat_name][col] = values.strip()

            detailed_results.extend(subcat_dict.values())
        else:
            detailed_results.append(new_base_line)

    return pd.DataFrame(detailed_results)

# This is needed because the above function creates unnecessary lines
def condense_rows(df):
    """
    Function to condense information from lines with the same variable name into a single line.

    :param df: DataFrame with potentially repeated lines for each variable.
    :return: Condensed DataFrame.
    """
    condensed_df = pd.DataFrame()
    unique_variables = df['Variables'].unique()

    for var in unique_variables:
        var_lines = df[df['Variables'] == var]
        condensed_line = var_lines.iloc[0].copy()  # Copy the first line as a base

        # Iterate through columns to condense information
        for col in df.columns[:-3]:  # Include "Total Dataset" and other columns, ignore the last 3 columns
            column_data = var_lines[col].dropna().unique()
            condensed_line[col] = ', '.join(map(str, column_data))

        # Append using concat instead of the deprecated append method
        condensed_df = pd.concat([condensed_df, pd.DataFrame([condensed_line])], ignore_index=True)

    return condensed_df


# ML data organization

In [ ]:
!ls

## Reading

In [ ]:
# add path to ml .xlsx file
ml = pd.read_excel('[NAME OF THE EXCEL FILE].xlsx')
ml.head()

## Creating Variables

### Year divide

In [ ]:
# Play analysis 2021 and after x before
criteria = [2020]

# Preparatory settings
colpos = 'Year of publication'
newcolname = 'Before_2020_After' # Name of the new column
insert_position = ml.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(ml[colpos]), np.nan,  # Keep NaN as NaN
    np.where(
        ml[colpos] <= criteria[0], 0, 1)).astype(str)

# Inserting the column
try:
  ml.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  ml[newcolname] = newcol # Updating columns

print(ml[newcolname].value_counts(dropna=False))

### Imputation

In [ ]:
# Play analysis 2021 and after x before
criteria = ['Not Performed', 'Not Clearly Stated']

# Preparatory settings
colpos = 'Data imputation'
newcolname = colpos + "_Yes_or_No" # Name of the new column
insert_position = ml.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(ml[colpos]), np.nan,  # Keep NaN as NaN
    np.where(
        ml[colpos] == criteria[0], 0, 1)).astype(str)

# Inserting the column
try:
  ml.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  ml[newcolname] = newcol # Updating columns

print(ml[newcolname].value_counts(dropna=False))

In [ ]:
ml.columns

### Clustering other



In [ ]:
# Play analysis 2021 and after x before
criteria = ['k-Means', 'k-Means']

# Preparatory settings
colpos = 'Clustering Method'
newcolname = colpos + "_K_Means_Yes_or_No" # Name of the new column
insert_position = ml.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(ml[colpos]), np.nan,  # Keep NaN as NaN
    np.where(
        ml[colpos].isin(criteria), 1, 0)).astype(str)

# Inserting the column
try:
  ml.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  ml[newcolname] = newcol # Updating columns

print(ml[newcolname].value_counts(dropna=False))

In [ ]:
# Play analysis 2021 and after x before
criteria = ['Hierarchical Clustering', 'Ward Hierarchical Clustering', 'Hierarchical Clustering ,Two-step clustering (SPSS)']

# Preparatory settings
colpos = 'Clustering Method'
newcolname = colpos + "_Hierarchical_Yes_or_No" # Name of the new column
insert_position = ml.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(ml[colpos]), np.nan,  # Keep NaN as NaN
    np.where(
        ml[colpos].isin(criteria), 1, 0)).astype(str)

# Inserting the column
try:
  ml.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  ml[newcolname] = newcol # Updating columns

print(ml[newcolname].value_counts(dropna=False))

In [ ]:
# Play analysis 2021 and after x before
criteria = ['Hierarchical Clustering', 'Ward Hierarchical Clustering', 'Hierarchical Clustering ,Two-step clustering (SPSS)',
            'k-Means', 'k-Means']

# Preparatory settings
colpos = 'Clustering Method'
newcolname = colpos + "_Hierarchical_or_K-means_Yes_or_No" # Name of the new column
insert_position = ml.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(ml[colpos]), np.nan,  # Keep NaN as NaN
    np.where(
        ml[colpos].isin(criteria), 1, 0)).astype(str)

# Inserting the column
try:
  ml.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  ml[newcolname] = newcol # Updating columns

print(ml[newcolname].value_counts(dropna=False))

### Feature reduction

In [ ]:
# Play analysis 2021 and after x before
criteria = ['Not Performed', 'Not Clearly Stated']

# Preparatory settings
colpos = 'Feature Reduction Strategy'
newcolname = colpos + "_Yes_or_No" # Name of the new column
insert_position = ml.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(ml[colpos]), np.nan,  # Keep NaN as NaN
    np.where(
        ml[colpos] == criteria[0], 0, 1)).astype(str)

# Inserting the column
try:
  ml.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  ml[newcolname] = newcol # Updating columns

print(ml[newcolname].value_counts(dropna=False))

### Validation

### Temporal stability

In [ ]:
# Play analysis 2021 and after x before
criteria = ['No temporal stability was for the identified subtypes', 'Not clearly stated']

# Preparatory settings Validation method
colpos = 'Temporal Stability'
newcolname = colpos + "_Yes_or_No" # Name of the new column
insert_position = ml.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(ml[colpos]), np.nan,  # Keep NaN as NaN
    np.where(
        ml[colpos] == criteria[0], 0, 1)).astype(str)

# Inserting the column
try:
  ml.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  ml[newcolname] = newcol # Updating columns

print(ml[newcolname].value_counts(dropna=False))

### Code availability

In [ ]:
# Play analysis 2021 and after x before
criteria = ['No', 'code not available', 'mas o link não funciona', 'Link not working. https://github.com/mbbrendel/MBBS-Subtyping']

# Preparatory settings Validation method
colpos = 'Code Provided'
newcolname = colpos + "_Yes_or_No" # Name of the new column
insert_position = ml.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(ml[colpos]), np.nan,  # Keep NaN as NaN
    np.where(
        ml[colpos] == criteria[0], 0, 1)).astype(str)

# Inserting the column
try:
  ml.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  ml[newcolname] = newcol # Updating columns

print(ml[newcolname].value_counts(dropna=False))

## Analysis

In [ ]:
dependentvar = 'Before_2020_After'

# Running the code - first creating the master table
comparison_analysis = analyze_data_corrected_v8(ml,
                                                qualitative_variable = dependentvar,
                                                p_significance=4, decimal_places=1,
                                                exclude_vars = ['PMID', 'Article Title', 'Author', 'Year of publication'],
                                                numerical_threshold=5, printsteps = True, hidetotal = False)

# Running the adjustment function
# First one - creates new rows for categories
detailed_df = detailed_output_analysis_condensed(comparison_analysis)

# Second one - adjusts the last created output
condensed_df = condense_rows(detailed_df)
print('Lenght of dataset:',len(condensed_df))
condensed_df.head(10)

In [ ]:
# Export the result
sheet_path = 'Tables/'+dependentvar+'_ML_analysis.xlsx'
sheet_name = dependentvar[0:31]

with pd.ExcelWriter(sheet_path, engine='xlsxwriter') as writer:
    condensed_df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0)

    # Adjusting column width
    for column in condensed_df:
        column_width = max(condensed_df[column].astype(str).map(len).max(), len(column))
        col_idx = condensed_df.columns.get_loc(column)
        writer.sheets[sheet_name].set_column(col_idx, col_idx, column_width)

    # Ensure p-value is treated as numeric
    if 'p-value' in condensed_df.columns:
        condensed_df['p-value'] = pd.to_numeric(condensed_df['p-value'], errors='coerce')

    # Only significant interactions
    sigcondensed = condensed_df[condensed_df['p-value'] < 0.05]
    sheet_name = 'Significant'

    sigcondensed.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0)

    # Adjusting column width for the significant sheet
    for column in sigcondensed:
        column_width = max(sigcondensed[column].astype(str).map(len).max(), len(column))
        col_idx = sigcondensed.columns.get_loc(column)
        writer.sheets[sheet_name].set_column(col_idx, col_idx, column_width)

# Reading and exporting

Each variable of interest is evaluated, then measures for how to divide and better analyse this variable are studied, if applicable

Inspiration for analyses: https://docs.google.com/document/d/12u1vc_-MHrKS5Cd8AIm9HI8wf0qoOtdrMvWz8TttRnw/edit?usp=sharing

Dataset: https://docs.google.com/spreadsheets/d/1IOgFD3-STSSr40UDL4XWNSGIvi70rFzT/edit?usp=sharing&ouid=113124543929039104946&rtpof=true&sd=true

Forms: https://docs.google.com/forms/u/2/d/1Q9f2LCurKg6TzKlKtFWTNe1MyZbQpFsws6zVB0nyHzM/edit?usp=drive_web

In [ ]:
# Reading - add name of the file
df1 = pd.read_excel('[NAME OF THE EXCEL FILE].xlsx')

# Checking
print(df1.columns)
print(df1.shape)
df1.head()

Extra data up to May/24

In [ ]:
# New data - papers that need to be included after the first search (up to May 2024)
df2 = pd.read_excel('[NAME OF THE EXCEL FILE].xlsx')

# Checking
print(df2.columns)
print(df2.shape)
df2.head()

Checking what is misssing between datasets

In [ ]:
# Get columns of both DataFrames
df_columns = set(df1.columns)
df2_columns = set(df2.columns)

# Columns in df but not in df2
columns_in_df_not_in_df2 = df_columns - df2_columns

# Columns in df2 but not in df
columns_in_df2_not_in_df = df2_columns - df_columns

# Print the results
print("Columns in df but not in df2:", columns_in_df_not_in_df2)
print("Columns in df2 but not in df:", columns_in_df2_not_in_df)

# For a more detailed view, you can create a summary
summary = {
    'Columns in df but not in df2': list(columns_in_df_not_in_df2),
    'Columns in df2 but not in df': list(columns_in_df2_not_in_df)
}

# Display the summary
summary_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in summary.items()]))
summary_df

Uniting dataset

In [ ]:
# Merge DataFrames vertically
df = pd.concat([df1, df2], axis=0, ignore_index=True)

Selecting what is of interest

In [ ]:
# Selecting only possible clinical variables of interest (no ML analyses)
df = df[['Article Title',
       'Author (only the last name)', 'Year of publication', 'Reviewer', 'Inclusion Criteria',
       'Exclusion Criteria',
       'Which type of PD patients were included in the clustering analyses?',
       'Were subtyped patients roughly at the same disease stage or duration?',
       'From how many different centers came the patients to be subtyped?',
       'How were patients recruited?', 'Which diagnostic criteria was utilized to diagnose PD?',
       'Which was the sampling method?', 'Was there a longitudinal follow-up in any way?',
       'If longitudinal follow-up was provided, how much years were patients followed for?',
       'Completeness of follow-up',
       'Which type of descriptive data regarding the PD population was presented?',
       'Were the included patients for clustering purposes drug-naive?',
       'Number of PD patients utilized for clustering purposes',
       'Number of PD patients utilized for validation purposes',
       'Did the study include healthy controls in the clustering analyses alongside PD patients?',
       'Number of healthy controls utilized for validation or technique elaboration purposes',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [AMP-PD]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [BioFIND]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Fox Insight]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [GP2 Dataset]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [LRRK2 Cohort Consortium]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Parkinson Progression Marker Initiative (PPMI)]',
       "Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Parkinson's Disease Biomarker Program (PDBP)]",
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [UK Biobank]',
      'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Oxford Parkinson Disease Center Discovery Cohort]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Other specific or local datasets]',
       'If a different publicly available dataset than the ones mentioned above was utlized for any means, please specify its full name',
       'From which countries or world region came the patients whose data was used in the clustering?',
       'Which type data was utilized for the clustering algorithm?',
       'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?',
       'If the paper contains multiple exploration of clusters (creates different subgroups multiple times) shortly describe the differences in the cluster strategy for each exploration:',
       'If clinical data obtained from rating scales was used for clustering purposes, which data and rating scales were utilized?',
       'If neuroimaging data was utilized for clustering, did the article perform any sort of quality control measure?',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [MRI cortical thickness]',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [MRI subcortical volume]',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [MRI white matter lesions / hyperintensities]',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [Functional MRI]',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [MRI Others]',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [PET]',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [SPECT DAT binding ratios]',
       'How many subtypes did the evaluated study identified?',
       'If they were identified, what are the names (meaning) of the identified clusters?',
       'What is the number of patients present in each of the identified clusters?',
       'Subtype stability','Did the article provide an individual level prediction algorithm?']].reset_index(drop=True)

'''Saving here some columns that were excluded because they are subjective and were not reviewed
       "Based on all the information you've read, how would you rate the clinical applicability of the studied subtypes? (Part 1) [Subtyping Algorithm]",
       "Based on all the information you've read, how would you rate the clinical applicability of the studied subtypes? (Part 1) [Time Required]",
       "Based on all the information you've read, how would you rate the clinical applicability of the studied subtypes? (Part 1) [Applicable to Drug-Naive Stage]",
       "Based on all the information you've read, how would you rate the clinical applicability of the studied subtypes? (Part 1) [Cost]",
       "Based on all the information you've read, how would you rate the clinical applicability of the studied subtypes? (Part 1) [Applicability to general population]",
       "Based on all the information you've read, how would you rate the clinical applicability of the studied subtypes? (Part 2) [Prognostic Value]",
       "Based on all the information you've read, how would you rate the clinical applicability of the studied subtypes? (Part 2) [Treatment Implication]",
       "Based on all the information you've read, how would you rate the clinical applicability of the studied subtypes? (Part 2) [Clinical importance of differences in the variables used to define subtypes]",
       "Based on all the information you've read, how would you rate the clinical applicability of the studied subtypes? (Part 2) [Clinical importance of differences in external clinical/demographic features between subtypes]",
       "Based on all the information you've read, how would you rate the clinical applicability of the studied subtypes? (Part 2) [Clinical importance of differences in pathological/biomarker features between subtypes]"'''

df.head()

In [ ]:
# The "Reviewer" has the "Consensus" value, which is the entry for the aforementioned
print(df['Reviewer'].value_counts(dropna=False))
print('Before subsetting',df.shape)
df = df[df['Reviewer'] == 'Consensus'].reset_index(drop=True)
print('After subsetting',df.shape)

df.head()

In [ ]:
len(df), len(df1), len(df2)

Exporting the joined dataset

In [ ]:
df.to_csv('Review_Dataset.csv')

In [ ]:
!ls

# Initial variable creation

In [ ]:
df = pd.read_csv('Review_Dataset.csv')
df.head()

## Good variables

In [ ]:
df.head(2)

In [ ]:
relevantvars = ['Article Title', 'Author (only the last name)', 'Year of publication',
                'Number of PD patients utilized for clustering purposes',
                'Number of healthy controls utilized for validation or technique elaboration purposes',
                'How many subtypes did the evaluated study identified?']

## Number of groups

In [ ]:
# Putting in numeric
df['Number of PD patients utilized for clustering purposes for bubble'] = df['Number of PD patients utilized for clustering purposes']
df['Number of PD patients utilized for clustering purposes'] = pd.to_numeric(df['Number of PD patients utilized for clustering purposes'], errors='coerce').astype('Int64')
df['Number of healthy controls utilized for validation or technique elaboration purposes'] = pd.to_numeric(df['Number of healthy controls utilized for validation or technique elaboration purposes'], errors='coerce').astype('Int64')
df['How many subtypes did the evaluated study identified?'] = pd.to_numeric(df['How many subtypes did the evaluated study identified?'], errors='coerce').astype('Int64')

## Monogenic x sporadic

It is important to know exactly the genetic background of the population you are studying. Have the reviewed studies done so? This information is presented in two columns, with the second one needing to be divided for its subvalues

First: 'Which type of PD patients were included in the clustering analyses?'

Second: 'Which type of descriptive data regarding the PD population was presented?'

In [ ]:
# Defining the different options for answers
siglas = ['None of the mentioned data on the PD population was informed by the study',
           'Age', 'Gender', 'Ethnicity', 'Education', 'Income', 'Family History',
           'Age at disease onset', 'Disease duration (years) or staging (such as Hoehn & Yahr and Schawb and England scales)',
           'Vital Signs (blood pressure, cardiac frequency)', 'Physical Attributes (weight, height etc)',
           'Medication Doses', 'Genetic Status (monogenic PD, polygenic risk score)',
           'Summarised data from clinical scales (such as mean MDS-UPDRS scores, number of non-motor symptoms)',
           'Summarised data from neuroimaging studies (such as cortical thickness)',
           "Biomarker's profile (results from biomarker tests)"]

# Preparing to run the function
main_column = 'Which type of descriptive data regarding the PD population was presented?'
significado = siglas

# Dividing in new columns with 0 or 1
df = divisor(df, colunaprincipal = main_column, siglas = siglas,
                            significado = significado, apagar=False, numerical=True)

# Adding this list of variables to the relevantvars
if siglas[0] not in relevantvars:
  relevantvars = relevantvars + siglas
else:
  print('Already added')

df['Genetic Status (monogenic PD, polygenic risk score)'].value_counts()

In [ ]:
# Viewing the distribution of responses
calculate_percentage_distribution(df, siglas)

Creating the definition of not possessing genetic information

If study reported either genetic information or makes some distinction between study types in their definition

In [ ]:
# New column to be created
newcol = np.where(
    (df['Genetic Status (monogenic PD, polygenic risk score)'] == 1) |
    (df['Which type of PD patients were included in the clustering analyses?'] == 'Monogenic and Sporadic / Idiopathic'),
    1, 0)

colpos = 'Which type of PD patients were included in the clustering analyses?' # Name of the column position to be inserted
newcolname = 'Genetic_Status_Informed' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))
df[['Article Title',colpos,newcolname,'Genetic Status (monogenic PD, polygenic risk score)']].head(20)

## Diagnostic criteria

More on PDBP Dx Criteria (highly variable): https://movementdisorders.onlinelibrary.wiley.com/doi/full/10.1002/mds.26438?casa_token=0Kk48i-Na7sAAAAA%3A80sx8jSKQVLwmT8ixQfiqqxSBT9PeFOedBg-nXQ_v24Bm3DzTtt8BeHgW0fQwk4pNzfnTxCaaCOs6L7f

In [ ]:
# Checking
df['Which diagnostic criteria was utilized to diagnose PD?'].value_counts()

In [ ]:
# Defining the different options for answers
siglas = ['UK Brain Bank Criteria', 'MDS Clinical Criteria',
          'Clinical Diagnosis + Positive Dopamine Transporter (DAT) SPECT (PPMI)',
          'Clinical Diagnosis (no specific criteria mentioned - PDBP)',
          'Douglas Criteria',
          'World Health Organization Criteria',
          'ICD-10 Criteria',
          'Postmortem pathological diagnosis (part of the sample)',
          'Postmortem pathological diagnosis (complete sample)',
          'Self Report',
          'Other criteria',
          'Not sure or clearly stated']

# Preparing to run the function
main_column = 'Which diagnostic criteria was utilized to diagnose PD?'
significado = siglas

# Dividing in new columns with 0 or 1
df = divisor(df, colunaprincipal = main_column, siglas = siglas,
                            significado = significado, apagar=False, numerical=True)

# Adding this list of variables to the relevantvars
if siglas[0] not in relevantvars:
  relevantvars = relevantvars + siglas
else:
  print('Already added')

# Viewing the distribution of responses
calculate_percentage_distribution(df, siglas)

Dividing into having a formal criteria or having an unkown or self report criteria

In [ ]:
# Formal criteria list
criteria = ['Self Report', 'Not sure or clearly stated', 'Clinical Diagnosis (no specific criteria mentioned - PDBP)']

# Preparatory settings
colpos = 'Which diagnostic criteria was utilized to diagnose PD?' # Analysis column and position
newcolname = 'Formal_Criteria_Utilized_or_Informed' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    df[colpos].isin(criteria),
    0, 1) # Inverted here

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

According to Mestre classification

In [ ]:
# Formal criteria list
criteria = ['Self Report', 'Not sure or clearly stated', 'Clinical Diagnosis (no specific criteria mentioned - PDBP)']
criteria2 = ['Postmortem pathological diagnosis (part of the sample)', 'Postmortem pathological diagnosis (complete sample)']

# Preparatory settings
colpos = 'Which diagnostic criteria was utilized to diagnose PD?' # Analysis column and position
newcolname = 'Formal_Criteria_From_Mestre' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    df[colpos].isin(criteria),  # First criterion
    0,  # No formal criteria - 0
    np.where(
        df[colpos].isin(criteria2),  # Second criterion
        2, # Highest standadrd - pathological diagnosis
        1  # Formal criteria, but not pathological diagnosis
    )
)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

## Recruitment strategy

In [ ]:
# Checking
df['How were patients recruited?'].value_counts()

In [ ]:
# Formal criteria list
criteria = ['Community or population based']

# Preparatory settings
colpos = 'How were patients recruited?' # Analysis column and position
newcolname = 'Recruitment_Community_or_Population' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    df[colpos].isin(criteria),
    1, 0)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

## Cohorts used

The values here is already in different and individual columns, but it has "Creation and "Validation" options. We should create new columns to update them only if this cohort was utilized or not

**Individual datasets**

In [ ]:
# List of columns
templist = ['Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [AMP-PD]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [BioFIND]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Fox Insight]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [GP2 Dataset]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [LRRK2 Cohort Consortium]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Parkinson Progression Marker Initiative (PPMI)]',
       "Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Parkinson's Disease Biomarker Program (PDBP)]",
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [UK Biobank]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Oxford Parkinson Disease Center Discovery Cohort]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Other specific or local datasets]']

In [ ]:
# Create new columns based on whether the original columns are NaN or not
newlist = []
for col in templist:
    df['Dataset_Used ' + col] = np.where(df[col].isna(), 0, 1)
    newlist.append('Dataset_Used ' + col)

# Adding this list of variables to the relevantvars
if newlist[0] not in relevantvars:
  relevantvars = relevantvars + newlist
else:
  print('Already added')

# Viewing
df[[newlist[5],'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Parkinson Progression Marker Initiative (PPMI)]']].head()

**Sum of elements**

Sum of columns that have creation and columns that have validation

In [ ]:
# Create new columns based on whether the original columns are NaN or not
creationlist = []
validationlist = []

for row in range(len(df)):
  tempnumber = 0
  tempnumber2 = 0
  for col in templist:
    if 'Creation' in str(df.at[row, col]):
      tempnumber = tempnumber + 1
    else:
      pass

    if 'Validation' in str(df.at[row, col]):
      tempnumber2 = tempnumber2 + 1
    else:
      pass
  creationlist.append(tempnumber)
  validationlist.append(tempnumber2)

# Preparatory settings
colpos = 'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Other specific or local datasets]' # Analysis column and position
newcolname = 'Creation_Datasets_Sum' # Name of the new column
newcolname2 = 'Validation_Datasets_Sum' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# Inserting the column
try:
  df.insert(insert_position, newcolname, creationlist) # Inserting the column
  df.insert(insert_position, newcolname2, validationlist) # Inserting the column
except ValueError:
  df[newcolname] = creationlist # Updating columns
  df[newcolname2] = validationlist # Updating columns

# Adding this list of variables to the relevantvars
if newlist[0] not in relevantvars:
  relevantvars = relevantvars + newcolname + newcolname2
else:
  print('Already added')

# Viewing
print(df[newcolname].value_counts(dropna=False))
print(df[newcolname2].value_counts(dropna=False))
df[[newcolname,newcolname2]+templist].head()

In [ ]:
# What are those values 0 in creation? All datasets needs to have creation to be in this analysis
pd.set_option('display.max_colwidth', 25) # Maximum width of each column
df[df[newcolname] == 0]

In [ ]:
# What are those studies with 0 in validation?
pd.set_option('display.max_colwidth', 25) # Maximum width of each column
df[df[newcolname2] == 0].head()

In [ ]:
# Checking if performed validation in a different cohort
validation_cols = df[templist].apply(lambda x: x == 'Validation').any(axis=1)
validationint = validation_cols.astype(int)

# Preparatory settings
colpos = 'Validation_Datasets_Sum' # Analysis column and position
newcolname = 'Validation_in_a_Different_Cohort' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# Inserting the column
try:
  df.insert(insert_position, newcolname, validationint) # Inserting the column
except ValueError:
  df[newcolname] = validationint # Updating columns

# Adding this list of variables to the relevantvars
if newlist[0] not in relevantvars:
  relevantvars = relevantvars + newcolname
else:
  print('Already added')

# Checking how it goes
print(df[newcolname].value_counts(dropna=False))
df[[newcolname] + templist].head()

**Open access datasets**

In [ ]:
# Remove local datasets from the list
try:
  newlist.remove('Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Other specific or local datasets]')
except:
  print('Already removed the local or specific dataset column')

# Sum all values and, if it 1 or gre# Checking if additional and relevant data was lost
df['Open_Datasets_Used_Sum'] = df[newlist].sum(axis=1)
df['Open_Datasets_Used_Yes_or_No'] = df['Open_Datasets_Used_Sum'].apply(lambda x: 1 if x >= 1 else 0)

# Adding this list of variables to the relevantvars
if 'Open_Datasets_Used_Sum' not in relevantvars:
  relevantvars.append('Open_Datasets_Used_Sum')
  relevantvars.append('Open_Datasets_Used_Yes_or_No')
else:
  print('Already added')

# Filter rows where 'Cluster_Sum' is 0, indicating no siglas were found in these rows
print(df['Open_Datasets_Used_Sum'].value_counts(dropna=False))
print(df['Open_Datasets_Used_Yes_or_No'].value_counts(dropna=False))

## Countries represented

In [ ]:
# Defining the different options for answers
siglas = ["Sub-Saharan Africa", "Middle East & North Africa", "East Asia & Pacific (China, Japan, Australia)",
          "South Asia (India, Pakistan, Bangladesh)", "Europe & Central Asia", "Latin America & the Caribbean",
          "North America"]

# Preparing to run the function
main_column = 'From which countries or world region came the patients whose data was used in the clustering?'
significado = siglas

# Dividing in new columns with 0 or 1
df = divisor(df, colunaprincipal = main_column, siglas = siglas,
                            significado = significado, apagar=False, numerical=True)

# Adding this list of variables to the relevantvars
if siglas[0] not in relevantvars:
  relevantvars = relevantvars + siglas
else:
  print('Already added')

df[main_column].value_counts(dropna=False)

## Similar PD staging

Ideally, subtyped patents should be in the same disease stage of duration, for factors such as a different progression not to interfere

In [ ]:
# Defining column of interest
colpos = 'Were subtyped patients roughly at the same disease stage or duration?' # Analysis column and position

# Checking
df[colpos].value_counts(dropna=False)

In [ ]:
# Formal criteria list
criteria = ['Yes']

# Preparatory settings
newcolname = 'Similar_PD_Staging' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(df[colpos]), np.nan,  # Keep NaN as NaN
    np.where(df[colpos].isin(criteria), 1, 0))

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

## Longitudinal follow-up

**Follow-up status**

In [ ]:
# Defining column of interest
colpos = 'Was there a longitudinal follow-up in any way?' # Analysis column and position

# Checking
df[colpos].value_counts(dropna=False)

In [ ]:
# Formal criteria list
criteria = ['The study utilized longitudinal clinical data for clustering and/or association or interaction analyses']

# Preparatory settings
newcolname = 'Longitudinal_Followup_Status' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(df[colpos]), np.nan,  # Keep NaN as NaN
    np.where(df[colpos].isin(criteria), 1, 0))

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

**Time and completeness**

If longitudinal follow-up was provided, how much years were patients followed for?	Completeness of follow-up

In [ ]:
print(df['If longitudinal follow-up was provided, how much years were patients followed for?'].value_counts(dropna=False))
print(df['Completeness of follow-up'].value_counts(dropna=False))

# Formal criteria list
criteria = ['The study utilized longitudinal clinical data for clustering and/or association or interaction analyses']

# Preparatory settings
newcolname = 'Longitudinal_Followup_Time' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(df[colpos]), np.nan,  # Keep NaN as NaN
    np.where(df[colpos].isin(criteria), 1, 0))

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

In [ ]:
# Just creating new column only with numbers and forcing strings to become NaN for both exames

# Creating colnames
newcolname = 'Longitudinal_Followup_Years'
newcolname2 = 'Completeness_of_Followup'
colpos = 'Completeness of follow-up'
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# Creating new Series with forced conversion and NaN for non-numeric values
newcol = pd.to_numeric(df['If longitudinal follow-up was provided, how much years were patients followed for?'], errors='coerce')
newcol2 = pd.to_numeric(df['Completeness of follow-up'], errors='coerce')

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
  df.insert(insert_position, newcolname2, newcol2) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns
  df[newcolname2] = newcol2 # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
  relevantvars.append(newcolname2)
else:
  print(newcolname, 'Already added')


print(df[newcolname].value_counts(dropna=False))
print(df[newcolname2].value_counts(dropna=False))
df[[newcolname,newcolname2]].describe(include='all')

**Mestre definitions**

In [ ]:
# Checking columns
df[['Longitudinal_Followup_Status','Longitudinal_Followup_Years','Completeness_of_Followup']].head(20)

In [ ]:
# Longitudinal follow-up definition
# Define the conditions
condition1 = (df['Longitudinal_Followup_Status'] == 1) & (df['Longitudinal_Followup_Years'] >= 1) & (df['Longitudinal_Followup_Years'] < 3)
condition2 = (df['Longitudinal_Followup_Status'] == 1) & (df['Longitudinal_Followup_Years'] >= 3)

# Creating new Series with forced conversion and NaN for non-numeric values
newcolname = 'Longitudinal_Folloup_Years_Mestre_Definition'
newcol = np.where(condition1, 1, np.where(condition2, 2, 0))

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

# Checking
print(df[newcolname].value_counts(dropna=False))

In [ ]:
# Longitudinal follow-up completion
# Define the conditions based on corrected criteria
condition0 = (df['Longitudinal_Followup_Status'] == 0) | (df['Completeness_of_Followup'].isna()) | (df['Completeness_of_Followup'] <= 0.5)
condition1 = (df['Longitudinal_Followup_Status'] == 1) & (df['Completeness_of_Followup'] > 0.5) & (df['Completeness_of_Followup'] <= 0.75)
condition2 = (df['Longitudinal_Followup_Status'] == 1) & (df['Completeness_of_Followup'] > 0.75)

# Create new column with the corrected conditions
newcolname = 'Longitudinal_Folloup_Completion_Mestre_Definition'
newcol = np.where(condition0, 0, np.where(condition1, 1, np.where(condition2, 2, 0)))

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

# Checking
print(df[newcolname].value_counts(dropna=False))

## Drug-naive status

In [ ]:
# Defining column of interest
colpos = 'Were the included patients for clustering purposes drug-naive?' # Analysis column and position

# Checking
df[colpos].value_counts(dropna=False)

In [ ]:
# Formal criteria list
criteria = ['Yes']

# Preparatory settings
newcolname = 'Drug_Naive_Status' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(df[colpos]), np.nan,  # Keep NaN as NaN
    np.where(df[colpos].isin(criteria), 1, 0))

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

## Number of centers

In [ ]:
# Defining column of interest
colpos = 'From how many different centers came the patients to be subtyped?' # Analysis column and position

# Checking
df[colpos].value_counts(dropna=False)

In [ ]:
# Formal criteria list
criteria = ['More than one center']

# Preparatory settings
newcolname = 'Multicentric_X_Others' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(df[colpos]), np.nan,  # Keep NaN as NaN
    np.where(df[colpos].isin(criteria), 1, 0))

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

## Sampling method

In [ ]:
# Defining column of interest
colpos = 'Which was the sampling method?' # Analysis column and position

# Checking
df[colpos].value_counts(dropna=False)

In [ ]:
# Formal criteria list
criteria = ['Consecutive or random']

# Preparatory settings
newcolname = 'Consecutive_or_Random_Sampling' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(df[colpos]), np.nan,  # Keep NaN as NaN
    np.where(df[colpos].isin(criteria), 1, 0))

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

## Treatment response (cluster or post hoc)

In [ ]:
# Definindo colunas de interesse
colpos = 'Which type data was utilized for the clustering algorithm?'  # Análise de coluna e posição
colpos2 = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?'  # Análise de coluna e posição

# Critério formal
criteria = 'Clinical - Medication Response'

# Configurações preparatórias
newcolname = 'Treatment_Response_Evaluated'  # Nome da nova coluna
insert_position = df.columns.get_loc(colpos) + 1  # Calculando posição

# Função para verificar se o critério está presente
def contains_criteria(value, criteria):
    if isinstance(value, str):
        return criteria in value.split(', ')
    return False

# Nova coluna a ser criada
newcol = np.where(
    pd.isna(df[colpos]) & pd.isna(df[colpos2]), np.nan,  # Mantém NaN como NaN
    np.where(df[colpos].apply(lambda x: contains_criteria(x, criteria)) |
             df[colpos2].apply(lambda x: contains_criteria(x, criteria)), 1, 0))

# Inserindo a coluna
try:
    df.insert(insert_position, newcolname, newcol)  # Inserindo a coluna
except ValueError:
    df[newcolname] = newcol  # Atualizando colunas

# Adicionando esta lista de variáveis a relevantvars
if newcolname not in relevantvars:
    relevantvars.append(newcolname)
else:
    print(newcolname, 'Já adicionada')

print(df[newcolname].value_counts(dropna=False))

## Performed validation

**Either cross-validation or in another cohort**

In [ ]:
# Defining column of interest
colpos = 'Number of PD patients utilized for validation purposes' # Analysis column and position

# Checking
df[colpos].value_counts(dropna=False)

**Validation in the same dataset**

In [ ]:
# List of columns
templist = ['Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [AMP-PD]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [BioFIND]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Fox Insight]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [GP2 Dataset]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [LRRK2 Cohort Consortium]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Parkinson Progression Marker Initiative (PPMI)]',
       "Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Parkinson's Disease Biomarker Program (PDBP)]",
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [UK Biobank]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Other specific or local datasets]']

In [ ]:
# Checking if performed validation in a different cohort
tempcols = df[templist].apply(lambda x: x == 'Creation, Validation').any(axis=1)
tempint = tempcols.astype(int)

# Preparatory settings
colpos = 'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Other specific or local datasets]' # Analysis column and position
newcolname = 'Validation_in_the_Same_Datasets_Sum' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# Inserting the column
try:
  df.insert(insert_position, newcolname, tempint) # Inserting the column
except ValueError:
  df[newcolname] = tempint # Updating columns

# Adding this list of variables to the relevantvars
if newlist[0] not in relevantvars:
  relevantvars = relevantvars + newcolname
else:
  print('Already added')

# Checking how it goes
print(df[newcolname].value_counts(dropna=False))
df[[newcolname] + templist].head()

In [ ]:
# Longitudinal follow-up definition
# Define the conditions
condition1 = (df['Validation_in_the_Same_Datasets_Sum'] == 1)
condition2 = (df['Validation_in_a_Different_Cohort'] == 1)

# Creating new Series with forced conversion and NaN for non-numeric values
newcolname = 'Validation_not_Perfored_X_same_Cohort_X_Different_Cohort'
newcolname2 = 'Performed_Validation' # Either manner of validation
newcol = np.where(condition1, 1, np.where(condition2, 2, 0))
newcol2 = np.where(condition1 | condition2, 1, 0)  # Use the OR operator to combine conditions

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
  df.insert(insert_position, newcolname2, newcol2) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns
  df[newcolname2] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
  relevantvars.append(newcolname2)
else:
  print(newcolname, 'Already added')

# Checking
print(df[newcolname].value_counts(dropna=False))

In [ ]:
df[['Validation_in_the_Same_Datasets_Sum','Validation_in_a_Different_Cohort','Validation_not_Perfored_X_same_Cohort_X_Different_Cohort']].head()

## Used controls for comparison

Either cross-validation or in another cohort

In [ ]:
# Defining column of interest
colpos = 'Number of healthy controls utilized for validation or technique elaboration purposes' # Analysis column and position

# Checking
df[colpos].value_counts(dropna=False)

In [ ]:
# Formal criteria list
criteria = ['No data from controls was utilized for any of these purporses']

# Preparatory settings
newcolname = 'Used_Controls_For_Comparisons' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    df[colpos].isin(criteria),
    0, 1)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

## Countries of patients

Those 17 from 'Sub-Saharan Africa, Middle East & North Africa, Europe & Central Asia, North America' are PPMI patients

https://www.ppmi-info.org/about-ppmi/ppmi-clinical-sites

In [ ]:
# Defining column of interest
colpos = 'From which countries or world region came the patients whose data was used in the clustering?' # Analysis column and position

# Checking
df[colpos].value_counts(dropna=False)

### Option 1

Option 1 - not considering that PPMI recruits from Nigeria and Israel

In [ ]:
df['Original country without Sub-Saharan Africa and Middle East & North Africa'] = df[colpos].copy()
df['Original country without Sub-Saharan Africa and Middle East & North Africa'] = df['Original country without Sub-Saharan Africa and Middle East & North Africa'].str.replace('Sub-Saharan Africa, Middle East & North Africa, Europe & Central Asia, North America',
                                                                                         'Europe & Central Asia, North America', regex=False)

df['Original country without Sub-Saharan Africa and Middle East & North Africa'].value_counts(dropna=False)

In [ ]:
# Defining the different options for answers
siglas = ['Sub-Saharan Africa', 'Middle East & North Africa', 'East Asia & Pacific (China, Japan, Australia)',
          'South Asia (India, Pakistan, Bangladesh)', 'Europe & Central Asia', 'Latin America & the Caribbean',
          'North America']

# Preparing to run the function
main_column = 'Original country without Sub-Saharan Africa and Middle East & North Africa'
significado = siglas

# Dividing in new columns with 0 or 1
df = divisor(df, colunaprincipal = main_column, siglas = siglas,
                            significado = significado, apagar=False, numerical=True)

# Adding this list of variables to the relevantvars
if siglas[0] not in relevantvars:
  relevantvars = relevantvars + siglas
else:
  print('Already added')

# Modifying the names to remember that this was modified
tempnames = []

for name in siglas:
  tempname = name + str('_Adjusted_PPMI')
  tempnames.append(tempname)

# Create a dictionary mapping old names to new names
column_mapping = dict(zip(siglas, tempnames))

# Rename columns in the DataFrame using the column_mapping dictionary
df.rename(columns=column_mapping, inplace=True)

# Viewing the distribution of responses
calculate_percentage_distribution(df, tempnames)

### Option 2

Option 2 - considering that PPMI recruits from Nigeria and Israel

In [ ]:
# Defining the different options for answers
siglas = ['Sub-Saharan Africa', 'Middle East & North Africa', 'East Asia & Pacific (China, Japan, Australia)',
          'South Asia (India, Pakistan, Bangladesh)', 'Europe & Central Asia', 'Latin America & the Caribbean',
          'North America']

# Preparing to run the function
main_column = 'From which countries or world region came the patients whose data was used in the clustering?'
significado = siglas

# Dividing in new columns with 0 or 1
df = divisor(df, colunaprincipal = main_column, siglas = siglas,
                            significado = significado, apagar=False, numerical=True)

# Adding this list of variables to the relevantvars
if siglas[0] not in relevantvars:
  relevantvars = relevantvars + siglas
else:
  print('Already added')

# Viewing the distribution of responses
calculate_percentage_distribution(df, siglas)

## Minimum descriptive data

In [ ]:
siglas = ['None of the mentioned data on the PD population was informed by the study',
           'Age', 'Gender', 'Ethnicity', 'Education', 'Income', 'Family History',
           'Age at disease onset', 'Disease duration (years) or staging (such as Hoehn & Yahr and Schawb and England scales)',
           'Vital Signs (blood pressure, cardiac frequency)', 'Physical Attributes (weight, height etc)',
           'Medication Doses', 'Genetic Status (monogenic PD, polygenic risk score)',
           'Summarised data from clinical scales (such as mean MDS-UPDRS scores, number of non-motor symptoms)',
           'Summarised data from neuroimaging studies (such as cortical thickness)',
           "Biomarker's profile (results from biomarker tests)"]

In [ ]:
# Viewing the distribution of responses
calculate_percentage_distribution(df, siglas)

### C1 - Age and Sex

In [ ]:
# Formal criteria list
templist = ['Age', 'Gender']

# Preparatory settings
colpos = 'Which type of descriptive data regarding the PD population was presented?' # Analysis column and position
newcolname = 'Minimum_Info_Age_Sex' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# Looping through columns
# New column to be created
newcol = parse_description(templist, all=True)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))
df[siglas].head(5)

In [ ]:
# Formal criteria list
templist = ['Age', 'Gender', 'Age at disease onset']

# Preparatory settings
colpos = 'Which type of descriptive data regarding the PD population was presented?' # Analysis column and position
newcolname = 'Minimum_Info_Age_Sex_OnsetAge' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# Looping through columns
# New column to be created
newcol = parse_description(templist, all=True)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))
df[siglas].head(5)

### C2 - Age, Sex and Staging

In [ ]:
# Formal criteria list
templist = ['Age', 'Gender', 'Disease duration (years) or staging (such as Hoehn & Yahr and Schawb and England scales)']

# Preparatory settings
colpos = 'Which type of descriptive data regarding the PD population was presented?' # Analysis column and position
newcolname = 'Minimum_Info_Age_Sex_Staging' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# Looping through columns
# New column to be created
newcol = parse_description(templist, all=True)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))
df[siglas].head(5)

### C3 - Age, Sex, Staging and Scales

In [ ]:
# Formal criteria list as a set
templist = ['Age', 'Gender',
            'Disease duration (years) or staging (such as Hoehn & Yahr and Schawb and England scales)',
            'Summarised data from clinical scales (such as mean MDS-UPDRS scores, number of non-motor symptoms)']

# Preparatory settings
colpos = 'Which type of descriptive data regarding the PD population was presented?' # Analysis column and position
newcolname = 'Minimum_Info_Age_Sex_Staging_Scales' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# Create new column by applying the parsing function and checking for subset
newcol = parse_description(templist, all=True)

# Inserting the new column
try:
    df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
    df[newcolname] = newcol # Updating column if already exists

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

# Output the new column's value counts and a preview of the DataFrame
print(df[newcolname].value_counts(dropna=False))
df.head(5)  # Assuming siglas was a placeholder for demonstration

### C4 - Age, Sex, Staging, Scales and Medication

In [ ]:
# Formal criteria list
templist = ['Age', 'Gender',
            'Disease duration (years) or staging (such as Hoehn & Yahr and Schawb and England scales)',
            'Summarised data from clinical scales (such as mean MDS-UPDRS scores, number of non-motor symptoms)',
            'Medication Doses']

# Preparatory settings
colpos = 'Which type of descriptive data regarding the PD population was presented?' # Analysis column and position
newcolname = 'Minimum_Info_Age_Sex_Staging_Scales_Medication' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# Looping through columns
# New column to be created
newcol = parse_description(templist, all=True)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))
df[siglas].head(5)

### C5 - Any Complex Data

Checking if ANY of those is present: Omics, Neuroimaging and Biomarkers

In [ ]:
# Formal criteria list
templist = ['Genetic Status (monogenic PD, polygenic risk score)',
           'Summarised data from neuroimaging studies (such as cortical thickness)',
           "Biomarker's profile (results from biomarker tests)"]

# Preparatory settings
colpos = 'Which type of descriptive data regarding the PD population was presented?' # Analysis column and position
newcolname = 'Minimum_Info_Complex_Data_Any_Present' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# Looping through columns
# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))
df[siglas].head(5)

# Creating clustering variables

## General division

In [ ]:
# Defining the different options for answers
siglas = [
    'Clinical - Vital Signs or Neurologic Examination Findings',
    'Clinical - Age',
    'Clinical - Gender',
    'Clinical - Ethnicity',
    'Clinical - Age at disease onset',
    'Clinical - Disease staging',
    'Clinical - Motor Features Obtained Through Rating Scales (tremor)',
    'Clinical - Motor Features Obtained Through Rating Scales (bradykinesia)',
    'Clinical - Motor Features Obtained Through Rating Scales (rigidity)',
    'Clinical - Motor Features Obtained Through Rating Scales (gait and/or posture)',
    'Clinical - Motor Features Obtained Through Rating Scales (speech)',
    'Clinical - Motor Features Obtained Through Wearable Sensors or other Technical Instruments',
    'Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)',
    'Clinical - Cognition (memory, attention, executive, visuospatial memory etc)',
    'Clinical - Neuropsychiatric (anxiety, depression, impulsivity, delusions, hallucinations etc)',
    'Clinical - Sleep (REM sleep behavior disorder, insomnia, daytime sleepiness)',
    'Clinical - Anosmia or Hyposmia (absence or loss of the capability to smell)',
    'Clinical - Autonomic (gastroparesis, constipation, orthostatic hypotension, urinary incontinence, sexual difficulties)',
    'Clinical - Quality of Life',
    'Clinical - Medication (types and/or dosages)',
    'Clinical - Other (pain, fatigue etc)',
    'Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)',
    'Neuroimaging - Structural or Volumetric MRI',
    'Neuroimaging - Diffusion Imaging MRI',
    'Neuroimaging - DaTSCAN',
    'Neuroimaging - Radiomics',
    'Neuroimaging - Functional MRI',
    'Neuroimaging - Other Methods (e.g. neuromelanin-sensitive MRI)',
    'Neurophysiology - Electroencephalography',
    'Neurophysiology - Electroneuromiography',
    'Neurophysiology - Other',
    'Omics - Proteomics',
    'Omics - Transcriptomics',
    'Omics - Genomics',
    'Omics - Lipidomics',
    'Omics - Other (e.g. metabolomics, metagenomics)',
    'Clinical - Phonation',
    'Clinical - Motor Tests (example: Timed-Up & Go)']

# Preparing to run the divisor function
main_column = 'Which type data was utilized for the clustering algorithm?'
significado = siglas

# Dividing in new columns with 0 or 1
df = divisor(df, colunaprincipal = main_column, siglas = siglas,
                            significado = significado, apagar=False, numerical=True)

# Renaming to identify they are from the clustering group
clustering_names = []
for value in siglas:
  clustering_names.append(str('Clustering_')+value)

column_mapping = dict(zip(siglas, clustering_names))

# Rename columns in the DataFrame using the column_mapping dictionary
if 'Clustering_Clinical - Age' not in df.columns:
  df.rename(columns=column_mapping, inplace=True)
  relevantvars = relevantvars + clustering_names
else:
  print('Already added and renamed')

# Viewing the distribution of responses
calculate_percentage_distribution(df, clustering_names)

In [ ]:
relevantvars

In [ ]:
# Checking if additional and relevant data was lost
df['Cluster_Sum'] = df[clustering_names].sum(axis=1)

# Filter rows where 'Cluster_Sum' is 0, indicating no siglas were found in these rows
df[df['Cluster_Sum'] == 0]

## Any data subtypes

Identifying big data subtypes

### Clinical

In [ ]:
# Formal criteria list
templist = [
    'Clinical - Vital Signs or Neurologic Examination Findings',
    'Clinical - Age',
    'Clinical - Gender',
    'Clinical - Ethnicity',
    'Clinical - Age at disease onset',
    'Clinical - Disease staging',
    'Clinical - Motor Features Obtained Through Rating Scales (tremor)',
    'Clinical - Motor Features Obtained Through Rating Scales (bradykinesia)',
    'Clinical - Motor Features Obtained Through Rating Scales (rigidity)',
    'Clinical - Motor Features Obtained Through Rating Scales (gait and/or posture)',
    'Clinical - Motor Features Obtained Through Rating Scales (speech)',
    'Clinical - Motor Features Obtained Through Wearable Sensors or other Technical Instruments',
    'Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)',
    'Clinical - Cognition (memory, attention, executive, visuospatial memory etc)',
    'Clinical - Neuropsychiatric (anxiety, depression, impulsivity, delusions, hallucinations etc)',
    'Clinical - Sleep (REM sleep behavior disorder, insomnia, daytime sleepiness)',
    'Clinical - Anosmia or Hyposmia (absence or loss of the capability to smell)',
    'Clinical - Autonomic (gastroparesis, constipation, orthostatic hypotension, urinary incontinence, sexual difficulties)',
    'Clinical - Quality of Life',
    'Clinical - Medication (types and/or dosages)',
    'Clinical - Other (pain, fatigue etc)',
    'Clinical - Phonation',
    'Clinical - Motor Tests (example: Timed-Up & Go)']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Any_Clinical' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Non Clinical

In [ ]:
# Formal criteria list
templist = ['Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)',
    'Neuroimaging - Structural or Volumetric MRI',
    'Neuroimaging - Diffusion Imaging MRI',
    'Neuroimaging - DaTSCAN',
    'Neuroimaging - Radiomics',
    'Neuroimaging - Functional MRI',
    'Neuroimaging - Other Methods (e.g. neuromelanin-sensitive MRI)',
    'Neurophysiology - Electroencephalography',
    'Neurophysiology - Electroneuromiography',
    'Neurophysiology - Other',
    'Omics - Proteomics',
    'Omics - Transcriptomics',
    'Omics - Genomics',
    'Omics - Lipidomics',
    'Omics - Other (e.g. metabolomics, metagenomics)']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Any_Non_Clinical' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Biomarkers

Check variable 'Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)'

In [ ]:
# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append('Clustering_Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)')
else:
  print('Already added')

### Neuroimaging

In [ ]:
# Formal criteria list
templist = ['Neuroimaging - Structural or Volumetric MRI',
    'Neuroimaging - Diffusion Imaging MRI',
    'Neuroimaging - DaTSCAN',
    'Neuroimaging - Radiomics',
    'Neuroimaging - Functional MRI',
    'Neuroimaging - Other Methods (e.g. neuromelanin-sensitive MRI)']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Any_Neuroimaging' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Neurophysiology

In [ ]:
# Formal criteria list
templist = ['Neurophysiology - Electroencephalography',
    'Neurophysiology - Electroneuromiography',
    'Neurophysiology - Other']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Any_Neurophysiology' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Omics

In [ ]:
# Formal criteria list
templist = ['Omics - Proteomics',
    'Omics - Transcriptomics',
    'Omics - Genomics',
    'Omics - Lipidomics',
    'Omics - Other (e.g. metabolomics, metagenomics)']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Any_Omics' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Omics or Biomarkers

In [ ]:
# Formal criteria list
templist = ['Omics - Proteomics',
    'Omics - Transcriptomics',
    'Omics - Genomics',
    'Omics - Lipidomics',
    'Omics - Other (e.g. metabolomics, metagenomics)',
            'Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Any_Omics_or_Biomarkers' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

## Specific data subtypes

### Demographic

In [ ]:
# Formal criteria list
templist = ['Clinical - Age',
    'Clinical - Gender',
    'Clinical - Ethnicity']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Any_Demographic' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Disease staging or age at onset

In [ ]:
# Formal criteria list
templist = ['Clinical - Age at disease onset','Clinical - Disease staging']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Any_Time_or_Staging_Measurements' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Clinical motor scales

Motor clinical data obtained from rating scales

Including complications

In [ ]:
# Formal criteria list
templist = ['Clinical - Motor Features Obtained Through Rating Scales (tremor)',
    'Clinical - Motor Features Obtained Through Rating Scales (bradykinesia)',
    'Clinical - Motor Features Obtained Through Rating Scales (rigidity)',
    'Clinical - Motor Features Obtained Through Rating Scales (gait and/or posture)',
    'Clinical - Motor Features Obtained Through Rating Scales (speech)',
    'Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Clinical_Any_Motor_Scales_Plus_Complications' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

Excluding complications

In [ ]:
# Formal criteria list
templist = ['Clinical - Motor Features Obtained Through Rating Scales (tremor)',
    'Clinical - Motor Features Obtained Through Rating Scales (bradykinesia)',
    'Clinical - Motor Features Obtained Through Rating Scales (rigidity)',
    'Clinical - Motor Features Obtained Through Rating Scales (gait and/or posture)',
    'Clinical - Motor Features Obtained Through Rating Scales (speech)']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Clinical_Any_Motor_Scales_No_Complications' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))

Including complications

In [ ]:
# Formal criteria list
templist = ['Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Clinical_Including_Motor_Complications' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))

### Clinical cognition

Including cognition

In [ ]:
# Formal criteria list
templist = ['Clinical - Cognition (memory, attention, executive, visuospatial memory etc)']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Clinical_Including_Cognition' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

Only cognition

In [ ]:
# Preparatory settings
tempvar = 'Clinical - Cognition (memory, attention, executive, visuospatial memory etc)'
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Clinical_Only_Cognition' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# Create a mask where the condition is True if the column matches tempvar exactly, False otherwise
mask = df[colpos] == tempvar

# Convert the Boolean mask to integers (True becomes 1, False becomes 0)
newcol = mask.astype(int)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))
df[['Article Title',colpos,newcolname]].head(5)

### Clinical non motor

Including cognition

In [ ]:
# Formal criteria list
templist = ['Clinical - Cognition (memory, attention, executive, visuospatial memory etc)',
    'Clinical - Neuropsychiatric (anxiety, depression, impulsivity, delusions, hallucinations etc)',
    'Clinical - Sleep (REM sleep behavior disorder, insomnia, daytime sleepiness)',
    'Clinical - Anosmia or Hyposmia (absence or loss of the capability to smell)',
    'Clinical - Autonomic (gastroparesis, constipation, orthostatic hypotension, urinary incontinence, sexual difficulties)',
    'Clinical - Quality of Life',
    'Clinical - Other (pain, fatigue etc)']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Clinical_Any_Non_Motor_Including_Cognition' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

Excluding cognition

In [ ]:
# Formal criteria list
templist = ['Clinical - Neuropsychiatric (anxiety, depression, impulsivity, delusions, hallucinations etc)',
    'Clinical - Sleep (REM sleep behavior disorder, insomnia, daytime sleepiness)',
    'Clinical - Anosmia or Hyposmia (absence or loss of the capability to smell)',
    'Clinical - Autonomic (gastroparesis, constipation, orthostatic hypotension, urinary incontinence, sexual difficulties)',
    'Clinical - Quality of Life',
    'Clinical - Other (pain, fatigue etc)']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Clinical_Any_Non_Motor_Excluding_Cognition' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))

Excluding cognition, other and QoL

In [ ]:
# Formal criteria list
templist = ['Clinical - Neuropsychiatric (anxiety, depression, impulsivity, delusions, hallucinations etc)',
    'Clinical - Sleep (REM sleep behavior disorder, insomnia, daytime sleepiness)',
    'Clinical - Anosmia or Hyposmia (absence or loss of the capability to smell)',
    'Clinical - Autonomic (gastroparesis, constipation, orthostatic hypotension, urinary incontinence, sexual difficulties)',]

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Clinical_Any_Non_Motor_Excluding_Cognition_QoL_Other' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))

### Clinical neuropsychiatric

In [ ]:
# Formal criteria list
templist = ['Clinical - Neuropsychiatric (anxiety, depression, impulsivity, delusions, hallucinations etc)']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Clinical_Any_Neuropsychiatric' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Clinical sleep

In [ ]:
# Formal criteria list
templist = ['Clinical - Sleep (REM sleep behavior disorder, insomnia, daytime sleepiness)']


# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Clinical_Any_Sleep' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Clinical olfaction

In [ ]:
# Formal criteria list
templist = ['Clinical - Anosmia or Hyposmia (absence or loss of the capability to smell)']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Clinical_Any_Olfaction' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Clinical autonomic

In [ ]:
# Formal criteria list
templist = ['Clinical - Autonomic (gastroparesis, constipation, orthostatic hypotension, urinary incontinence, sexual difficulties)']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Clinical_Any_Autonomic' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Clinical other

Quality of life

In [ ]:
# Formal criteria list
templist = ['Clinical - Quality of Life']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Clinical_QoL' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

Other non-motor

In [ ]:
# Formal criteria list
templist = ['Clinical - Other (pain, fatigue etc)']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Clinical_Non_Motor_Other' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))

### Clinical other motor

All united

In [ ]:
# Formal criteria list
templist = ['Clinical - Vital Signs or Neurologic Examination Findings',
            'Clinical - Phonation',
            'Clinical - Motor Tests (example: Timed-Up & Go)']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Clinical_Other_Motor' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

Vital signs of examination

In [ ]:
# Formal criteria list
templist = ['Clinical - Vital Signs or Neurologic Examination Findings']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Clinical_Vital_or_Neurologic_Examination' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))

Phonation

In [ ]:
# Formal criteria list
templist = ['Clinical - Phonation']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Clinical_Phonation' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))

Motor Tests (example: Timed-Up & Go)

In [ ]:
# Formal criteria list
templist = ['Clinical - Motor Tests (example: Timed-Up & Go)']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Clinical_Motor_Tests' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))

## Counting domains

For usage associated with Mestre article

In [ ]:
clusteringlist = ['Clustering_Clinical_Any_Motor_Scales_No_Complications',
                  'Clustering_Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)',
                  'Clustering_Clinical - Motor Features Obtained Through Wearable Sensors or other Technical Instruments',
                  'Clustering_Clinical_Other_Motor',
                  'Clustering_Any_Demographic',
                  'Clustering_Any_Time_or_Staging_Measurements',
                  'Clustering_Clinical_Including_Cognition',
                  'Clustering_Clinical_Any_Neuropsychiatric',
                  'Clustering_Clinical_Any_Sleep',
                  'Clustering_Clinical_Any_Olfaction',
                  'Clustering_Clinical_Any_Autonomic',
                  'Clustering_Clinical - Medication (types and/or dosages)',
                  'Clustering_Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)',
                  'Clustering_Any_Neuroimaging', 'Clustering_Any_Neurophysiology', 'Clustering_Any_Omics']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Number_of_Specific_Domains' # Name of the new column
newcolname2 = 'Clustering_More_Than_One_Domain_Mestre_Division' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df[clusteringlist].apply(lambda x: x.sum(), axis=1)

# New column to be created - classification according to mestre
criteria = [1] # Number of features

# Creating mestre division
newcol2 = np.where(
    newcol.isin(criteria),
    0, 1)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
  df.insert(insert_position, newcolname2, newcol2) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns
  df[newcolname2] = newcol2 # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
  relevantvars.append(newcolname2)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))
print(df[newcolname2].value_counts(dropna=False))

In [ ]:
clusteringlist = ['Clustering_Clinical_Any_Motor_Scales_No_Complications',
                  'Clustering_Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)',
                  'Clustering_Clinical - Motor Features Obtained Through Wearable Sensors or other Technical Instruments',
                  'Clustering_Clinical_Other_Motor',
                  'Clustering_Any_Demographic',
                  'Clustering_Any_Time_or_Staging_Measurements',
                  'Clustering_Clinical_Including_Cognition',
                  'Clustering_Clinical_Any_Neuropsychiatric',
                  'Clustering_Clinical_Any_Sleep',
                  'Clustering_Clinical_Any_Olfaction',
                  'Clustering_Clinical_Any_Autonomic',
                  'Clustering_Clinical - Medication (types and/or dosages)']

# Preparatory settings
colpos = 'Which type data was utilized for the clustering algorithm?' # Analysis column and position
newcolname = 'Clustering_Number_of_Specific_Clinical_Domains' # Name of the new column
newcolname2 = 'Clustering_More_Than_One_Clinical_Domain_Mestre_Division' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df[clusteringlist].apply(lambda x: x.sum(), axis=1)

# New column to be created - classification according to mestre
criteria = [1] # Number of features

# Creating mestre division
newcol2 = np.where(
    newcol.isin(criteria),
    0, 1)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
  df.insert(insert_position, newcolname2, newcol2) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns
  df[newcolname2] = newcol2 # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
  relevantvars.append(newcolname2)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))
print(df[newcolname2].value_counts(dropna=False))

# Creating validation variables

## General division

In [ ]:
# Defining the different options for answers
siglas = ['The study did not provide association / interpretation analyses',
    'Clinical - Vital Signs or Neurologic Examination Findings',
    'Clinical - Age',
    'Clinical - Gender',
    'Clinical - Ethnicity',
    'Clinical - Age at disease onset',
    'Clinical - Disease staging',
    'Clinical - Motor Features Obtained Through Rating Scales (tremor)',
    'Clinical - Motor Features Obtained Through Rating Scales (bradykinesia)',
    'Clinical - Motor Features Obtained Through Rating Scales (rigidity)',
    'Clinical - Motor Features Obtained Through Rating Scales (gait and/or posture)',
    'Clinical - Motor Features Obtained Through Rating Scales (speech)',
    'Clinical - Motor Features Obtained Through Wearable Sensors or other Technical Instruments',
    'Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)',
    'Clinical - Cognition (memory, attention, executive, visuospatial memory etc)',
    'Clinical - Neuropsychiatric (anxiety, depression, impulsivity, delusions, hallucinations etc)',
    'Clinical - Sleep (REM sleep behavior disorder, insomnia, daytime sleepiness)',
    'Clinical - Anosmia or Hyposmia (absence or loss of the capability to smell)',
    'Clinical - Autonomic (gastroparesis, constipation, orthostatic hypotension, urinary incontinence, sexual difficulties)',
    'Clinical - Quality of Life',
    'Clinical - Medication (types and/or dosages)',
    'Clinical - Other (pain, fatigue etc)',
    'Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)',
    'Neuroimaging - Structural or Volumetric MRI',
    'Neuroimaging - Diffusion Imaging MRI',
    'Neuroimaging - DaTSCAN',
    'Neuroimaging - Radiomics',
    'Neuroimaging - Functional MRI',
    'Neuroimaging - Other Methods (e.g. neuromelanin-sensitive MRI)',
    'Neurophysiology - Electroencephalography',
    'Neurophysiology - Electroneuromiography',
    'Neurophysiology - Other',
    'Omics - Proteomics',
    'Omics - Transcriptomics',
    'Omics - Genomics',
    'Omics - Lipidomics',
    'Omics - Other (e.g. metabolomics, metagenomics)',
    'Clinical - Phonation',
    'Clinical - Motor Tests (example: Timed-Up & Go)']

# Preparing to run the divisor function
main_column = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?'
significado = siglas

# Dividing in new columns with 0 or 1
df = divisor(df, colunaprincipal = main_column, siglas = siglas,
                            significado = significado, apagar=False, numerical=True)

# Renaming to identify they are from the clustering group
validation_names = []
for value in siglas:
  validation_names.append(str('Validation_')+value)

column_mapping = dict(zip(siglas, validation_names))

# Rename columns in the DataFrame using the column_mapping dictionary
if 'Validation_Clinical - Age' not in df.columns:
  df.rename(columns=column_mapping, inplace=True)
  relevantvars = relevantvars + validation_names
else:
  print('Already added and renamed')

# Viewing the distribution of responses
calculate_percentage_distribution(df, validation_names)

In [ ]:
# Checking if additional and relevant data was lost
df['Validation_Sum'] = df[validation_names].sum(axis=1)

# Filter rows where 'Cluster_Sum' is 0, indicating no siglas were found in these rows
df[df['Validation_Sum'] == 0]

## Any data subtypes

Identifying big data subtypes

### Clinical

In [ ]:
# Formal criteria list
templist = [
    'Clinical - Vital Signs or Neurologic Examination Findings',
    'Clinical - Age',
    'Clinical - Gender',
    'Clinical - Ethnicity',
    'Clinical - Age at disease onset',
    'Clinical - Disease staging',
    'Clinical - Motor Features Obtained Through Rating Scales (tremor)',
    'Clinical - Motor Features Obtained Through Rating Scales (bradykinesia)',
    'Clinical - Motor Features Obtained Through Rating Scales (rigidity)',
    'Clinical - Motor Features Obtained Through Rating Scales (gait and/or posture)',
    'Clinical - Motor Features Obtained Through Rating Scales (speech)',
    'Clinical - Motor Features Obtained Through Wearable Sensors or other Technical Instruments',
    'Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)',
    'Clinical - Cognition (memory, attention, executive, visuospatial memory etc)',
    'Clinical - Neuropsychiatric (anxiety, depression, impulsivity, delusions, hallucinations etc)',
    'Clinical - Sleep (REM sleep behavior disorder, insomnia, daytime sleepiness)',
    'Clinical - Anosmia or Hyposmia (absence or loss of the capability to smell)',
    'Clinical - Autonomic (gastroparesis, constipation, orthostatic hypotension, urinary incontinence, sexual difficulties)',
    'Clinical - Quality of Life',
    'Clinical - Medication (types and/or dosages)',
    'Clinical - Other (pain, fatigue etc)',
    'Clinical - Phonation',
    'Clinical - Motor Tests (example: Timed-Up & Go)']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Any_Clinical' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Non Clinical

In [ ]:
# Formal criteria list
templist = ['Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)',
    'Neuroimaging - Structural or Volumetric MRI',
    'Neuroimaging - Diffusion Imaging MRI',
    'Neuroimaging - DaTSCAN',
    'Neuroimaging - Radiomics',
    'Neuroimaging - Functional MRI',
    'Neuroimaging - Other Methods (e.g. neuromelanin-sensitive MRI)',
    'Neurophysiology - Electroencephalography',
    'Neurophysiology - Electroneuromiography',
    'Neurophysiology - Other',
    'Omics - Proteomics',
    'Omics - Transcriptomics',
    'Omics - Genomics',
    'Omics - Lipidomics',
    'Omics - Other (e.g. metabolomics, metagenomics)']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Any_Non_Clinical' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Biomarkers

Check variable 'Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)'

### Neuroimaging

In [ ]:
# Formal criteria list
templist = ['Neuroimaging - Structural or Volumetric MRI',
    'Neuroimaging - Diffusion Imaging MRI',
    'Neuroimaging - DaTSCAN',
    'Neuroimaging - Radiomics',
    'Neuroimaging - Functional MRI',
    'Neuroimaging - Other Methods (e.g. neuromelanin-sensitive MRI)']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Any_Neuroimaging' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Neurophysiology

In [ ]:
# Formal criteria list
templist = ['Neurophysiology - Electroencephalography',
    'Neurophysiology - Electroneuromiography',
    'Neurophysiology - Other']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Any_Neurophysiology' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Omics

In [ ]:
# Formal criteria list
templist = ['Omics - Proteomics',
    'Omics - Transcriptomics',
    'Omics - Genomics',
    'Omics - Lipidomics',
    'Omics - Other (e.g. metabolomics, metagenomics)']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Any_Omics' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Omics or Biomarkers

In [ ]:
# Formal criteria list
templist = ['Omics - Proteomics',
    'Omics - Transcriptomics',
    'Omics - Genomics',
    'Omics - Lipidomics',
    'Omics - Other (e.g. metabolomics, metagenomics)',
            'Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Any_Omics_or_Biomarkers' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

## Specific data subtypes

### Demographic

In [ ]:
# Formal criteria list
templist = ['Clinical - Age',
    'Clinical - Gender',
    'Clinical - Ethnicity']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Any_Demographic' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Disease staging or age at onset

In [ ]:
# Formal criteria list
templist = ['Clinical - Age at disease onset','Clinical - Disease staging']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Any_Time_or_Staging_Measurements' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Clinical motor scales

Motor clinical data obtained from rating scales

Including complications

In [ ]:
# Formal criteria list
templist = ['Clinical - Motor Features Obtained Through Rating Scales (tremor)',
    'Clinical - Motor Features Obtained Through Rating Scales (bradykinesia)',
    'Clinical - Motor Features Obtained Through Rating Scales (rigidity)',
    'Clinical - Motor Features Obtained Through Rating Scales (gait and/or posture)',
    'Clinical - Motor Features Obtained Through Rating Scales (speech)',
    'Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Clinical_Any_Motor_Scales_Plus_Complications' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

Excluding complications

In [ ]:
# Formal criteria list
templist = ['Clinical - Motor Features Obtained Through Rating Scales (tremor)',
    'Clinical - Motor Features Obtained Through Rating Scales (bradykinesia)',
    'Clinical - Motor Features Obtained Through Rating Scales (rigidity)',
    'Clinical - Motor Features Obtained Through Rating Scales (gait and/or posture)',
    'Clinical - Motor Features Obtained Through Rating Scales (speech)']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Clinical_Any_Motor_Scales_No_Complications' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))

Including only complications

In [ ]:
# Formal criteria list
templist = ['Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Clinical_Including_Motor_Complications' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))

### Clinical cognition

Including cognition

In [ ]:
# Formal criteria list
templist = ['Clinical - Cognition (memory, attention, executive, visuospatial memory etc)']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Clinical_Including_Cognition' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

Only cognition

In [ ]:
# Preparatory settings
tempvar = 'Clinical - Cognition (memory, attention, executive, visuospatial memory etc)'
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Clinical_Only_Cognition' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# Create a mask where the condition is True if the column matches tempvar exactly, False otherwise
mask = df[colpos] == tempvar

# Convert the Boolean mask to integers (True becomes 1, False becomes 0)
newcol = mask.astype(int)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))
df[['Article Title',colpos,newcolname]].head(5)

### Clinical non motor

Including cognition

In [ ]:
# Formal criteria list
templist = ['Clinical - Cognition (memory, attention, executive, visuospatial memory etc)',
    'Clinical - Neuropsychiatric (anxiety, depression, impulsivity, delusions, hallucinations etc)',
    'Clinical - Sleep (REM sleep behavior disorder, insomnia, daytime sleepiness)',
    'Clinical - Anosmia or Hyposmia (absence or loss of the capability to smell)',
    'Clinical - Autonomic (gastroparesis, constipation, orthostatic hypotension, urinary incontinence, sexual difficulties)',
    'Clinical - Quality of Life',
    'Clinical - Other (pain, fatigue etc)']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Clinical_Any_Non_Motor_Including_Cognition' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

Excluding cognition

In [ ]:
# Formal criteria list
templist = ['Clinical - Neuropsychiatric (anxiety, depression, impulsivity, delusions, hallucinations etc)',
    'Clinical - Sleep (REM sleep behavior disorder, insomnia, daytime sleepiness)',
    'Clinical - Anosmia or Hyposmia (absence or loss of the capability to smell)',
    'Clinical - Autonomic (gastroparesis, constipation, orthostatic hypotension, urinary incontinence, sexual difficulties)',
    'Clinical - Quality of Life',
    'Clinical - Other (pain, fatigue etc)']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Clinical_Any_Non_Motor_Excluding_Cognition' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))

Excluding cognition, other and QoL

In [ ]:
# Formal criteria list
templist = ['Clinical - Neuropsychiatric (anxiety, depression, impulsivity, delusions, hallucinations etc)',
    'Clinical - Sleep (REM sleep behavior disorder, insomnia, daytime sleepiness)',
    'Clinical - Anosmia or Hyposmia (absence or loss of the capability to smell)',
    'Clinical - Autonomic (gastroparesis, constipation, orthostatic hypotension, urinary incontinence, sexual difficulties)',]

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Clinical_Any_Non_Motor_Excluding_Cognition_QoL_Other' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))

### Clinical neuropsychiatric

In [ ]:
# Formal criteria list
templist = ['Clinical - Neuropsychiatric (anxiety, depression, impulsivity, delusions, hallucinations etc)']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Clinical_Any_Neuropsychiatric' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Clinical sleep

In [ ]:
# Formal criteria list
templist = ['Clinical - Sleep (REM sleep behavior disorder, insomnia, daytime sleepiness)']


# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Clinical_Any_Sleep' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Clinical olfaction

In [ ]:
# Formal criteria list
templist = ['Clinical - Anosmia or Hyposmia (absence or loss of the capability to smell)']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Clinical_Any_Olfaction' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Clinical autonomic

In [ ]:
# Formal criteria list
templist = ['Clinical - Autonomic (gastroparesis, constipation, orthostatic hypotension, urinary incontinence, sexual difficulties)']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Clinical_Any_Autonomic' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

### Clinical other

Quality of life

In [ ]:
# Formal criteria list
templist = ['Clinical - Quality of Life']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Clinical_QoL' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

Other non-motor

In [ ]:
# Formal criteria list
templist = ['Clinical - Other (pain, fatigue etc)']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Clinical_Non_Motor_Other' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))

### Clinical other motor

All united

In [ ]:
# Formal criteria list
templist = ['Clinical - Vital Signs or Neurologic Examination Findings',
            'Clinical - Phonation',
            'Clinical - Motor Tests (example: Timed-Up & Go)']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Clinical_Other_Motor' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))

Vital signs of examination

In [ ]:
# Formal criteria list
templist = ['Clinical - Vital Signs or Neurologic Examination Findings']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Clinical_Vital_or_Neurologic_Examination' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))

Phonation

In [ ]:
# Formal criteria list
templist = ['Clinical - Phonation']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Clinical_Phonation' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))

Motor Tests (example: Timed-Up & Go)

In [ ]:
# Formal criteria list
templist = ['Clinical - Motor Tests (example: Timed-Up & Go)']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Clinical_Motor_Tests' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = parse_description(templist, all=False)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

print(df[newcolname].value_counts(dropna=False))

## Counting domains

Based on Mestre definition of post hoc

In [ ]:
validationlist = ['Validation_Clinical_Any_Motor_Scales_No_Complications',
                  'Validation_Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)',
                  'Validation_Clinical - Motor Features Obtained Through Wearable Sensors or other Technical Instruments',
                  'Validation_Clinical_Other_Motor',
                  'Validation_Any_Demographic',
                  'Validation_Any_Time_or_Staging_Measurements',
                  'Validation_Clinical_Including_Cognition',
                  'Validation_Clinical_Any_Neuropsychiatric',
                  'Validation_Clinical_Any_Sleep',
                  'Validation_Clinical_Any_Olfaction',
                  'Validation_Clinical_Any_Autonomic',
                  'Validation_Clinical - Medication (types and/or dosages)',
                  'Validation_Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)',
                  'Validation_Any_Neuroimaging', 'Validation_Any_Neurophysiology', 'Validation_Any_Omics']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Number_of_Specific_Domains' # Name of the new column
newcolname2 = 'Validation_More_Than_One_Domain_Mestre_Division' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df[validationlist].apply(lambda x: x.sum(), axis=1)

# Define the criteria clearly
criteria_zero = [0]  # No domains
criteria_one = [1]  # Exactly one domain

# Calculate the number of validation areas each record pertains to
newcol = df[validationlist].apply(lambda x: x.sum(), axis=1)

# Assign categories based on the count of domains
newcol2 = np.where(
    newcol.isin(criteria_zero),
    0,  # No domains
    np.where(
        newcol.isin(criteria_one),
        1,  # Exactly one domain
        2  # More than one domain
    )
)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
  df.insert(insert_position, newcolname2, newcol2) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns
  df[newcolname2] = newcol2 # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
  relevantvars.append(newcolname2)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))
print(df[newcolname2].value_counts(dropna=False))

In [ ]:
validationlist = ['Validation_Clinical_Any_Motor_Scales_No_Complications',
                  'Validation_Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)',
                  'Validation_Clinical - Motor Features Obtained Through Wearable Sensors or other Technical Instruments',
                  'Validation_Clinical_Other_Motor',
                  'Validation_Any_Demographic',
                  'Validation_Any_Time_or_Staging_Measurements',
                  'Validation_Clinical_Including_Cognition',
                  'Validation_Clinical_Any_Neuropsychiatric',
                  'Validation_Clinical_Any_Sleep',
                  'Validation_Clinical_Any_Olfaction',
                  'Validation_Clinical_Any_Autonomic',
                  'Validation_Clinical - Medication (types and/or dosages)']

# Preparatory settings
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?' # Analysis column and position
newcolname = 'Validation_Number_of_Specific_Clinical_Domains' # Name of the new column
newcolname2 = 'Validation_More_Than_One_Clinical_Domain_Mestre_Division' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df[validationlist].apply(lambda x: x.sum(), axis=1)

# Define the criteria clearly
criteria_zero = [0]  # No domains
criteria_one = [1]  # Exactly one domain

# Calculate the number of validation areas each record pertains to
newcol = df[validationlist].apply(lambda x: x.sum(), axis=1)

# Assign categories based on the count of domains
newcol2 = np.where(
    newcol.isin(criteria_zero),
    0,  # No domains
    np.where(
        newcol.isin(criteria_one),
        1,  # Exactly one domain
        2  # More than one domain
    )
)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
  df.insert(insert_position, newcolname2, newcol2) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns
  df[newcolname2] = newcol2 # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
  relevantvars.append(newcolname2)
else:
  print('Already added')

print(df[newcolname].value_counts(dropna=False))
print(df[newcolname2].value_counts(dropna=False))

# Additional Variable Creation

In [ ]:
# Reviewing the dataframe
df.head(2)

In [ ]:
df.columns[75:95]

## Clustering and Validation Criteria

### General Group

Clinical x Biomarker x Neuroimaging x Neurophisiology x Omics

In [ ]:
clusteringlist = ['Clustering_Any_Clinical', 'Clustering_Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)',
                  'Clustering_Any_Neuroimaging', 'Clustering_Any_Neurophysiology', 'Clustering_Any_Omics']

validationlist = ['Validation_Any_Clinical', 'Validation_Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)',
                  'Validation_Any_Neuroimaging', 'Validation_Any_Neurophysiology', 'Validation_Any_Omics']

# Calculate differences
newcol = sum((df[validate] == 1) & (df[cluster] == 0) for cluster, validate in zip(clusteringlist, validationlist))
newcol2 = newcol.apply(lambda x: 1 if x >= 1 else 0) # Binarize if used another domain or not

# Creating colnames
newcolname = 'General_Cluster_Validation_Differences_Sum'
newcolname2 = 'General_Cluster_Validation_Differences_Yes_or_No'
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?'
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
  df.insert(insert_position, newcolname2, newcol2) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns
  df[newcolname2] = newcol2 # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
  relevantvars.append(newcolname2)
else:
  print(newcolname, 'Already added')

# Displaying the count of differences and a preview of the DataFrame
print(df['General_Cluster_Validation_Differences_Sum'].value_counts(dropna=False))
print(df['General_Cluster_Validation_Differences_Yes_or_No'].value_counts(dropna=False))
df[['Article Title'] + ['General_Cluster_Validation_Differences_Sum'] + ['General_Cluster_Validation_Differences_Yes_or_No'] + clusteringlist + validationlist].head(5)

### Specific Group

Clinical Subdivided x Biomarker x Neuroimaging x Neurophisiology x Omics

Probably this one will be utilized

**Includes wearable sensors**

In [ ]:
clusteringlist = ['Clustering_Any_Demographic',
                  'Clustering_Any_Time_or_Staging_Measurements','Clustering_Clinical_Any_Motor_Scales_No_Complications',
                  'Clustering_Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)',
                  'Clustering_Clinical - Motor Features Obtained Through Wearable Sensors or other Technical Instruments',
                  'Clustering_Clinical_Other_Motor',
                  'Clustering_Clinical_Including_Cognition',
                  'Clustering_Clinical_Any_Neuropsychiatric',
                  'Clustering_Clinical_Any_Sleep',
                  'Clustering_Clinical_Any_Olfaction',
                  'Clustering_Clinical_Any_Autonomic',
                  'Clustering_Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)',
                  'Clustering_Any_Neuroimaging', 'Clustering_Any_Neurophysiology', 'Clustering_Any_Omics']

validationlist = ['Validation_Any_Demographic',
                  'Validation_Any_Time_or_Staging_Measurements',
                  'Validation_Clinical_Any_Motor_Scales_No_Complications',
                  'Validation_Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)',
                  'Validation_Clinical - Motor Features Obtained Through Wearable Sensors or other Technical Instruments',
                  'Validation_Clinical_Other_Motor',
                  'Validation_Clinical_Including_Cognition',
                  'Validation_Clinical_Any_Neuropsychiatric',
                  'Validation_Clinical_Any_Sleep',
                  'Validation_Clinical_Any_Olfaction',
                  'Validation_Clinical_Any_Autonomic',
                  'Validation_Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)',
                  'Validation_Any_Neuroimaging', 'Validation_Any_Neurophysiology', 'Validation_Any_Omics']

# Non-motor - other was removed because I didn't think it was relevant enough

# Calculate differences
newcol = sum((df[validate] == 1) & (df[cluster] == 0) for cluster, validate in zip(clusteringlist, validationlist))
newcol2 = newcol.apply(lambda x: 1 if x >= 1 else 0) # Binarize if used another domain or not
newcol3 = newcol.apply(lambda x: 1 if x >= 2 else 0) # Binarize if used 2 or more other domains or not

# Creating colnames
newcolname = 'Specific_Cluster_Validation_Differences_Sum'
newcolname2 = 'Specific_Cluster_Validation_Differences_Yes_or_No'
newcolname3 = 'Specific_Cluster_Validation_Differences_2_or_More_Domains_Yes_or_No'
colpos = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?'
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
  df.insert(insert_position, newcolname2, newcol2) # Inserting the column
  df.insert(insert_position, newcolname3, newcol3) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns
  df[newcolname2] = newcol2 # Updating columns,
  df[newcolname3] = newcol3 # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
  relevantvars.append(newcolname2)
else:
  print(newcolname, 'Already added')

# Displaying the count of differences and a preview of the DataFrame
print(df[newcolname].value_counts(dropna=False))
print(df[newcolname2].value_counts(dropna=False))
print(df[newcolname3].value_counts(dropna=False))
df[['Article Title'] + ['Specific_Cluster_Validation_Differences_Sum'] + ['Specific_Cluster_Validation_Differences_Yes_or_No'] + clusteringlist + validationlist].head(5)

### Pairwise category creation

Identifies what categories of general group are present either in clustering or validation

#### Specific

##### Clinical

Demographic

In [ ]:
# Creating column if the study utilized either clustering or validation biomarkers
criteria = {
    'Clustering_Any_Demographic': lambda x: x == 1,
    'Validation_Any_Demographic': lambda x: x == 1,
}

# Quality check
print(calculate_percentage_distribution(df, list(criteria.keys()), use_counts=True))

# Preparatory settings
colpos = 'Validation_Any_Omics_or_Biomarkers' # Analysis column and position
newcolname = 'Utilized_Demographic_Either_in_Clustering_or_Validation' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df.apply(lambda row: any(criteria[col](row[col]) for col in criteria), axis=1).astype(int) # Who fills either criteria

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

Time based features

In [ ]:
# Creating column if the study utilized either clustering or validation biomarkers
criteria = {
    'Clustering_Any_Time_or_Staging_Measurements': lambda x: x == 1,
    'Validation_Any_Time_or_Staging_Measurements': lambda x: x == 1,
}

# Quality check
print(calculate_percentage_distribution(df, list(criteria.keys()), use_counts=True))

# Preparatory settings
colpos = 'Validation_Any_Omics_or_Biomarkers' # Analysis column and position
newcolname = 'Utilized_Time_or_Staging_Measurements_Either_in_Clustering_or_Validation' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df.apply(lambda row: any(criteria[col](row[col]) for col in criteria), axis=1).astype(int) # Who fills either criteria

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

Motor scales

In [ ]:
# Creating column if the study utilized either clustering or validation biomarkers
criteria = {
    'Clustering_Clinical_Any_Motor_Scales_No_Complications': lambda x: x == 1,
    'Validation_Clinical_Any_Motor_Scales_No_Complications': lambda x: x == 1,
}

# Quality check
print(calculate_percentage_distribution(df, list(criteria.keys()), use_counts=True))

# Preparatory settings
colpos = 'Validation_Any_Omics_or_Biomarkers' # Analysis column and position
newcolname = 'Utilized_Motor_Scales_no_Complications_Either_in_Clustering_or_Validation' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df.apply(lambda row: any(criteria[col](row[col]) for col in criteria), axis=1).astype(int) # Who fills either criteria

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

Motor complications

In [ ]:
# Creating column if the study utilized either clustering or validation biomarkers
criteria = {
    'Clustering_Clinical_Including_Motor_Complications': lambda x: x == 1,
    'Validation_Clinical_Including_Motor_Complications': lambda x: x == 1,
}

# Quality check
print(calculate_percentage_distribution(df, list(criteria.keys()), use_counts=True))

# Preparatory settings
colpos = 'Validation_Any_Omics_or_Biomarkers' # Analysis column and position
newcolname = 'Utilized_Motor_Complications_Either_in_Clustering_or_Validation' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df.apply(lambda row: any(criteria[col](row[col]) for col in criteria), axis=1).astype(int) # Who fills either criteria

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

Cognition

In [ ]:
# Creating column if the study utilized either clustering or validation biomarkers
criteria = {
    'Clustering_Clinical_Including_Cognition': lambda x: x == 1,
    'Validation_Clinical_Including_Cognition': lambda x: x == 1,
}

# Quality check
print(calculate_percentage_distribution(df, list(criteria.keys()), use_counts=True))

# Preparatory settings
colpos = 'Validation_Any_Omics_or_Biomarkers' # Analysis column and position
newcolname = 'Utilized_Cognition_Either_in_Clustering_or_Validation' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df.apply(lambda row: any(criteria[col](row[col]) for col in criteria), axis=1).astype(int) # Who fills either criteria

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

Non-motor (non cognition)

In [ ]:
# Creating column if the study utilized either clustering or validation biomarkers
criteria = {
    'Clustering_Clinical_Any_Non_Motor_Excluding_Cognition_QoL_Other': lambda x: x == 1,
    'Validation_Clinical_Any_Non_Motor_Excluding_Cognition_QoL_Other': lambda x: x == 1,
}

# Quality check
print(calculate_percentage_distribution(df, list(criteria.keys()), use_counts=True))

# Preparatory settings
colpos = 'Validation_Any_Omics_or_Biomarkers' # Analysis column and position
newcolname = 'Utilized_Non_Motor_Excluding_Cognition_QoL_Other_Either_in_Clustering_or_Validation' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df.apply(lambda row: any(criteria[col](row[col]) for col in criteria), axis=1).astype(int) # Who fills either criteria

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

##### Clinical special

In [ ]:
# Names of the columns to check against the elements in 'templist'
col1 = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?'
col2 = 'Which type data was utilized for the clustering algorithm?'

# List representing values that can be present in two different types of columns
templist = ['Clinical - Neuropsychiatric (anxiety, depression, impulsivity, delusions, hallucinations etc)',
    'Clinical - Sleep (REM sleep behavior disorder, insomnia, daytime sleepiness)',
    'Clinical - Anosmia or Hyposmia (absence or loss of the capability to smell)',
    'Clinical - Autonomic (gastroparesis, constipation, orthostatic hypotension, urinary incontinence, sexual difficulties)',
    'Clinical - Quality of Life',
    'Clinical - Other (pain, fatigue etc)',
 'Clinical - Vital Signs or Neurologic Examination Findings',
 'Clinical - Phonation',
 'Clinical - Motor Tests (example: Timed-Up & Go)']

# Function to create columns based on the presence of each item in 'templist'
for item in templist:
  tempname = item + '_Either_in_Clustering_or_Validation'
  # Create a new column for each item checking presence in either of the two columns
  df[tempname] = df.apply(lambda row: int(item in row[col1] or item in row[col2]), axis=1)

  # Adding this list of variables to the relevantvars
  if tempname not in relevantvars:
    relevantvars.append(tempname)
  else:
    print(tempname, 'Already added')

# Displaying the counts for one of the new columns to verify (replace the index for other items)
print(df['Clinical - Phonation_Either_in_Clustering_or_Validation'].value_counts(dropna=False))

##### Neuroimaging

In [ ]:
# Names of the columns to check against the elements in 'templist'
col1 = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?'
col2 = 'Which type data was utilized for the clustering algorithm?'

# List representing values that can be present in two different types of columns
templist = [
    'Neuroimaging - Structural or Volumetric MRI',
    'Neuroimaging - Diffusion Imaging MRI',
    'Neuroimaging - DaTSCAN',
    'Neuroimaging - Radiomics',
    'Neuroimaging - Functional MRI',
    'Neuroimaging - Other Methods (e.g. neuromelanin-sensitive MRI)'
]

# Function to create columns based on the presence of each item in 'templist'
for item in templist:
  tempname = item + '_Either_in_Clustering_or_Validation'
  # Create a new column for each item checking presence in either of the two columns
  df[tempname] = df.apply(lambda row: int(item in row[col1] or item in row[col2]), axis=1)

  # Adding this list of variables to the relevantvars
  if tempname not in relevantvars:
    relevantvars.append(tempname)
  else:
    print(tempname, 'Already added')

# Displaying the counts for one of the new columns to verify (replace the index for other items)
print(df['Neuroimaging - Structural or Volumetric MRI_Either_in_Clustering_or_Validation'].value_counts(dropna=False))

##### Omics

In [ ]:
# Names of the columns to check against the elements in 'templist'
col1 = 'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?'
col2 = 'Which type data was utilized for the clustering algorithm?'

# List representing values that can be present in two different types of columns
templist = ['Omics - Proteomics',
    'Omics - Transcriptomics',
    'Omics - Genomics',
    'Omics - Lipidomics',
    'Omics - Other (e.g. metabolomics, metagenomics)']

# Function to create columns based on the presence of each item in 'templist'
for item in templist:
  tempname = item + '_Either_in_Clustering_or_Validation'
  # Create a new column for each item checking presence in either of the two columns
  df[tempname] = df.apply(lambda row: int(item in row[col1] or item in row[col2]), axis=1)
  # Adding this list of variables to the relevantvars
  if tempname not in relevantvars:
    relevantvars.append(tempname)
  else:
    print(tempname, 'Already added')

# Displaying the counts for one of the new columns to verify (replace the index for other items)
print(df['Omics - Genomics_Either_in_Clustering_or_Validation'].value_counts(dropna=False))

#### General

Clinical

In [ ]:
# Creating column if the study utilized either clustering or validation biomarkers
criteria = {
    'Clustering_Any_Clinical': lambda x: x == 1,
    'Validation_Any_Clinical': lambda x: x == 1,
}

# Preparatory settings
colpos = 'Validation_Any_Omics_or_Biomarkers' # Analysis column and position
newcolname = 'Utilized_Clinical_Either_in_Clustering_or_Validation' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df.apply(lambda row: any(criteria[col](row[col]) for col in criteria), axis=1).astype(int) # Who fills either criteria

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

Non Clinical

In [ ]:
# Creating column if the study utilized either clustering or validation biomarkers
criteria = {
    'Clustering_Any_Non_Clinical': lambda x: x == 1,
    'Validation_Any_Non_Clinical': lambda x: x == 1,
}

# Preparatory settings
colpos = 'Validation_Any_Omics_or_Biomarkers' # Analysis column and position
newcolname = 'Utilized_Any_Non_Clinical_Either_in_Clustering_or_Validation' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df.apply(lambda row: any(criteria[col](row[col]) for col in criteria), axis=1).astype(int) # Who fills either criteria

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

Biomarkers

In [ ]:
# Creating column if the study utilized either clustering or validation biomarkers
criteria = {
    'Clustering_Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)': lambda x: x == 1,
    'Validation_Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)': lambda x: x == 1,
}

# Preparatory settings
colpos = 'Validation_Any_Omics_or_Biomarkers' # Analysis column and position
newcolname = 'Utilized_Any_Biomarkers_Either_in_Clustering_or_Validation' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df.apply(lambda row: any(criteria[col](row[col]) for col in criteria), axis=1).astype(int) # Who fills either criteria

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

Neurophisiology

In [ ]:
# Creating column if the study utilized either clustering or validation biomarkers
criteria = {
    'Clustering_Any_Neurophysiology': lambda x: x == 1,
    'Validation_Any_Neurophysiology': lambda x: x == 1,
}

# Preparatory settings
colpos = 'Validation_Any_Omics_or_Biomarkers' # Analysis column and position
newcolname = 'Utilized_Any_Neurophisiology_Either_in_Clustering_or_Validation' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df.apply(lambda row: any(criteria[col](row[col]) for col in criteria), axis=1).astype(int) # Who fills either criteria

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

Neuroimaging

In [ ]:
# Creating column if the study utilized either clustering or validation biomarkers
criteria = {
    'Clustering_Any_Neuroimaging': lambda x: x == 1,
    'Validation_Any_Neuroimaging': lambda x: x == 1,
}

# Preparatory settings
colpos = 'Validation_Any_Omics_or_Biomarkers' # Analysis column and position
newcolname = 'Utilized_Any_Neuroimaging_Either_in_Clustering_or_Validation' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df.apply(lambda row: any(criteria[col](row[col]) for col in criteria), axis=1).astype(int) # Who fills either criteria

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

Omics

In [ ]:
# Creating column if the study utilized either clustering or validation biomarkers
criteria = {
    'Clustering_Any_Omics': lambda x: x == 1,
    'Validation_Any_Omics': lambda x: x == 1,
}

# Preparatory settings
colpos = 'Validation_Any_Omics_or_Biomarkers' # Analysis column and position
newcolname = 'Utilized_Any_Omics_Either_in_Clustering_or_Validation' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df.apply(lambda row: any(criteria[col](row[col]) for col in criteria), axis=1).astype(int) # Who fills either criteria

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

## Rating scales used

I will list here only the most significant and common ones

In [ ]:
# Checking
df['If clinical data obtained from rating scales was used for clustering purposes, which data and rating scales were utilized?'].value_counts()[0:5]

In [ ]:
# Correcting some data - unifying Hoehn & Yahr
df['If clinical data obtained from rating scales was used for clustering purposes, which data and rating scales were utilized?'] = df['If clinical data obtained from rating scales was used for clustering purposes, which data and rating scales were utilized?'].str.replace('The Hoehn & Yahr Scale (or the modified Hoehn & Yahr)',
                                                                                         'Hoehn and Yahr Scale or Modified Hoehn and Yahr Scale', regex=False)

# STAI
df['If clinical data obtained from rating scales was used for clustering purposes, which data and rating scales were utilized?'] = df['If clinical data obtained from rating scales was used for clustering purposes, which data and rating scales were utilized?'].str.replace('State-Trait Anxiety Inventory for Adults',
                                                                                         'State-Trait Anxiety Inventory (STAI)', regex=False)


In [ ]:
# Defining the different options for answers
siglas = ['UPDRS motor scores','UPDRS non-motor scores',
          'MDS-UPDRS Part I', 'MDS-UPDRS Part II', 'MDS-UPDRS Part III', 'MDS-UPDRS Part IV',
          'Hoehn and Yahr Scale or Modified Hoehn and Yahr Scale', 'University of Pennsylvania Smell Identification Test (UPSIT)',
          'State-Trait Anxiety Inventory for Adults (STAI)', 'Non-Motor Symptoms Scale (NNMS)', 'Non-Motor Symptoms Questionnaire (NMSQ)',
          'Montreal Cognitive Assessment (MoCA)', 'Mini-Mental State Examination (MMSE)', 'Semantic Fluency Test (e.g. fruits, objects, names etc)',
          'SCOPA-AUT', 'The Schwab and England ADL (Activities of Daily Living) scale', 'Geriatric Depression Scale (GDS)', 'Beck Depression Inventory (BDI)',
          'Hospital Anxiety and Depression Scale (HADS)', 'Questionnaire for Impulsive-Compulsive Disorders in Parkinson’s Disease (QUIP)',
          'REM-Sleep Behavior Disorder Screening Questionnaire (RBDSQ)', 'Epworth Sleepiness Scale (ESS)']

ratingscales = siglas

# Preparing to run the function
main_column = 'If clinical data obtained from rating scales was used for clustering purposes, which data and rating scales were utilized?'
significado = siglas

# Dividing in new columns with 0 or 1
df = divisor(df, colunaprincipal = main_column, siglas = siglas,
                            significado = significado, apagar=False, numerical=True)

# Adding this list of variables to the relevantvars
if siglas[0] not in relevantvars:
  relevantvars = relevantvars + siglas
else:
  print('Already added')

# Viewing the distribution of responses
calculate_percentage_distribution(df, siglas)

## Neuroimaging details

In [ ]:
df.columns[75:95]

### Quality control

In [ ]:
# Defining column of interest
colpos = 'If neuroimaging data was utilized for clustering, did the article perform any sort of quality control measure?' # Analysis column and position

# Checking
df[colpos].value_counts(dropna=False)

In [ ]:
# Formal criteria list
criteria = ['Neuroimaging data was utilized for clustering purposes and a quality control was performed manually',
            'Neuroimaging data was utilized for clustering purposes but do they do not require quality control (example: DATScan)']

# Preparatory settings
newcolname = 'Neuroimaging_QC_Performed_or_Not_Needed' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(df[colpos]), np.nan,  # Keep NaN as NaN
    np.where(df[colpos].isin(criteria), 0, 1)  # Apply 0 or 1 based on criteria
)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

### Individual features

Just adding to relevantvars

In [ ]:
siglas = [ 'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [MRI cortical thickness]',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [MRI subcortical volume]',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [MRI white matter lesions / hyperintensities]',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [Functional MRI]',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [MRI Others]',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [PET]',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [SPECT DAT binding ratios]']

neuroimagingvars = siglas

for value in siglas:
  if value not in relevantvars:
    relevantvars.append(value)
  else:
    print('Already added')
    break

In [ ]:
calculate_percentage_distribution(df, columns = siglas)

## Number of subtypes

Subtyping strategies that aren't conventional are labelled NaN por .describe reasons, but needs to be reported as they are

In [ ]:
# Defining column of interest
colpos = 'How many subtypes did the evaluated study identified?' # Analysis column and position

# Checking
df[colpos].value_counts(dropna=False)

In [ ]:
df[colpos].describe()

In [ ]:
# Formal criteria list
criteria = ['two-cluster (2 subtypes);  three-cluster (3 subtypes)',
            '8 multi-partitition clusters, each only, respectively, with: 2; 2; 3; 2; 2; 2; 2; 2',
            'Domains Clustering - 4 clusters; Symptoms Clustering - 6 clusters ']

# Preparatory settings
newcolname = 'ClustersNeuroimaging_QC_Performed_or_Not_Needed' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(df[colpos]), np.nan,  # Keep NaN as NaN
    np.where(df[colpos].isin(criteria), 0, 1)  # Apply 0 or 1 based on criteria
)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

## Subtype stability

In [ ]:
# Identifying those studies that claim that the found subtypes are stable
print(df['Subtype stability'].value_counts())

In [ ]:
# Checking studies - all need to be longitudinal to prove they are stable
df[df['Subtype stability'] == 'Stable'][['Article Title','Longitudinal_Followup_Status','Subtype stability']]

In [ ]:
# Formal criteria list
criteria = ['Stable']

# Preparatory settings
colpos = 'Subtype stability'
newcolname = 'Subtype_Stability_Mestre_Definition' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(df[colpos]), np.nan,  # Keep NaN as NaN
    np.where(df[colpos].isin(criteria), 1, 0))

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

## **Statistical methods - REMOVED**

## Algorithm for classifying individual patients

In [ ]:
# Defining column of interest
colpos = 'Did the article provide an individual level prediction algorithm?' # Analysis column and position

# Checking
df[colpos].value_counts(dropna=False)

In [ ]:
# Formal criteria list
criteria = ['Yes']

# Preparatory settings
newcolname = 'Individual_Classification_Algorithm' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(df[colpos]), np.nan,  # Keep NaN as NaN
    np.where(df[colpos].isin(criteria), 1, 0))

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

## Year Divisions

### Quantile division

Divided in 4 quartiles

In [ ]:
# Dividing by quartiles - needed for all
df, names = quantis_single_column(df, ['Year of publication'], divisoes = 3, show_ranges=True)

# Adding this list of variables to the relevantvars
if 'Year of publication Quantile Division' not in relevantvars:
  relevantvars.append('Year of publication Quantile Division')
else:
  print('Year of publication Quantile Division', 'Already added')

print(df[names].value_counts())
df[names].head()

### Cut-off: 2021 (including)

In [ ]:
# Play analysis 2021 and after x before
criteria = [2020]

# Preparatory settings
colpos = 'Year of publication'
newcolname = 'Before_2020_After' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = np.where(
    pd.isna(df[colpos]), np.nan,  # Keep NaN as NaN
    np.where(
        df[colpos] <= criteria[0], 0, 1)).astype(str)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

# Special variable creation

## Systematic review of subtyping article from 2021

Quality score from Mestre article (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8150501/)

In [ ]:
# Formal criteria list
qualitylist = ['Similar_PD_Staging', # 1 = homogeneous staging or duration
               'Multicentric_X_Others', # 1 = multicentric
               'Recruitment_Community_or_Population', # 1 = community of population based
               'Formal_Criteria_From_Mestre', # 1 = formal criteria or neurologist | 2 = postmortem
               'Consecutive_or_Random_Sampling', # 1 = consecutive or random
               'Clustering_More_Than_One_Domain_Mestre_Division', # 1 = more than 1 domain
               'Validation_More_Than_One_Domain_Mestre_Division', # 1 = one domain | 2 = more than 1 domain
               'Longitudinal_Folloup_Years_Mestre_Definition', # 1 = Follow up for 1 to 3 years | 2 = Followup for more than 3 years
               'Longitudinal_Folloup_Completion_Mestre_Definition', # 1 = 50 - 75% followup rate | 2 = > 75% followup rate
               'Subtype_Stability_Mestre_Definition', # 1 = stable
               'Individual_Classification_Algorithm', # = provides an individual classification algorithm
               'Validation_not_Perfored_X_same_Cohort_X_Different_Cohort' # 1 = performed in the same cohort | 2 = performed in another cohort
]

# Checking if data is alright as expected
calculate_percentage_distribution(df, columns = qualitylist)

In [ ]:
# Preparatory settings
colpos = 'Article Title'
newcolname = 'Quality_Index_Mestre' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df[qualitylist].sum(axis=1)

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))
print(df[newcolname].describe())

## Transitioning from subtype to precisision medicine article

Taken from the recent precision medicine article: https://movementdisorders.onlinelibrary.wiley.com/doi/epdf/10.1002/mds.29708

### Purpose 1 = To predict disease progression for clinical care

In [ ]:
df[df['Validation_in_a_Different_Cohort'] == 1]

In [ ]:
newcolname = 'Longitudinal_Followup_Years'
newcolname2 = 'Completeness_of_Followup'

In [ ]:
# Requirements
criteria = {
    'Individual_Classification_Algorithm': lambda x: x == 1, # Provides individual level prediction
    'Longitudinal_Followup_Status': lambda x: x == 1, # Has a longitudinal follow-up
    'Longitudinal_Followup_Years': lambda x: x >= 3, # Needs an adequate follow-up time, 3 seems to be the minimum
    'Clustering_Number_of_Specific_Domains': lambda x: x > 1, # Needs to be multidimensional, 2 seems to be the bare minimum
    'Validation_Number_of_Specific_Domains': lambda x: x > 1, # Needs to be multidimensional, 2 seems to be the bare minimum
    'Number of PD patients utilized for clustering purposes': lambda x: x > 100, # Needs adequate sample size, 100 seems to be the minimum
    'Validation_in_a_Different_Cohort': lambda x: x == 1, # Validation performed in a different cohort
    'Minimum_Info_Age_Sex_Staging': lambda x: x == 1 # Validation performed in a different cohort
    }

calculate_percentage_distribution(df, columns = list(criteria.keys()), max_unique_values=12) # Viewing those criteria

In [ ]:
# Preparatory settings
colpos = 'Article Title' # Analysis column and position
newcolname = 'Follows_All_Criteria_of_Good_Subtyping_for_Disease_Prediction' # Name of the new column
newcolname2 = 'Number_of_Criteria_Followed_for_Good_Subtyping_for_Disease_Prediction' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df.apply(lambda row: all(criteria[col](row[col]) for col in criteria), axis=1) # Who fills all the criteria
newcol2 = df.apply(lambda row: sum(criteria[col](row[col]) for col in criteria), axis=1) # Number of criterias that are filled

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
  df.insert(insert_position, newcolname2, newcol2) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns
  df[newcolname2] = newcol2 # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
  relevantvars.append(newcolname2)
else:
  print(newcolname, 'Already added')
  print(newcolname2, 'Already added')

print(df[newcolname].value_counts(dropna=False))
print(df[newcolname2].value_counts(dropna=False))

Quick note - the article also mentions that lifestyle and genetic factors should be accounted for in subtyping, aswell as the ethnicity background

Refer to the section on "Minimum descriptive data"

In [ ]:
siglas = ['None of the mentioned data on the PD population was informed by the study',
           'Age', 'Gender', 'Ethnicity', 'Education', 'Income', 'Family History',
           'Age at disease onset', 'Disease duration (years) or staging (such as Hoehn & Yahr and Schawb and England scales)',
           'Vital Signs (blood pressure, cardiac frequency)', 'Physical Attributes (weight, height etc)',
           'Medication Doses', 'Genetic Status (monogenic PD, polygenic risk score)',
           'Summarised data from clinical scales (such as mean MDS-UPDRS scores, number of non-motor symptoms)',
           'Summarised data from neuroimaging studies (such as cortical thickness)',
           "Biomarker's profile (results from biomarker tests)"]

# Viewing the distribution of responses
calculate_percentage_distribution(df, siglas)

**On MRI:** Moreover, much is still unknown about themolecular, genetic, and protein structures that contributeto disease progression and how they interact, even in the“genetic”PD subtypes.3Magnetic resonance imaging (MRI) markers show some promise for their prognosticvalue for clinically relevant disease milestones

**On Clinical Trials:** Another comment is that clinical trials should collect data for post host analyses to be maade and to identify the found subtypes

### Purpose 2 = To Predict Response to Treatments in Clinical Practice

Needs to perform analyses on longitudinal medication usage

### Purpose 3 = To Identify Therapeutic Targets for Disease Modification

Good for extracting reference on our "biological" to "clinical" approach. Good for selection of studies focused only on omics data to see their qualities and quantities

In [ ]:
print(df['Clustering_Any_Omics_or_Biomarkers'].value_counts(dropna=False))
print(df['Validation_Any_Omics_or_Biomarkers'].value_counts(dropna=False))

In [ ]:
# Creating column if the study utilized either clustering or validation biomarkers
criteria = {
    'Clustering_Any_Omics_or_Biomarkers': lambda x: x == 1,
    'Validation_Any_Omics_or_Biomarkers': lambda x: x == 1,
}

# Preparatory settings
colpos = 'Validation_Any_Omics_or_Biomarkers' # Analysis column and position
newcolname = 'Utilized_Biomarkers_or_Omics_Either_in_Clustering_or_Validation' # Name of the new column
insert_position = df.columns.get_loc(colpos) + 1 # Calculating position

# New column to be created
newcol = df.apply(lambda row: any(criteria[col](row[col]) for col in criteria), axis=1).astype(int) # Who fills either criteria

# Inserting the column
try:
  df.insert(insert_position, newcolname, newcol) # Inserting the column
except ValueError:
  df[newcolname] = newcol # Updating columns

# Adding this list of variables to the relevantvars
if newcolname not in relevantvars:
  relevantvars.append(newcolname)
else:
  print(newcolname, 'Already added')

print(df[newcolname].value_counts(dropna=False))

df[['Clustering_Any_Omics_or_Biomarkers','Validation_Any_Omics_or_Biomarkers','Utilized_Biomarkers_or_Omics_Either_in_Clustering_or_Validation']].head()

### Other notes

 It is our recommendation that the
definition of “why, what, and where” becomes the
starting steps for any future research identifying or validating PD subtyping systems, complemented by the use
of methodological quality tools we have presented in
our earlier work.

One can question whether the various purposes identified in this paper could possibly be served by a single PD
subtyping system. The development of a single PD subtype
system serving different purposes would require significant
collaborative efforts and computational power with the
use of novel machine learning approaches, to combine
molecular data with rich clinical data in high-volume longitudinal datasets from a large variety of different sources
and with widely differing structures and qualities

However, existing datasets have limitations, including relatively
short follow-up biased toward initial and middle disease
stages, outcome assessment at fixed time points not
accounting for day-to-day variability, lack of non-clinical
biomarkers, little information on non-PD peripheral
or central changes (eg, cardiovascular risk,46 diabetes
mellitus,47,48 and compensatory factors), and bias
toward slowly progressive cases able to stay engaged

In our previous review of subtyping studies, we concluded that, with few exceptions, PD subtypes have not
been fruitful enough mainly due to lack of replication
and validation across cohorts/papers, the instability of
PD subtype membership, or a lack of deep phenotyping
with biomarker data in PD subtype development

Machine learning plays an important role in the delivery of individual-based knowledge,49 as it can analyze and cluster data in high-volume longitudinal datasets from a large variety of different sources and with widely differing structures and qualities. Depending on the purpose of the subtyping effort, these models may need to consider the temporal dynamics of clinical symptoms, biomarkers, and response to treatment. In this scenario, when reliable and accurate predictive modeling for individual patients is a reality, group-level predictions will become less relevant

# Export Table

In [ ]:
# Export the result
sheet_path = 'Tables/Data of the study.xlsx'
sheet_name = 'Data'

with pd.ExcelWriter(sheet_path, engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0)

    # Adjusting column width
    for column in df:
        column_width = max(df[column].astype(str).map(len).max(), len(column))
        col_idx = df.columns.get_loc(column)
        writer.sheets[sheet_name].set_column(col_idx, col_idx, column_width)

# Fill Supplementary Table 1

In [ ]:
suptable = pd.read_excel('Artigo e Drafts/Supplementary Material/Supplementary Table 3. All Data.xlsx')
suptable.head()

In [ ]:
suptable.columns

In [ ]:
interestcols = ['Number_of_Criteria_Followed_for_Good_Subtyping_for_Disease_Prediction',
       'Quality_Index_Mestre',
       'Which type of PD patients were included in the clustering analyses?',
       'Were subtyped patients roughly at the same disease stage or duration?',
       'From how many different centers came the patients to be subtyped?',
       'How were patients recruited?',
       'Which diagnostic criteria was utilized to diagnose PD?',
       'Which was the sampling method?',
       'Was there a longitudinal follow-up in any way?',
       'If longitudinal follow-up was provided, how much years were patients followed for?',
       'Completeness of follow-up', 'Completeness_of_Followup',
       'Longitudinal_Followup_Years',
       'Which type of descriptive data regarding the PD population was presented?',
       'Were the included patients for clustering purposes drug-naive?',
       'Number of PD patients utilized for clustering purposes',
       'Number of PD patients utilized for validation purposes',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [AMP-PD]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [BioFIND]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Fox Insight]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [GP2 Dataset]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [LRRK2 Cohort Consortium]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Parkinson Progression Marker Initiative (PPMI)]',
       "Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Parkinson's Disease Biomarker Program (PDBP)]",
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [UK Biobank]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Oxford Parkinson Disease Center Discovery Cohort]',
       'Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Other specific or local datasets]',
       'If a different publicly available dataset than the ones mentioned above was utlized for any means, please specify its full name',
       'From which countries or world region came the patients whose data was used in the clustering?',
       'Which type data was utilized for the clustering algorithm?',
       'Which type data was evaluated for association / interpretation purposes of the identified clusters, indepentently if they were significant or not?',
       'If the paper contains multiple exploration of clusters (creates different subgroups multiple times) shortly describe the differences in the cluster strategy for each exploration:',
       'If clinical data obtained from rating scales was used for clustering purposes, which data and rating scales were utilized?',
       'If neuroimaging data was utilized for clustering, did the article perform any sort of quality control measure?',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [MRI cortical thickness]',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [MRI subcortical volume]',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [MRI white matter lesions / hyperintensities]',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [Functional MRI]',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [MRI Others]',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [PET]',
       'If neuroimaging data was utilized for clustering or interpretation / association  purposes, which ones were utilized? [SPECT DAT binding ratios]',
       'How many subtypes did the evaluated study identified?',
       'If they were identified, what are the names (meaning) of the identified clusters?',
       'What is the number of patients present in each of the identified clusters?',
       'Subtype stability',
       'Did the article provide an individual level prediction algorithm?']

In [ ]:
# Iterate through the rows of suptable
for row in range(len(suptable)):
    # Check if the 'Article Title' matches in both DataFrames
    if suptable['Article Title'].iloc[row] == df['Article Title'].iloc[row]:
        # Iterate through the columns of interest and set values using .loc
        for col in interestcols:
            suptable.loc[row, col] = df.loc[row, col]

# Optionally save the updated suptable to a CSV file
suptable.to_excel('Artigo e Drafts/Supplementary Material/Supplementary Table 3. All Data (Automatical).xlsx', index=False)

print("Updated suptable with data from df")


In [ ]:
suptable.tail()

# Analysis

## Year comparisons

### Cut-off: quantile division

In [ ]:
# REMOVING VALIDATION AND CLUSTERING NAMES - THEY ARE INTERESTING, BUT I WANT THE MOST RELEVANT ASPECTS
# REMOVING ratingscales ALSO - FOCUSED

dependentvar = 'Year of publication Quantile Division'

# Running the code - first creating the master table
comparison_analysis = analyze_data_corrected_v8(df[relevantvars],
                                                qualitative_variable = dependentvar,
                                                p_significance=4, decimal_places=1,
                                                exclude_vars = ['Article Title', 'Author (only the last name)',
                                                                'Reviewer'] + neuroimagingvars + validation_names + clustering_names + ratingscales, # Those are giving problems to the analysis
                                                numerical_threshold=5, printsteps = False, hidetotal = False)

# Running the adjustment function
# First one - creates new rows for categories
detailed_df = detailed_output_analysis_condensed(comparison_analysis)

# Second one - adjusts the last created output
condensed_df = condense_rows(detailed_df)
print('Lenght of dataset:',len(condensed_df))
condensed_df.head(10)

In [ ]:
# Export the result

sheet_path = 'Tables/'+dependentvar+'.xlsx'
sheet_name = dependentvar[0:31]

with pd.ExcelWriter(sheet_path, engine='xlsxwriter') as writer:
    condensed_df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0)

    # Adjusting column width
    for column in condensed_df:
        column_width = max(condensed_df[column].astype(str).map(len).max(), len(column))
        col_idx = condensed_df.columns.get_loc(column)
        writer.sheets[sheet_name].set_column(col_idx, col_idx, column_width)

    # Ensure p-value is treated as numeric
    if 'p-value' in condensed_df.columns:
        condensed_df['p-value'] = pd.to_numeric(condensed_df['p-value'], errors='coerce')

    # Only significant interactions
    sigcondensed = condensed_df[condensed_df['p-value'] < 0.05]
    sheet_name = 'Significant'

    sigcondensed.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0)

    # Adjusting column width for the significant sheet
    for column in sigcondensed:
        column_width = max(sigcondensed[column].astype(str).map(len).max(), len(column))
        col_idx = sigcondensed.columns.get_loc(column)
        writer.sheets[sheet_name].set_column(col_idx, col_idx, column_width)

### Cut-off: 2020

In [ ]:
# REMOVING VALIDATION AND CLUSTERING NAMES - THEY ARE INTERESTING, BUT I WANT THE MOST RELEVANT ASPECTS
# REMOVING ratingscales ALSO - FOCUSED

dependentvar = 'Before_2020_After'

# Running the code - first creating the master table
comparison_analysis = analyze_data_corrected_v8(df[relevantvars],
                                                qualitative_variable = dependentvar,
                                                numerical_threshold=5,
                                                p_significance=4, decimal_places=1,
                                                exclude_vars = ['Article Title', 'Author (only the last name)',
                                                                'Reviewer'] + neuroimagingvars + ratingscales, # Those are giving problems to the analysis
                                                printsteps = False, hidetotal = False)

# Running the adjustment function
# First one - creates new rows for categories
detailed_df = detailed_output_analysis_condensed(comparison_analysis)

# Second one - adjusts the last created output
condensed_df = condense_rows(detailed_df)
print('Lenght of dataset:',len(condensed_df))
condensed_df.head(25)

In [ ]:
# Export the result
sheet_path = 'Tables/'+dependentvar+'.xlsx'
sheet_name = dependentvar

with pd.ExcelWriter(sheet_path, engine='xlsxwriter') as writer:
    condensed_df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0)

    # Adjusting column width
    for column in condensed_df:
        column_width = max(condensed_df[column].astype(str).map(len).max(), len(column))
        col_idx = condensed_df.columns.get_loc(column)
        writer.sheets[sheet_name].set_column(col_idx, col_idx, column_width)

    # Ensure p-value is treated as numeric
    if 'p-value' in condensed_df.columns:
        condensed_df['p-value'] = pd.to_numeric(condensed_df['p-value'], errors='coerce')

    # Only significant interactions
    sigcondensed = condensed_df[condensed_df['p-value'] < 0.05]
    sheet_name = 'Significant'

    sigcondensed.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0)

    # Adjusting column width for the significant sheet
    for column in sigcondensed:
        column_width = max(sigcondensed[column].astype(str).map(len).max(), len(column))
        col_idx = sigcondensed.columns.get_loc(column)
        writer.sheets[sheet_name].set_column(col_idx, col_idx, column_width)

## Open access datasets

In [ ]:
# Defining column of interest
colpos = 'Open_Datasets_Used_Yes_or_No' # Analysis column and position
sheet_path = 'Tables/' + colpos + '.xlsx'

# Checking
df[colpos].value_counts(dropna=False)

In [ ]:
# Removed: neuroimagingvars (error), clustering names, Validation names and ratingscales - too much detail

# Replace dependent names to avoid it being the same as independent
# Step 1: Store original names/values
original_values = df[colpos].copy()

# Step 2: Substitute names/values for analysis
df[colpos] = df[colpos].replace(0, 'No')
df[colpos] = df[colpos].replace(1, 'Yes')



# Execute analysis or other processing steps
comparison_analysis = analyze_data_corrected_v8(df[relevantvars],
                                                qualitative_variable=colpos,
                                                p_significance=4, decimal_places=1,
                                                exclude_vars=['Article Title', 'Author (only the last name)',
                                                              'Reviewer'] + neuroimagingvars + validation_names + clustering_names + ratingscales,
                                                numerical_threshold=5, printsteps=False, hidetotal=False)

# Running the adjustment function
# First one - creates new rows for categories
detailed_df = detailed_output_analysis_condensed(comparison_analysis)

# Second one - adjusts the last created output
condensed_df = condense_rows(detailed_df)

# Step 3: Revert to original names/values after analysis
df[colpos] = original_values

# Exporting
with pd.ExcelWriter(sheet_path, engine='xlsxwriter') as writer:
    # Writing the full dataset to its own sheet
    condensed_df.to_excel(writer, sheet_name=colpos, index=False, startrow=0)

    # Adjusting column width for the full dataset
    max_width_scale_factor = 0.95
    for column in condensed_df:
        column_width = max_width_scale_factor * max(condensed_df[column].astype(str).map(len).max(), len(column))
        col_idx = condensed_df.columns.get_loc(column)
        writer.sheets[colpos].set_column(col_idx, col_idx, column_width*max_width_scale_factor)

    # Filtering and writing only significant interactions to another sheet
    sigcondensed = condensed_df[condensed_df['p-value'] < 0.05]
    sigcondensed.to_excel(writer, sheet_name='Significant', index=False, startrow=0)

    # Adjusting column width for the significant sheet
    for column in sigcondensed:
        column_width = max_width_scale_factor * max(sigcondensed[column].astype(str).map(len).max(), len(column))
        col_idx = sigcondensed.columns.get_loc(column)
        writer.sheets['Significant'].set_column(col_idx, col_idx, column_width*max_width_scale_factor)

# Showing results and output
print('Length of dataset:', len(condensed_df))
condensed_df.head(10)

In [ ]:
pd.crosstab(df['None of the mentioned data on the PD population was informed by the study'], df['Open_Datasets_Used_Yes_or_No'])

# Filling pre-made table

Created a table that will be in the official article and I want to fill it automatically with each information in their position

To use this, take the names present in the column called "Var name" in the Shape table and put them in "Table 1 (To be Filled)" in substitution of the formal name

Once it is finished, reset it with the appropriate names from the (Shape) table

In [ ]:
!ls

In [ ]:
# Table to input data
filltable = pd.read_excel('Artigo e Drafts/Obsoleto/Table 1 (To be Filled).xlsx')
filltable.head()

In [ ]:
# Table to take data from
datatable = pd.read_excel('Tables/Before_2020_After.xlsx')
datatable.head()

In [ ]:
# Converting p-value to less decimals
# Function to modify p-value
def modify_p_value(p_value):
    if isinstance(p_value, float):
        if p_value < 0.001:
            return "< 0.001"
        if p_value < 0.01:
            return "< 0.01"
        else:
            return f"{p_value:.2f}"
    return p_value

# Apply the function to the p-value column
datatable['p-value'] = datatable['p-value'].apply(modify_p_value)

# Function to round percentage values to the nearest integer
def round_percentage(value):
    try:
        temp = int(round(float(value.strip('%'))))
        tempstring = str(temp) + '%'
        return tempstring
    except ValueError:
        return value  # Return the value as is if it can't be converted to float

# Apply the function to the % Complete Data column
datatable['% Complete Data'] = datatable['% Complete Data'].apply(round_percentage)

datatable.head()

In [ ]:
def map_values(row):
    var_name = row['Table 1. General characteristics of the included studies according to year of publication range']
    print(var_name)
    if pd.notnull(var_name):
        for variable in datatable['Variables'].values:
            if var_name in variable:
                match = datatable[datatable['Variables'] == variable]
                if not match.empty:
                    row['Unnamed: 4'] = match['Total (n = 82)'].values[0]
                    row['Unnamed: 5'] = match['0.0 (n = 43)'].values[0]
                    row['Unnamed: 6'] = match['1.0 (n = 39)'].values[0]
                    row['Unnamed: 7'] = match['p-value'].values[0]
                break
    return row

# Apply the mapping function to filltable
filltable = filltable.apply(map_values, axis=1)

# Display the updated filltable
filltable.head()

In [ ]:
'''# Updated mapping function
def map_values(row):
    var_name = row['Unnamed: 4']
    if pd.notnull(var_name) and var_name in datatable['Variables'].values:
        match = datatable[datatable['Variables'] == var_name]
        if not match.empty:
            row['Unnamed: 5'] = match['0.0 (n = 42)'].values[0]
            row['Unnamed: 6'] = match['1.0 (n = 39)'].values[0]
            row['Unnamed: 7'] = match['p-value'].values[0]
            row['Unnamed: 8'] = match['% Complete Data'].values[0]
    return row

# Apply the mapping function to filltable
filltable = filltable.apply(map_values, axis=1)

filltable.head()'''

In [ ]:
# Defining things
colpos = 'Table 1 (Filled)' # Analysis column and position
sheet_path = 'Artigo e Drafts/Obsoleto/' + colpos + '.xlsx'

# Exporting
with pd.ExcelWriter(sheet_path, engine='xlsxwriter') as writer:
    # Writing the full dataset to its own sheet
    filltable.to_excel(writer, sheet_name=colpos, index=False, startrow=0)

# Showing results and output
print('Length of dataset:', len(filltable))
filltable.head(10)

# Filling pre-made table (Open_dataset)

Created a table that will be in the official article and I want to fill it automatically with each information in their position

To use this, take the names present in the column called "Var name" in the Shape table and put them in "Table 1 (To be Filled)" in substitution of the formal name

Once it is finished, reset it with the appropriate names from the (Shape) table

In [ ]:
!ls

In [ ]:
# Table to input data
filltable = pd.read_excel('Artigo e Drafts/Obsoleto/Table 1 (To be Filled Open Datasets).xlsx')
filltable.head()

In [ ]:
# Table to take data from
datatable = pd.read_excel('Tables/Open_Datasets_Used_Yes_or_No.xlsx')
datatable.head()

In [ ]:
# Converting p-value to less decimals
# Function to modify p-value
def modify_p_value(p_value):
    if isinstance(p_value, float):
        if p_value < 0.001:
            return "< 0.001"
        if p_value < 0.01:
            return "< 0.01"
        else:
            return f"{p_value:.2f}"
    return p_value

# Apply the function to the p-value column
datatable['p-value'] = datatable['p-value'].apply(modify_p_value)

# Function to round percentage values to the nearest integer
def round_percentage(value):
    try:
        temp = int(round(float(value.strip('%'))))
        tempstring = str(temp) + '%'
        return tempstring
    except ValueError:
        return value  # Return the value as is if it can't be converted to float

# Apply the function to the % Complete Data column
datatable['% Complete Data'] = datatable['% Complete Data'].apply(round_percentage)

datatable.head()

In [ ]:
def map_values(row):
    var_name = row['Table 1. General characteristics of the included studies according to year of publication range']
    print(var_name)
    if pd.notnull(var_name):
        for variable in datatable['Variables'].values:
            if var_name in variable:
                match = datatable[datatable['Variables'] == variable]
                if not match.empty:
                    row['Unnamed: 4'] = match['No (n = 57)'].values[0]
                    row['Unnamed: 5'] = match['Yes (n = 25)'].values[0]
                    row['Unnamed: 6'] = match['p-value'].values[0]
                break
    return row

# Apply the mapping function to filltable
filltable = filltable.apply(map_values, axis=1)

# Display the updated filltable
filltable.head()

In [ ]:
'''# Updated mapping function
def map_values(row):
    var_name = row['Unnamed: 4']
    if pd.notnull(var_name) and var_name in datatable['Variables'].values:
        match = datatable[datatable['Variables'] == var_name]
        if not match.empty:
            row['Unnamed: 5'] = match['0.0 (n = 42)'].values[0]
            row['Unnamed: 6'] = match['1.0 (n = 39)'].values[0]
            row['Unnamed: 7'] = match['p-value'].values[0]
            row['Unnamed: 8'] = match['% Complete Data'].values[0]
    return row

# Apply the mapping function to filltable
filltable = filltable.apply(map_values, axis=1)

filltable.head()'''

In [ ]:
# Defining things
colpos = 'Table 1 (Filled Open Datasets)' # Analysis column and position
sheet_path = 'Artigo e Drafts/Obsoleto/' + colpos + '.xlsx'

# Exporting
with pd.ExcelWriter(sheet_path, engine='xlsxwriter') as writer:
    # Writing the full dataset to its own sheet
    filltable.to_excel(writer, sheet_name=colpos, index=False, startrow=0)

# Showing results and output
print('Length of dataset:', len(filltable))
filltable.head(10)

# Graphs

## Essential progression criteria

**Idea:** select X articles and produce a bar chart like this one: https://drive.google.com/file/d/1r4qX5pM9eG5HR_PjCNsy9kKG4-o-r0EN/view?usp=sharing

### Creating variables

In [ ]:
# Creating new columns based on aforementioned criteria
df['Individual classification algorithm'] = df['Individual_Classification_Algorithm'].apply(lambda x: 1 if x == 1 else 0)
df['Longitudinal follow-up'] = df['Longitudinal_Followup_Status'].apply(lambda x: 1 if x == 1 else 0)
df['Follow up data for at least 3 years'] = df['Longitudinal_Followup_Years'].apply(lambda x: 1 if x >= 3 else 0)
df['Clustering with 2 or more domains'] = df['Clustering_Number_of_Specific_Domains'].apply(lambda x: 1 if x > 1 else 0)
df['Post hoc analysis with 2 or more domains'] = df['Validation_Number_of_Specific_Domains'].apply(lambda x: 1 if x > 1 else 0)
df['Clustering with at least 100 patients'] = df['Number of PD patients utilized for clustering purposes'].apply(lambda x: 1 if x > 100 else 0)
df['Validation in a different cohort'] = df['Validation_in_a_Different_Cohort'].apply(lambda x: 1 if x == 1 else 0)
df['Describes basic demographic and clinical characteristics of the patients analysed'] = df['Minimum_Info_Age_Sex_Staging'].apply(lambda x: 1 if x == 1 else 0) # Age, sex and disease staging
df['Year Category'] = df['Before_2020_After'].apply(lambda x: '2021 - 2024' if str(x) in ['1', '1.0'] else '1999 - 2020')

transition_criteria = [
    'Year Category',
    'Individual classification algorithm',
    'Longitudinal follow-up',
    'Follow up data for at least 3 years',
    'Clustering with 2 or more domains',
    'Post hoc analysis with 2 or more domains',
    'Clustering with at least 100 patients',
    'Validation in a different cohort',
    'Describes basic demographic and clinical characteristics of the patients analysed'
]

df[['Year of publication'] + transition_criteria].head(5)

In [ ]:
calculate_percentage_distribution(df, transition_criteria[1:])

### Option 1 - All Studies

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import textwrap

# Define a figure name
figurename = 'Non essential criteria general'

# Calculate percentages
percentages = (df[transition_criteria[1:]].mean() * 100).round()

# Sort percentages in ascending order
percentages = percentages.sort_values()

# Create the bar chart
fig, ax = plt.subplots(figsize=(12, 8))  # Increase figure size for more space

# Get evenly spaced colors from the viridis colormap
num_bars = len(percentages)
colors = plt.cm.viridis(np.linspace(0, 1, num_bars))

# Plot the bars with custom colors and alpha
bars = ax.barh(percentages.index, percentages.values, color=colors, alpha=0.8)

# Add labels to the bars
label_padding = 1  # Distance of labels from the bars
for bar in bars:
    width = bar.get_width()
    # Position the text to the right of the bars, with a custom padding
    ax.text(width + label_padding, bar.get_y() + bar.get_height()/2, f'{width:.0f}%', va='center', ha='left', fontsize=12)

# Adjust column names to occupy 2 lines in graph (centered in middle, limited line width)
column_names = [textwrap.fill(col, width=38) for col in percentages.index]
ax.set_yticklabels(column_names)

# Set labels and title
ax.set_xlabel('Percentage', fontsize=12)

# Adjust x-axis limits to 100% with spaces every 10%
ax.set_xlim(0, 110)
ax.set_xticks(range(0, 101, 10))

# Increase axis tick size
ax.tick_params(axis='both', which='major', labelsize=12)

# Remove vertical grid lines
ax.grid(False)
plt.tight_layout()  # Ensure labels are not cut off

# Save the figure
plt.savefig(f'Plots/SVG/{figurename}.svg', format='svg', dpi=300)
plt.savefig(f'Plots/PNG/{figurename}.png', format='png', dpi=300)

# Show plot

plt.show()


### Option 2 - Divided by Year

In [ ]:
# Doing simple chi square among categories
from scipy.stats import chi2_contingency

def chi_square_test(column1, column2):
    # Create a contingency table
    contingency_table = pd.crosstab(column1.astype(str), column2.astype(str))

    # Perform the chi-square test
    try:
      chi2, p, dof, ex = chi2_contingency(contingency_table)
    except Exception as e:
      print(e)
      return None

    # Return the p-value
    return round(p,3)

# Apply chi-square test for each column and create a new column for p-values

transition_criteria_no_year_chi_square = []
for col in transition_criteria[1:]:
    transition_criteria_no_year_chi_square.append(chi_square_test(df[col], df['Year Category']))

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import textwrap
import numpy as np
from matplotlib.font_manager import FontProperties
import matplotlib.cm as cm

# Define a figurename
figurename = 'Essential progression criteria by year'

# Assuming 'df' and 'transition_criteria' are defined
# Calculate percentages for each 'Year Category'
percentages = df[transition_criteria].groupby('Year Category').mean().T * 100

# Create the bar chart
fig, ax = plt.subplots(figsize=(14, 10))  # Adjust the size as needed

bar_width = 0.35  # Set the width of each bar
index = np.arange(len(percentages))  # The label locations

# Reverse the order of columns to plot '1999 - 2021' first
columns = percentages.columns[::-1]

# Choose a color palette
current_palette = cm.RdBu  # Options: 'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu', 'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic'
alpha = 0.75 # alpha

# Plot bars for each year category, reversed to have '1999 - 2021' at top
for i, year in enumerate(percentages.columns[::-1]):
    # Use colors from extreme ends of the colormap
    color_1 = current_palette(0.0)  # Start of the colormap
    color_2 = current_palette(1.0)  # End of the colormap

    # Assign colors alternately from each end of the colormap for visibility
    if i % 2 == 0:
        color = color_1
    else:
        color = color_2

    bars = ax.barh(index + i * bar_width, percentages[year], bar_width, label=year, alpha=alpha, color=color)

    # Add labels to the bars
    label_padding = 1  # Slightly increase padding for clarity
    for bar in bars:
        width = bar.get_width()
        ax.text(width + label_padding, bar.get_y() + bar.get_height()/2, f'{width:.0f}%', va='center', ha='left', fontsize=12)

# Define p-values and corresponding bold condition
p_values = transition_criteria_no_year_chi_square
columns_p_values = dict(zip(transition_criteria[1:], p_values))

# Adjust column names to occupy 2 lines in graph and append * if p-value < 0.05
column_names = [textwrap.fill(col + ('*' if columns_p_values[col] < 0.05 else ''), width=25) for col in percentages.index]

ax.set_yticks(index + bar_width / 2 * (len(columns) - 1))
ax.set_yticklabels(column_names)

# Add labels, title, and legend
ax.set_xlabel('Percentage', fontsize=16)
'''ax.set_title('Percentage of Criteria Met by Year Category', fontsize=16)'''
ax.legend()

# Adjust x-axis limits to 100% with spaces every 10%
ax.set_xlim(0, 100)  # Extend x-axis limit to fit labels comfortably
ax.set_xticks(range(0, 101, 10))

# Increase x-axis tick size
ax.tick_params(axis='x', labelsize=13)
ax.tick_params(axis='y', labelsize=13)

### Adjusting legends ###
# Get handles and labels from the existing plot
handles, labels = ax.get_legend_handles_labels()

# Reverse the order of handles/labels for the legend
handles = handles[::-1]
labels = labels[::-1]

# Display the legend with modifications
ax.legend(handles, labels, fontsize='large', title='Year Category', title_fontsize='large', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False)

# Adjust layout
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Show plot
plt.show()


In [ ]:
# Very similar, shows there isn't really anything significant
percentages

###  Option 3 - Donut Chart

Showing the total of studies that fall within specific criterias counts

Total possible: 8

In [ ]:
print(df['Number_of_Criteria_Followed_for_Good_Subtyping_for_Disease_Prediction'].value_counts())

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Define a figurename
figurename = 'Essential progression criteria donut chart without text'

# Setting data for use
data = df['Number_of_Criteria_Followed_for_Good_Subtyping_for_Disease_Prediction'].value_counts()

# Convert the series to a DataFrame
tempdf = data.reset_index()
tempdf.columns = ['Number_of_Criteria', 'Count']

# Sort the data by 'Number_of_Criteria' in ascending order (not necessary here as we want words in order)
tempdf = tempdf.sort_values(by='Number_of_Criteria', ascending=True)

# Create a mapping dictionary
criteria_mapping = {
    0: "Zero",
    1: "One",
    2: "Two",
    3: "Three",
    4: "Four",
    5: "Five",
    6: "Six",
    7: "Seven",
    8: "Eight"
}

# Apply the mapping
tempdf['Number_of_Criteria'] = tempdf['Number_of_Criteria'].replace(criteria_mapping)

# Create a color palette with Matplotlib's viridis, applying a softer alpha to each color
colors = plt.cm.viridis(np.linspace(0, 1, len(tempdf)))
colors = [(r, g, b, 0.6) for r, g, b, _ in colors]  # Adding alpha transparency

fig, ax = plt.subplots(figsize=(10, 10))  # Maintain a large figure size for clarity

# Suppress the first and last labels
labels = tempdf['Number_of_Criteria'].tolist()
labels[0] = ""
labels[-1] = ""

wedges, texts, autotexts = ax.pie(tempdf['Count'], labels=labels, autopct='%1.0f%%', startangle=90, colors=colors, pctdistance=0.85, textprops={'color':"black", 'fontsize': 14})

# Draw a circle at the center to create a donut shape
centre_circle = plt.Circle((0,0), 0.50, fc='white')  # Adjust the radius to 0.50 to reduce the size of the central white space
fig.gca().add_artist(centre_circle)

# Improve text readability and aesthetics for remaining labels
for text in texts:
    text.set_fontsize(24)   # Increase font size for labels

for autotext in autotexts:
    autotext.set_color('black')  # Ensure percentage labels are visible
    autotext.set_fontsize(16)    # Consistent and readable font size

# Reposition the first and last labels manually
angle_first = wedges[0].theta2 - (wedges[0].theta2 - wedges[0].theta1) / 2
angle_last = wedges[-1].theta2 - (wedges[-1].theta2 - wedges[-1].theta1) / 2

x_first = 2.0 * np.cos(np.radians(angle_first))
y_first = 1.1 * np.sin(np.radians(angle_first))
x_last = 1.55 * np.cos(np.radians(angle_last))
y_last = 1.1 * np.sin(np.radians(angle_last))

# Add custom text for the first label
ax.text(x_first, y_first, tempdf['Number_of_Criteria'].iloc[0], ha='center', va='center', fontsize=24, color='black')

# Add custom text for the last label
ax.text(x_last, y_last, tempdf['Number_of_Criteria'].iloc[-1], ha='center', va='center', fontsize=24, color='black')

plt.tight_layout()  # Adjust layout to ensure there is no content overlap

# Save the figure
plt.savefig(f'Plots/SVG/{figurename}.svg', format='svg', dpi=300)
plt.savefig(f'Plots/PNG/{figurename}.png', format='png', dpi=300)

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Define a figurename
figurename = 'Essential progression criteria donut chart with text'

# Setting data for use
data = df['Number_of_Criteria_Followed_for_Good_Subtyping_for_Disease_Prediction'].value_counts()

# Convert the series to a DataFrame
tempdf = data.reset_index()
tempdf.columns = ['Number_of_Criteria', 'Count']

# Sort the data by 'Number_of_Criteria' in ascending order (not necessary here as we want words in order)
tempdf = tempdf.sort_values(by='Number_of_Criteria', ascending=True)

# Create a mapping dictionary
criteria_mapping = {
    0: "Zero",
    1: "One",
    2: "Two",
    3: "Three",
    4: "Four",
    5: "Five",
    6: "Six",
    7: "Seven",
    8: "Eight"
}

# Apply the mapping
tempdf['Number_of_Criteria'] = tempdf['Number_of_Criteria'].replace(criteria_mapping)

# Create a color palette with Matplotlib's viridis, applying a softer alpha to each color
colors = plt.cm.viridis(np.linspace(0, 1, len(tempdf)))
colors = [(r, g, b, 0.6) for r, g, b, _ in colors]  # Adding alpha transparency

fig, ax = plt.subplots(figsize=(10, 10))  # Maintain a large figure size for clarity

# Suppress the first and last labels
labels = tempdf['Number_of_Criteria'].tolist()
labels[0] = ""
labels[-1] = ""

wedges, texts, autotexts = ax.pie(tempdf['Count'], labels=labels, autopct='%1.0f%%', startangle=90, colors=colors, pctdistance=0.85, textprops={'color':"black", 'fontsize': 14})

# Draw a circle at the center to create a donut shape
centre_circle = plt.Circle((0,0), 0.50, fc='white')  # Adjust the radius to 0.50 to reduce the size of the central white space
fig.gca().add_artist(centre_circle)

# Improve text readability and aesthetics for remaining labels
for text in texts:
    text.set_fontsize(24)   # Increase font size for labels

for autotext in autotexts:
    autotext.set_color('black')  # Ensure percentage labels are visible
    autotext.set_fontsize(16)    # Consistent and readable font size

# Reposition the first and last labels manually
angle_first = wedges[0].theta2 - (wedges[0].theta2 - wedges[0].theta1) / 2
angle_last = wedges[-1].theta2 - (wedges[-1].theta2 - wedges[-1].theta1) / 2

x_first = 2.0 * np.cos(np.radians(angle_first))
y_first = 1.1 * np.sin(np.radians(angle_first))
x_last = 1.55 * np.cos(np.radians(angle_last))
y_last = 1.1 * np.sin(np.radians(angle_last))

# Add custom text for the first label
ax.text(x_first, y_first, tempdf['Number_of_Criteria'].iloc[0], ha='center', va='center', fontsize=24, color='black')

# Add custom text for the last label
ax.text(x_last, y_last, tempdf['Number_of_Criteria'].iloc[-1], ha='center', va='center', fontsize=24, color='black')

# Add phrase in the middle of the donut chart
plt.text(0, 0, 'No study achieved\nall the 8 desired\nessential criteria',
         horizontalalignment='center', verticalalignment='center', fontsize=18, color='black')

plt.tight_layout()  # Adjust layout to ensure there is no content overlap

# Save the figure
plt.savefig(f'Plots/SVG/{figurename}.svg', format='svg', dpi=300)
plt.savefig(f'Plots/PNG/{figurename}.png', format='png', dpi=300)

plt.show()

## Non essential criteria

**Idea:** present visually other criteria that are relevant but weren't cited by the article on the essential criteria

It is mostly a display of measures collected by the quality index

### Creating variables

In [ ]:
# Defining criteria for the graph
criteria = [
    'Year Category',
    'Formal_Criteria_Utilized_or_Informed',
    'Recruitment_Community_or_Population',
    'Specific_Cluster_Validation_Differences_Yes_or_No',
    'Similar_PD_Staging',
    'Drug_Naive_Status',
    'Multicentric_X_Others',
    'Consecutive_or_Random_Sampling',
    'Performed_Validation']

data = df[criteria]

# Renaming columns
column_renames = {
    'Year Category': 'Year Category',
    'Formal_Criteria_Utilized_or_Informed': 'Formal diagnostic criteria used',
    'Recruitment_Community_or_Population': 'Populational or community based recruitment',
    'Specific_Cluster_Validation_Differences_Yes_or_No': 'Post hoc analysis with different variables than those used in clustering',
    'Similar_PD_Staging': 'Patients in a similar PD staging or duration',
    'Drug_Naive_Status': 'Patients being drug naive at clustering',
    'Multicentric_X_Others': 'Multicentric study',
    'Consecutive_or_Random_Sampling': 'Consecutive or random sampling',
    'Performed_Validation': 'Validation performed either within or in another dataset'}

# Data situation
data.rename(columns=column_renames, inplace=True)

# Review
calculate_percentage_distribution(data, list(column_renames.values())[1:])

### Option 1 - All Studies

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import textwrap

# Define a figure name
figurename = 'Non essential criteria general'

# Calculate percentages
percentages = (data[list(column_renames.values())[1:]].mean() * 100).round()

# Sort percentages in ascending order
percentages = percentages.sort_values()

# Create the bar chart
fig, ax = plt.subplots(figsize=(12, 8))  # Increase figure size for more space

# Get evenly spaced colors from the viridis colormap
num_bars = len(percentages)
colors = plt.cm.viridis(np.linspace(0, 1, num_bars))

# Plot the bars with custom colors and alpha
bars = ax.barh(percentages.index, percentages.values, color=colors, alpha=0.8)

# Add labels to the bars
label_padding = 1  # Distance of labels from the bars
for bar in bars:
    width = bar.get_width()
    # Position the text to the right of the bars, with a custom padding
    ax.text(width + label_padding, bar.get_y() + bar.get_height()/2, f'{width:.0f}%', va='center', ha='left', fontsize=12)

# Adjust column names to occupy 2 lines in graph (centered in middle, limited line width)
column_names = [textwrap.fill(col, width=38) for col in percentages.index]
ax.set_yticklabels(column_names)

# Set labels and title
ax.set_xlabel('Percentage', fontsize=12)

# Adjust x-axis limits to 100% with spaces every 10%
ax.set_xlim(0, 110)
ax.set_xticks(range(0, 101, 10))

# Increase axis tick size
ax.tick_params(axis='both', which='major', labelsize=12)

# Remove vertical grid lines
ax.grid(False)
plt.tight_layout()  # Ensure labels are not cut off

# Save the figure
plt.savefig(f'Plots/SVG/{figurename}.svg', format='svg', dpi=300)
plt.savefig(f'Plots/PNG/{figurename}.png', format='png', dpi=300)

plt.show()


### Option 2 - Divided by Year

In [ ]:
# Doing simple chi square among categories
from scipy.stats import chi2_contingency

def chi_square_test(column1, column2):
    # Create a contingency table
    contingency_table = pd.crosstab(column1.astype(str), column2.astype(str))

    # Perform the chi-square test
    try:
      chi2, p, dof, ex = chi2_contingency(contingency_table)
    except Exception as e:
      print(e)
      return None

    # Return the p-value
    return round(p,3)

# Apply chi-square test for each column and create a new column for p-values

# Apply chi-square test for each column against 'Year Category' and store p-values
chi_square = []
for col in list(column_renames.values())[1:]:
    p_value = chi_square_test(data[col], data['Year Category'])  # Assuming 'data' has 'Year Category'
    chi_square.append(p_value)

print(chi_square)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import textwrap
import numpy as np
from matplotlib.font_manager import FontProperties
import matplotlib.cm as cm

# Define a figurename
figurename = 'Non essential criteria by year'

# Assuming 'df' and 'transition_criteria' are defined
# Calculate percentages for each 'Year Category'
percentages = data[list(column_renames.values())].groupby('Year Category').mean().T * 100

# Create the bar chart
fig, ax = plt.subplots(figsize=(14, 10))  # Adjust the size as needed

bar_width = 0.35  # Set the width of each bar
index = np.arange(len(percentages))  # The label locations

# Reverse the order of columns to plot '1999 - 2021' first
columns = percentages.columns[::-1]

# Choose a color palette
current_palette = cm.coolwarm  # Options: 'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu', 'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic'
alpha = 0.7 # alpha

# Plot bars for each year category, reversed to have '1999 - 2021' at top
for i, year in enumerate(percentages.columns[::-1]):
    # Use colors from extreme ends of the colormap
    color_1 = current_palette(0.0)  # Start of the colormap
    color_2 = current_palette(1.0)  # End of the colormap

    # Assign colors alternately from each end of the colormap for visibility
    if i % 2 == 0:
        color = color_1
    else:
        color = color_2

    bars = ax.barh(index + i * bar_width, percentages[year], bar_width, label=year, alpha=alpha, color=color)

    # Add labels to the bars
    label_padding = 1  # Slightly increase padding for clarity
    for bar in bars:
        width = bar.get_width()
        ax.text(width + label_padding, bar.get_y() + bar.get_height()/2, f'{width:.0f}%', va='center', ha='left', fontsize=12)

# Define p-values and corresponding bold condition
columns_p_values = dict(zip(list(column_renames.values())[1:], chi_square))

# Adjust column names to occupy 2 lines in graph and append * if p-value < 0.05
column_names = [textwrap.fill(col + ('*' if columns_p_values[col] < 0.05 else ''), width=25) for col in percentages.index]

ax.set_yticks(index + bar_width / 2 * (len(columns) - 1))
ax.set_yticklabels(column_names)

# Add labels, title, and legend
ax.set_xlabel('Percentage', fontsize=16)
ax.legend()

# Adjust x-axis limits to 100% with spaces every 10%
ax.set_xlim(0, 100)  # Extend x-axis limit to fit labels comfortably
ax.set_xticks(range(0, 101, 10))

# Increase x-axis tick size
ax.tick_params(axis='x', labelsize=13)
ax.tick_params(axis='y', labelsize=13)

### Adjusting legends ###
# Get handles and labels from the existing plot
handles, labels = ax.get_legend_handles_labels()

# Reverse the order of handles/labels for the legend
handles = handles[::-1]
labels = labels[::-1]

# Display the legend with modifications
ax.legend(handles, labels, fontsize='large', title='Year Category', title_fontsize='large', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False)

# Adjust layout
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Show plot
plt.show()


## Descriptive data availability

### Preparation

In [ ]:
# Removed (no interest) - 'Income', 'Vital Signs (blood pressure, cardiac frequency)', 'Physical Attributes (weight, height etc)'

descriptivedata = ['Before_2020_After','None of the mentioned data on the PD population was informed by the study',
           'Age', 'Gender', 'Ethnicity', 'Education', 'Family History',
           'Age at disease onset', 'Disease duration (years) or staging (such as Hoehn & Yahr and Schawb and England scales)',
           'Medication Doses', 'Genetic_Status_Informed',
           'Summarised data from clinical scales (such as mean MDS-UPDRS scores, number of non-motor symptoms)',
           'Summarised data from neuroimaging studies (such as cortical thickness)',
           "Biomarker's profile (results from biomarker tests)"]

# Doing simple chi square among categories
from scipy.stats import chi2_contingency

def chi_square_test(column1, column2):
    # Create a contingency table
    contingency_table = pd.crosstab(column1.astype(str), column2.astype(str))

    # Perform the chi-square test
    try:
      chi2, p, dof, ex = chi2_contingency(contingency_table)
    except Exception as e:
      print(e)
      return None

    # Return the p-value
    return round(p,3)

# Apply chi-square test for each column and create a new column for p-values

pvalueslist = []
for col in descriptivedata[1:]:
    pvalueslist.append(chi_square_test(df[col], df['Before_2020_After']))

pvalueslist

In [ ]:
data = df[descriptivedata]

# Renaming columns
column_renames = {
    'Before_2020_After':'Year Category',
    'None of the mentioned data on the PD population was informed by the study': 'No data provided',
    'Age': 'Age',
    'Gender': 'Sex',
    'Ethnicity': 'Ethnicity',
    'Education': 'Education',
    'Family History': 'Family history',
    'Age at disease onset': 'Disease onset age',
    'Disease duration (years) or staging (such as Hoehn & Yahr and Schawb and England scales)': 'Disease duration/staging',
    'Medication Doses': 'Medication doses',
    'Genetic_Status_Informed': 'Genetic status',
    'Summarised data from clinical scales (such as mean MDS-UPDRS scores, number of non-motor symptoms)': 'Clinical scales data',
    'Summarised data from neuroimaging studies (such as cortical thickness)': 'Neuroimaging data',
    "Biomarker's profile (results from biomarker tests)": 'Biomarker profile'
}

data.rename(columns=column_renames, inplace=True)

# Showing
calculate_percentage_distribution(data, list(column_renames.values())[1:])

### Option 1 - All Studies


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import textwrap

# Define a figure name
figurename = 'Descriptive data availability general'

# Calculate percentages
percentages = (data[list(column_renames.values())[1:]].mean() * 100).round()

# Sort percentages in ascending order
percentages = percentages.sort_values()

# Create the bar chart
fig, ax = plt.subplots(figsize=(12, 8))  # Increase figure size for more space

# Get evenly spaced colors from the viridis colormap
num_bars = len(percentages)
colors = plt.cm.viridis(np.linspace(0, 1, num_bars))

# Plot the bars with custom colors and alpha
bars = ax.barh(percentages.index, percentages.values, color=colors, alpha=0.8)

# Add labels to the bars
label_padding = 1  # Distance of labels from the bars
for bar in bars:
    width = bar.get_width()
    # Position the text to the right of the bars, with a custom padding
    ax.text(width + label_padding, bar.get_y() + bar.get_height()/2, f'{width:.0f}%', va='center', ha='left', fontsize=12)

# Adjust column names to occupy 2 lines in graph (centered in middle, limited line width)
column_names = [textwrap.fill(col, width=38) for col in percentages.index]
ax.set_yticklabels(column_names)

# Set labels and title
ax.set_xlabel('Percentage', fontsize=12)

# Adjust x-axis limits to 100% with spaces every 10%
ax.set_xlim(0, 110)
ax.set_xticks(range(0, 101, 10))

# Increase axis tick size
ax.tick_params(axis='both', which='major', labelsize=12)

# Remove vertical grid lines
ax.grid(False)
plt.tight_layout()  # Ensure labels are not cut off

# Save the figure
plt.savefig(f'Plots/SVG/{figurename}.svg', format='svg', dpi=300)
plt.savefig(f'Plots/PNG/{figurename}.png', format='png', dpi=300)

plt.show()


### Option 2 - Divided by Year


In [ ]:
data['Year Category'] = data['Year Category'].replace('1.0', '2021 - 2024')
data['Year Category'] = data['Year Category'].replace('0.0', '1999 - 2020')

In [ ]:
data[list(column_renames.values())].groupby('Year Category').mean()

In [ ]:
# Getting a figure name
figurename = 'Descriptive data informed by year'

# Assuming 'df' and 'transition_criteria' are defined
# Calculate percentages for each 'Year Category'
percentages = data[list(column_renames.values())].groupby('Year Category').mean().T * 100

# Reverse the DataFrame along the index to feed the results backwards
percentages = percentages.iloc[::-1]

# Create the bar chart
fig, ax = plt.subplots(figsize=(14, 10))  # Adjust the size as needed

bar_width = 0.35  # Set the width of each bar
index = np.arange(len(percentages))  # The label locations

# Reverse the order of columns to plot '1999 - 2021' first
columns = percentages.columns[::-1]

### OPTION 1 FOR COLOR - PROVIDING HEX VALUES TO A LIST
color_palette = ['#1f77b4', '#ff7f0e']  # Two hex values
alpha = 0.8  # Transparency

# Plot bars for each year category, reversed to have '1999 - 2021' at top
for i, year in enumerate(columns):
    # Assign colors alternately from the custom palette for visibility
    color = color_palette[i % 2]  # Alternating colors
    bars = ax.barh(index + i * bar_width, percentages[year], bar_width, label=year, alpha=alpha, color=color)

    # Add labels to the bars
    label_padding = 1  # Slightly increase padding for clarity
    for bar in bars:
        width = bar.get_width()
        ax.text(width + label_padding, bar.get_y() + bar.get_height()/2, f'{width:.0f}%', va='center', ha='left', fontsize=12)

# Define p-values and corresponding bold condition
columns_p_values = dict(zip(list(column_renames.values())[1:], pvalueslist))

# Adjust column names to occupy 2 lines in graph and append * if p-value < 0.05
column_names = [textwrap.fill(col + ('*' if columns_p_values[col] < 0.05 else ''), width=20) for col in percentages.index]

ax.set_yticks(index + bar_width / 2 * (len(columns) - 1))
ax.set_yticklabels(column_names)

# Add labels, title, and legend
ax.set_xlabel('Percentage', fontsize=16)
ax.legend()

# Adjust x-axis limits to 100% with spaces every 10%
ax.set_xlim(0, 100)  # Extend x-axis limit to fit labels comfortably
ax.set_xticks(range(0, 101, 10))

# Increase x-axis tick size
ax.tick_params(axis='x', labelsize=13)
ax.tick_params(axis='y', labelsize=13)

### Adjusting legends ###
# Get handles and labels from the existing plot
handles, labels = ax.get_legend_handles_labels()
handles.reverse()
labels.reverse()

# Adjusting legends: Positioned outside the graph to the right, centered vertically, no frame
ax.legend(handles, labels, fontsize='large', title='Year Category', title_fontsize='large', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False)

# Adjust layout
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Show plot
plt.show()

In [ ]:
### OPTION 2 FOR PROVIDING COLOR
# Choose a color palette
current_palette = cm.BrBG  # Options: 'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu', 'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic'
alpha = 0.75 # alpha

# Plot bars for each year category, reversed to have '1999 - 2021' at top
for i, year in enumerate(percentages.columns[::-1]):
    # Use colors from extreme ends of the colormap
    color_1 = current_palette(0.0)  # Start of the colormap
    color_2 = current_palette(1.0)  # End of the colormap

    # Assign colors alternately from each end of the colormap for visibility
    if i % 2 == 0:
        color = color_1
    else:
        color = color_2

    bars = ax.barh(index + i * bar_width, percentages[year], bar_width, label=year, alpha=alpha, color=color)

Test with radar chart - bad

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib.font_manager import FontProperties

# Assuming 'data' and 'column_renames' are defined
# Calculate percentages for each 'Year Category'
percentages = data[list(column_renames.values())].groupby('Year Category').mean().T * 100

# Prepare Radar Chart Data
labels = percentages.index
num_vars = len(labels)
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()  # Close the loop
angles += angles[:1]  # Ensure the graph starts and ends at the same point

fig, ax = plt.subplots(figsize=(14, 10), subplot_kw=dict(polar=True))

# Draw one axe per variable and add labels
ax.set_theta_offset(np.pi / 2)
ax.set_theta_direction(-1)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(labels)

# Draw ylabels
ax.set_rlabel_position(0)
ax.set_yticks([20, 40, 60, 80, 100])  # Define the y-ticks to match your scale
ax.set_yticklabels(["20%", "40%", "60%", "80%", "100%"])
ax.set_ylim(0, 100)

# Plot each 'Year Category'
for col in percentages.columns:
    values = percentages[col].tolist()
    values += values[:1]  # Close the loop
    ax.plot(angles, values, linewidth=1, linestyle='solid', label=col)
    ax.fill(angles, values, alpha=0.25)  # Adjust alpha to improve visibility if needed

# Add a legend
ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))

plt.title('Descriptive Data Informed by Year - Radar Chart', size=20)
plt.tight_layout()
plt.show()


## Countries represented

Adjusted - PPMI isn't counting as Middle East and Africa (only 1 center)

In [ ]:
# Defining the different options for answers
descriptivedata = [
    'Sub-Saharan Africa_Adjusted_PPMI',
    'Middle East & North Africa_Adjusted_PPMI',
    'East Asia & Pacific (China, Japan, Australia)_Adjusted_PPMI',
    'South Asia (India, Pakistan, Bangladesh)_Adjusted_PPMI',
    'Europe & Central Asia_Adjusted_PPMI',
    'Latin America & the Caribbean_Adjusted_PPMI',
    'North America_Adjusted_PPMI'
]

data = df[descriptivedata]

# Renaming columns
column_renames = {
    'Sub-Saharan Africa_Adjusted_PPMI':'Sub-Saharan Africa',
    'Middle East & North Africa_Adjusted_PPMI':'Middle East & North Africa',
    'East Asia & Pacific (China, Japan, Australia)_Adjusted_PPMI':'East Asia & Pacific',
    'South Asia (India, Pakistan, Bangladesh)_Adjusted_PPMI':'South Asia',
    'Europe & Central Asia_Adjusted_PPMI':'Europe & Central Asia',
    'Latin America & the Caribbean_Adjusted_PPMI':'Latin America & the Caribbean',
    'North America_Adjusted_PPMI':'North America'}

# Data situation
data.rename(columns=column_renames, inplace=True)

# Sum up all 1's in each column to get the count of studies per region
region_counts = data.sum()

# Calculate percentages
total_patients = region_counts.sum()
percentages = region_counts / total_patients * 100
percentages = round(percentages,0)

# Showing
calculate_percentage_distribution(data, column_renames.values())

In [ ]:
percentages

In [ ]:
import matplotlib.pyplot as plt
!pip install squarify
import squarify
import numpy as np
import matplotlib.patches as patches

figurename = 'Countries represented adjusted PPMI'

# Assuming 'percentages' contains the percentage distribution data
# Prepare data and filter out zero values
filtered_percentages = percentages[percentages > 0]

# Adjust very small values slightly for better visibility in the plot
min_area = 4  # Minimum percentage area for visibility
adjusted_sizes = [max(value, min_area) for value in filtered_percentages]

# Generate labels with percentages
labels = [f"{idx}\n({val:.0f}%)" for idx, val in zip(filtered_percentages.index, filtered_percentages)]

# Plotting a Treemap only with non-zero regions
fig, ax = plt.subplots(figsize=(12, 8))
ax.set_facecolor('black')  # Set the background color to black
colors = plt.cm.viridis(np.linspace(0, 1, len(filtered_percentages)))  # Color map adjustment
squarify.plot(sizes=adjusted_sizes,
              label=labels,
              alpha=0.55,
              color=colors,
              text_kwargs={'fontsize':17.5, 'color':'black'},  # Ensure labels are visible against a black background
              ec='black',  # Set the edge color to white for visibility
              linewidth=2.5)  # Increase line width for thicker borders

# Add an outer border
rect = patches.Rectangle((0, 0), 1, 1, fill=False, color="black", linewidth=3, transform=ax.transAxes, clip_on=False)
ax.add_patch(rect)
plt.axis('off')

# Adjust layout
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

plt.show()


In [ ]:
'''# Install rpy2
!pip install rpy2

# Load the R extension
%load_ext rpy2.ipython'''

In [ ]:
'''%%R
# Install necessary packages if not already installed
if (!require("ggplot2")) install.packages("ggplot2")
if (!require("treemapify")) install.packages("treemapify")
if (!require("viridis")) install.packages("viridis")

# Load the packages
library(ggplot2)
library(treemapify)
library(viridis)'''

In [ ]:
'''%%R
# Define the data
data <- data.frame(
  Region = c("Sub-Saharan Africa", "Middle East & North Africa", "East Asia & Pacific",
             "South Asia", "Europe & Central Asia", "Latin America & the Caribbean",
             "North America"),
  Counts = c(0, 0, 19, 0, 47, 1, 32)
)

# Apply viridis color map with alpha set to 0.55
colors <- viridis_pal(alpha = 1)(length(unique(data$Region)))

# Create the treemap
treemap <- ggplot(data, aes(area = Counts, fill = Region, label = paste(Region, "\n(", Counts, "%)", sep=""))) +
  geom_treemap() +
  geom_treemap_text(fontface = "bold", colour = "black", place = "centre", grow = TRUE, reflow = TRUE) +
  scale_fill_manual(values = colors) +
  theme_minimal() +
  theme(legend.position = "none")

# Print the treemap
print(treemap)
'''

In [ ]:
'''import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

# Assuming 'data' is a DataFrame where rows are entries and columns are regions
region_totals = data.sum()

# Prepare data for the treemap
regions = region_totals.index
counts = region_totals.values

# Apply square root transformation to 'counts' for a more balanced visual representation
transformed_counts = np.sqrt(counts)

# Create the treemap
fig = px.treemap(
    data_frame=pd.DataFrame({'Region': regions, 'Counts': transformed_counts}),  # Creating a DataFrame directly
    path=[px.Constant("all"), 'Region'],  # Defines the hierarchy of the treemap
    values='Counts',  # The area of each sector
    title="Treemap of Patient Distribution by Region"
)

# Apply color palette similar to viridis
colors = plt.cm.viridis(np.linspace(0, 1, len(regions)))
colors_hex = [f'rgb({int(c[0]*255)}, {int(c[1]*255)}, {int(c[2]*255)})' for c in colors]
fig.update_traces(marker_colors=colors_hex)

# Update layout aesthetics
fig.update_layout(
    margin=dict(t=50, l=25, r=25, b=25)
)

# Display the treemap
fig.show()
'''

## Feature types according to year (clustering)

### All features

In [ ]:
featuretypes = ['Year of publication Quantile Division',
                'Clustering_Clinical_Any_Motor_Scales_No_Complications',
                'Clustering_Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)',
                'Clustering_Clinical_Including_Cognition',
                'Clustering_Clinical_Any_Non_Motor_Excluding_Cognition_QoL_Other',
                'Clustering_Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)',
                'Clustering_Any_Neurophysiology',
                'Clustering_Any_Neuroimaging',
                'Clustering_Any_Omics']

# Creating temporary dataset to work with
try:
    data = df[featuretypes]
except KeyError as e:
    print(f"KeyError: {e}")
    print("Check the following column names:", featuretypes)
    print("Existing columns in df:", df.columns)

# Renaming columns
column_renames = {
    'Year of publication Quantile Division': 'Year of publication Quantile Division',
    'Clustering_Clinical_Any_Motor_Scales_No_Complications': 'Motor scales',
    'Clustering_Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)': 'Motor complications',
    'Clustering_Clinical_Including_Cognition': 'Cognition',
    'Clustering_Clinical_Any_Non_Motor_Excluding_Cognition_QoL_Other': 'Other non-motor',
    'Clustering_Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)': 'Biomarkers',
    'Clustering_Any_Neurophysiology': 'Neurophysiology',
    'Clustering_Any_Neuroimaging': 'Neuroimaging',
    'Clustering_Any_Omics': 'Omics'
}

data.rename(columns=column_renames, inplace=True)

# Showing
calculate_percentage_distribution(data, list(column_renames.values())[1:])

In [ ]:
# Configuration
figurename = 'Clustering feature types according to year - all'

# Grouping by 'Year of publication Quantile Division' and calculating mean for plotting
df_grouped = data.groupby('Year of publication Quantile Division').mean()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
for feature in column_renames.values():
  if feature != 'Year of publication Quantile Division':
    ax.plot(df_grouped.index, df_grouped[feature], marker='o', label=feature)

# Adjusting labels, ticks, and legend
ax.set_ylabel('Proportion of utilization (%)', fontsize=14)
ax.set_xlabel('Year range', fontsize=14)
ax.set_xticks(np.arange(len(df_grouped)))
ax.set_xticklabels(df_grouped.index, rotation=0)
ax.set_yticks(np.arange(0, 1.1, 0.1))  # Adjusting y-ticks to every 0.1
ax.set_yticklabels(['{:.0f}%'.format(x * 100) for x in np.arange(0, 1.1, 0.1)])  # Converting to percentage
ax.tick_params(axis='x', which='major', labelsize=12)  # Increase font size for x-axis ticks
ax.tick_params(axis='y', which='major', labelsize=12)  # Increase font size for x-axis ticks

# Set the y-axis limits to ensure consistent spacing
ax.set_ylim(-0.05, 1.05)

# Customizing the legend
legend = ax.legend(title='', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=11)

# Aesthetic adjustments
ax.set_facecolor('white')  # Set background to white
ax.grid(False)  # Turn off the grid
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Showing
plt.show()

### All features 2.0

In [ ]:
featuretypes = ['Year of publication Quantile Division',
                'Clustering_Any_Clinical',
                'Clustering_Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)',
                'Clustering_Any_Neurophysiology',
                'Clustering_Any_Neuroimaging',
                'Clustering_Any_Omics']

# Creating temporary dataset to work with
try:
    data = df[featuretypes]
except KeyError as e:
    print(f"KeyError: {e}")
    print("Check the following column names:", featuretypes)
    print("Existing columns in df:", df.columns)

# Renaming columns
column_renames = {
    'Year of publication Quantile Division': 'Year of publication Quantile Division',
    'Clustering_Any_Clinical':'Clinical',
    'Clustering_Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)': 'Biomarkers',
    'Clustering_Any_Neurophysiology': 'Neurophysiology',
    'Clustering_Any_Neuroimaging': 'Neuroimaging',
    'Clustering_Any_Omics': 'Omics'
}

data.rename(columns=column_renames, inplace=True)

# Showing
calculate_percentage_distribution(data, list(column_renames.values())[1:])

In [ ]:
# Configuration
figurename = 'Clustering feature types according to year - all 2.0'

# Grouping by 'Year of publication Quantile Division' and calculating mean for plotting
df_grouped = data.groupby('Year of publication Quantile Division').mean()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
for feature in column_renames.values():
  if feature != 'Year of publication Quantile Division':
    ax.plot(df_grouped.index, df_grouped[feature], marker='o', label=feature)

# Adjusting labels, ticks, and legend
ax.set_ylabel('Proportion of utilization (%)', fontsize=14)
ax.set_xlabel('Year range', fontsize=14)
ax.set_xticks(np.arange(len(df_grouped)))
ax.set_xticklabels(df_grouped.index, rotation=0)
ax.set_yticks(np.arange(0, 1.1, 0.1))  # Adjusting y-ticks to every 0.1
ax.set_yticklabels(['{:.0f}%'.format(x * 100) for x in np.arange(0, 1.1, 0.1)])  # Converting to percentage
ax.tick_params(axis='x', which='major', labelsize=12)  # Increase font size for x-axis ticks
ax.tick_params(axis='y', which='major', labelsize=12)  # Increase font size for x-axis ticks

# Set the y-axis limits to ensure consistent spacing
ax.set_ylim(-0.05, 1.05)

# Customizing the legend
legend = ax.legend(title='', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=11)

# Aesthetic adjustments
ax.set_facecolor('white')  # Set background to white
ax.grid(False)  # Turn off the grid
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Showing
plt.show()

### Clinical non-motor

In [ ]:
featuretypes = ['Year of publication Quantile Division',
                'Clustering_Clinical_Including_Cognition',
                'Clustering_Clinical_Any_Neuropsychiatric',
 'Clustering_Clinical_Any_Sleep',
 'Clustering_Clinical_Any_Olfaction',
 'Clustering_Clinical_Any_Autonomic',
 'Clustering_Clinical_QoL',
 'Clustering_Clinical_Non_Motor_Other']

# Creating temporary dataset to work with
data = df[featuretypes]

column_renames = {
    'Clustering_Clinical_Including_Cognition' : 'Cognition',
    'Clustering_Clinical_Any_Neuropsychiatric': 'Neuropsychiatric symptoms',
    'Clustering_Clinical_Any_Sleep': 'Sleep symptoms',
    'Clustering_Clinical_Any_Olfaction': 'Anosmia or hyposmia',
    'Clustering_Clinical_Any_Autonomic': 'Autonomic symptoms',
    'Clustering_Clinical_QoL': 'Quality of life',
    'Clustering_Clinical_Non_Motor_Other': 'Other symptoms'}

data.rename(columns=column_renames, inplace=True)

# Showing
calculate_percentage_distribution(data, column_renames.values())

In [ ]:
# Configuration
figurename = 'Clustering feature types according to year - clinical non-motor detailed'

# Grouping by 'Year of publication Quantile Division' and calculating mean for plotting
df_grouped = data.groupby('Year of publication Quantile Division').mean()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
for feature in column_renames.values():
  if feature != 'Year of publication Quantile Division':
    ax.plot(df_grouped.index, df_grouped[feature], marker='o', label=feature)

# Adjusting labels, ticks, and legend
ax.set_ylabel('Proportion of utilization (%)', fontsize=14)
ax.set_xlabel('Year range', fontsize=14)
ax.set_xticks(np.arange(len(df_grouped)))
ax.set_xticklabels(df_grouped.index, rotation=0)
ax.set_yticks(np.arange(0, 1.1, 0.1))  # Adjusting y-ticks to every 0.1
ax.set_yticklabels(['{:.0f}%'.format(x * 100) for x in np.arange(0, 1.1, 0.1)])  # Converting to percentage
ax.tick_params(axis='x', which='major', labelsize=12)  # Increase font size for x-axis ticks
ax.tick_params(axis='y', which='major', labelsize=12)  # Increase font size for x-axis ticks

# Customizing the legend
legend = ax.legend(title='', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=11)

# Set the y-axis limits to ensure consistent spacing
ax.set_ylim(-0.05, 1.05)

# Aesthetic adjustments
ax.set_facecolor('white')  # Set background to white
ax.grid(False)  # Turn off the grid
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Showing
plt.show()

### Clinical motor

In [ ]:
featuretypes = ['Year of publication Quantile Division',
                'Clustering_Clinical_Any_Motor_Scales_No_Complications',
                'Clustering_Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)',
                 'Clustering_Clinical_Phonation',
 'Clustering_Clinical_Motor_Tests',
 'Clustering_Any_Demographic',
 'Clustering_Any_Time_or_Staging_Measurements',
 'Clustering_Clinical_Vital_or_Neurologic_Examination'
]

# Creating temporary dataset to work with
data = df[featuretypes]

column_renames = {
    'Clustering_Clinical_Any_Motor_Scales_No_Complications': 'Motor scales',
    'Clustering_Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)': 'Motor complications',
        'Clustering_Clinical_Phonation': 'Phonation',
    'Clustering_Clinical_Motor_Tests': 'Motor tests',
    'Clustering_Any_Demographic': 'Demographic data',
    'Clustering_Any_Time_or_Staging_Measurements':'Disease staging',
    'Clustering_Clinical_Vital_or_Neurologic_Examination': 'Neurologic examination findings'

}

data.rename(columns=column_renames, inplace=True)

# Showing
calculate_percentage_distribution(data, list(column_renames.values()))

In [ ]:
# Configuration
figurename = 'Clustering feature types according to year - clinical motor detailed'

# Grouping by 'Year of publication Quantile Division' and calculating mean for plotting
df_grouped = data.groupby('Year of publication Quantile Division').mean()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
for feature in column_renames.values():
  if feature != 'Year of publication Quantile Division':
    ax.plot(df_grouped.index, df_grouped[feature], marker='o', label=feature)

# Adjusting labels, ticks, and legend
ax.set_ylabel('Proportion of utilization (%)', fontsize=14)
ax.set_xlabel('Year range', fontsize=14)
ax.set_xticks(np.arange(len(df_grouped)))
ax.set_xticklabels(df_grouped.index, rotation=0)
ax.set_yticks(np.arange(0, 1.1, 0.1))  # Adjusting y-ticks to every 0.1
ax.set_yticklabels(['{:.0f}%'.format(x * 100) for x in np.arange(0, 1.1, 0.1)])  # Converting to percentage
ax.tick_params(axis='x', which='major', labelsize=12)  # Increase font size for x-axis ticks
ax.tick_params(axis='y', which='major', labelsize=12)  # Increase font size for x-axis ticks

# Customizing the legend
legend = ax.legend(title='', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=11)

# Set the y-axis limits to ensure consistent spacing
ax.set_ylim(-0.05, 1.05)

# Aesthetic adjustments
ax.set_facecolor('white')  # Set background to white
ax.grid(False)  # Turn off the grid
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Showing
plt.show()

### Neuroimaging

In [ ]:
featuretypes = ['Year of publication Quantile Division',
                'Clustering_Neuroimaging - Structural or Volumetric MRI',
 'Clustering_Neuroimaging - Diffusion Imaging MRI',
 'Clustering_Neuroimaging - DaTSCAN',
 'Clustering_Neuroimaging - Radiomics',
 'Clustering_Neuroimaging - Functional MRI',
 'Clustering_Neuroimaging - Other Methods (e.g. neuromelanin-sensitive MRI)']

# Creating temporary dataset to work with
data = df[featuretypes]

# Renaming columns
column_renames = {
    'Clustering_Neuroimaging - Structural or Volumetric MRI': 'Structural or Volumetric MRI',
 'Clustering_Neuroimaging - Diffusion Imaging MRI': 'Diffusion Imaging MRI',
 'Clustering_Neuroimaging - DaTSCAN': 'DaTSCAN',
 'Clustering_Neuroimaging - Radiomics': 'Radiomics',
 'Clustering_Neuroimaging - Functional MRI': 'Functional MRI',
 'Clustering_Neuroimaging - Other Methods (e.g. neuromelanin-sensitive MRI)': 'Other Methods'
}

data.rename(columns=column_renames, inplace=True)

# Showing
calculate_percentage_distribution(data, list(column_renames.values()))

In [ ]:
# Configuration
figurename = 'Clustering feature types according to year - neuroimaging'

# Grouping by 'Year of publication Quantile Division' and calculating mean for plotting
df_grouped = data.groupby('Year of publication Quantile Division').mean()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
for feature in column_renames.values():
  if feature != 'Year of publication Quantile Division':
    ax.plot(df_grouped.index, df_grouped[feature], marker='o', label=feature)

# Adjusting labels, ticks, and legend
ax.set_ylabel('Proportion of utilization (%)', fontsize=14)
ax.set_xlabel('Year range', fontsize=14)
ax.set_xticks(np.arange(len(df_grouped)))
ax.set_xticklabels(df_grouped.index, rotation=0)
ax.set_yticks(np.arange(0, 0.55, 0.05))  # Adjusting y-ticks to every 0.1
ax.set_yticklabels(['{:.0f}%'.format(x * 100) for x in np.arange(0, 0.55, 0.05)])  # Converting to percentage
ax.tick_params(axis='x', which='major', labelsize=12)  # Increase font size for x-axis ticks
ax.tick_params(axis='y', which='major', labelsize=12)  # Increase font size for x-axis ticks

# Customizing the legend
legend = ax.legend(title='', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=11)

# Set the y-axis limits to ensure consistent spacing
ax.set_ylim(-0.05, 0.55)

# Aesthetic adjustments
ax.set_facecolor('white')  # Set background to white
ax.grid(False)  # Turn off the grid
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Showing
plt.show()

### Omics

In [ ]:
featuretypes = ['Year of publication Quantile Division',
                 'Clustering_Omics - Proteomics',
 'Clustering_Omics - Transcriptomics',
 'Clustering_Omics - Genomics',
 'Clustering_Omics - Lipidomics',
 'Clustering_Omics - Other (e.g. metabolomics, metagenomics)']

# Creating temporary dataset to work with
data = df[featuretypes]

column_renames = {
    'Year of publication Quantile Division': 'Year of publication Quantile Division',
    'Clustering_Omics - Proteomics': 'Proteomics',
    'Clustering_Omics - Transcriptomics': 'Transcriptomics',
    'Clustering_Omics - Genomics': 'Genomics',
    'Clustering_Omics - Lipidomics': 'Lipidomics',
    'Clustering_Omics - Other (e.g. metabolomics, metagenomics)': 'Other omics'
}

data.rename(columns=column_renames, inplace=True)

# Showing
calculate_percentage_distribution(data, list(column_renames.values())[1:])

In [ ]:
# Configuration
figurename = 'Clustering feature types according to year - omics'

# Grouping by 'Year of publication Quantile Division' and calculating mean for plotting
df_grouped = data.groupby('Year of publication Quantile Division').mean()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
for feature in column_renames.values():
  if feature != 'Year of publication Quantile Division':
    ax.plot(df_grouped.index, df_grouped[feature], marker='o', label=feature)

# Adjusting labels, ticks, and legend
ax.set_ylabel('Proportion of utilization (%)', fontsize=14)
ax.set_xlabel('Year range', fontsize=14)
ax.set_xticks(np.arange(len(df_grouped)))
ax.set_xticklabels(df_grouped.index, rotation=0)
ax.set_yticks(np.arange(0, 0.55, 0.05))  # Adjusting y-ticks to every 0.1
ax.set_yticklabels(['{:.0f}%'.format(x * 100) for x in np.arange(0, 0.55, 0.05)])  # Converting to percentage
ax.tick_params(axis='x', which='major', labelsize=12)  # Increase font size for x-axis ticks
ax.tick_params(axis='y', which='major', labelsize=12)  # Increase font size for x-axis ticks

# Customizing the legend
legend = ax.legend(title='', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=11)

# Set the y-axis limits to ensure consistent spacing
ax.set_ylim(-0.05, 0.55)

# Aesthetic adjustments
ax.set_facecolor('white')  # Set background to white
ax.grid(False)  # Turn off the grid
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Showing
plt.show()

## Feature types according to year (validation)

### All features

In [ ]:
featuretypes = ['Year of publication Quantile Division',
                'Validation_Clinical_Any_Motor_Scales_No_Complications',
                'Validation_Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)',
                'Validation_Clinical_Including_Cognition',
                'Validation_Clinical_Any_Non_Motor_Excluding_Cognition_QoL_Other',
                'Validation_Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)',
                'Validation_Any_Neurophysiology',
                'Validation_Any_Neuroimaging',
                'Validation_Any_Omics']

# Creating temporary dataset to work with
try:
    data = df[featuretypes]
except KeyError as e:
    print(f"KeyError: {e}")
    print("Check the following column names:", featuretypes)
    print("Existing columns in df:", df.columns)

# Renaming columns
column_renames = {
    'Year of publication Quantile Division': 'Year of publication Quantile Division',
    'Validation_Clinical_Any_Motor_Scales_No_Complications': 'Motor scales',
    'Validation_Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)': 'Motor complications',
    'Validation_Clinical_Including_Cognition': 'Cognition',
    'Validation_Clinical_Any_Non_Motor_Excluding_Cognition_QoL_Other': 'Other non-motor',
    'Validation_Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)': 'Biomarkers',
    'Validation_Any_Neurophysiology': 'Neurophysiology',
    'Validation_Any_Neuroimaging': 'Neuroimaging',
    'Validation_Any_Omics': 'Omics'
}

data.rename(columns=column_renames, inplace=True)

# Showing
calculate_percentage_distribution(data, list(column_renames.values())[1:])

In [ ]:
# Configuration
figurename = 'Validation feature types according to year - all'

# Grouping by 'Year of publication Quantile Division' and calculating mean for plotting
df_grouped = data.groupby('Year of publication Quantile Division').mean()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
for feature in column_renames.values():
  if feature != 'Year of publication Quantile Division':
    ax.plot(df_grouped.index, df_grouped[feature], marker='o', label=feature)

# Adjusting labels, ticks, and legend
ax.set_ylabel('Proportion of utilization (%)', fontsize=14)
ax.set_xlabel('Year range', fontsize=14)
ax.set_xticks(np.arange(len(df_grouped)))
ax.set_xticklabels(df_grouped.index, rotation=0)
ax.set_yticks(np.arange(0, 1.1, 0.1))  # Adjusting y-ticks to every 0.1
ax.set_yticklabels(['{:.0f}%'.format(x * 100) for x in np.arange(0, 1.1, 0.1)])  # Converting to percentage
ax.tick_params(axis='x', which='major', labelsize=12)  # Increase font size for x-axis ticks
ax.tick_params(axis='y', which='major', labelsize=12)  # Increase font size for x-axis ticks

# Set the y-axis limits to ensure consistent spacing
ax.set_ylim(-0.05, 1.05)

# Customizing the legend
legend = ax.legend(title='', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=11)

# Aesthetic adjustments
ax.set_facecolor('white')  # Set background to white
ax.grid(False)  # Turn off the grid
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Showing
plt.show()

### All features 2.0

In [ ]:
featuretypes = ['Year of publication Quantile Division',
                'Validation_Any_Clinical',
                'Validation_Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)',
                'Validation_Any_Neurophysiology',
                'Validation_Any_Neuroimaging',
                'Validation_Any_Omics']

# Creating temporary dataset to work with
try:
    data = df[featuretypes]
except KeyError as e:
    print(f"KeyError: {e}")
    print("Check the following column names:", featuretypes)
    print("Existing columns in df:", df.columns)

# Renaming columns
column_renames = {
    'Year of publication Quantile Division': 'Year of publication Quantile Division',
    'Validation_Any_Clinical':'Clinical',
    'Validation_Biomarkers (uric acid, LDL, tryglicerides, neurofilament light [Nfl], tau, SAA etc)': 'Biomarkers',
    'Validation_Any_Neurophysiology': 'Neurophysiology',
    'Validation_Any_Neuroimaging': 'Neuroimaging',
    'Validation_Any_Omics': 'Omics'
}

data.rename(columns=column_renames, inplace=True)

# Showing
calculate_percentage_distribution(data, list(column_renames.values())[1:])

In [ ]:
# Configuration
figurename = 'Validation feature types according to year - all 2.0'

# Grouping by 'Year of publication Quantile Division' and calculating mean for plotting
df_grouped = data.groupby('Year of publication Quantile Division').mean()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
for feature in column_renames.values():
  if feature != 'Year of publication Quantile Division':
    ax.plot(df_grouped.index, df_grouped[feature], marker='o', label=feature)

# Adjusting labels, ticks, and legend
ax.set_ylabel('Proportion of utilization (%)', fontsize=14)
ax.set_xlabel('Year range', fontsize=14)
ax.set_xticks(np.arange(len(df_grouped)))
ax.set_xticklabels(df_grouped.index, rotation=0)
ax.set_yticks(np.arange(0, 1.1, 0.1))  # Adjusting y-ticks to every 0.1
ax.set_yticklabels(['{:.0f}%'.format(x * 100) for x in np.arange(0, 1.1, 0.1)])  # Converting to percentage
ax.tick_params(axis='x', which='major', labelsize=12)  # Increase font size for x-axis ticks
ax.tick_params(axis='y', which='major', labelsize=12)  # Increase font size for x-axis ticks

# Set the y-axis limits to ensure consistent spacing
ax.set_ylim(-0.05, 1.05)

# Customizing the legend
legend = ax.legend(title='', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=11)

# Aesthetic adjustments
ax.set_facecolor('white')  # Set background to white
ax.grid(False)  # Turn off the grid
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Showing
plt.show()

### Clinical non-motor

In [ ]:
featuretypes = ['Year of publication Quantile Division',
                'Validation_Clinical_Including_Cognition',
                'Validation_Clinical_Any_Neuropsychiatric',
 'Validation_Clinical_Any_Sleep',
 'Validation_Clinical_Any_Olfaction',
 'Validation_Clinical_Any_Autonomic',
 'Validation_Clinical_QoL',
 'Validation_Clinical_Non_Motor_Other']

# Creating temporary dataset to work with
data = df[featuretypes]

column_renames = {
    'Validation_Clinical_Including_Cognition' : 'Cognition',
    'Validation_Clinical_Any_Neuropsychiatric': 'Neuropsychiatric symptoms',
    'Validation_Clinical_Any_Sleep': 'Sleep symptoms',
    'Validation_Clinical_Any_Olfaction': 'Anosmia or hyposmia',
    'Validation_Clinical_Any_Autonomic': 'Autonomic symptoms',
    'Validation_Clinical_QoL': 'Quality of life',
    'Validation_Clinical_Non_Motor_Other': 'Other symptoms'}

data.rename(columns=column_renames, inplace=True)

# Showing
calculate_percentage_distribution(data, column_renames.values())

In [ ]:
# Configuration
figurename = 'Validation feature types according to year - clinical non-motor detailed'

# Grouping by 'Year of publication Quantile Division' and calculating mean for plotting
df_grouped = data.groupby('Year of publication Quantile Division').mean()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
for feature in column_renames.values():
  if feature != 'Year of publication Quantile Division':
    ax.plot(df_grouped.index, df_grouped[feature], marker='o', label=feature)

# Adjusting labels, ticks, and legend
ax.set_ylabel('Proportion of utilization (%)', fontsize=14)
ax.set_xlabel('Year range', fontsize=14)
ax.set_xticks(np.arange(len(df_grouped)))
ax.set_xticklabels(df_grouped.index, rotation=0)
ax.set_yticks(np.arange(0, 1.1, 0.1))  # Adjusting y-ticks to every 0.1
ax.set_yticklabels(['{:.0f}%'.format(x * 100) for x in np.arange(0, 1.1, 0.1)])  # Converting to percentage
ax.tick_params(axis='x', which='major', labelsize=12)  # Increase font size for x-axis ticks
ax.tick_params(axis='y', which='major', labelsize=12)  # Increase font size for x-axis ticks

# Customizing the legend
legend = ax.legend(title='', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=11)

# Set the y-axis limits to ensure consistent spacing
ax.set_ylim(-0.05, 1.05)

# Aesthetic adjustments
ax.set_facecolor('white')  # Set background to white
ax.grid(False)  # Turn off the grid
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Showing
plt.show()

### Clinical motor

In [ ]:
featuretypes = ['Year of publication Quantile Division',
                'Validation_Clinical_Any_Motor_Scales_No_Complications',
                'Validation_Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)',
                 'Validation_Clinical_Phonation',
 'Validation_Clinical_Motor_Tests',
 'Validation_Any_Demographic',
 'Validation_Any_Time_or_Staging_Measurements',
 'Validation_Clinical_Vital_or_Neurologic_Examination'
]

# Creating temporary dataset to work with
data = df[featuretypes]

column_renames = {
    'Validation_Clinical_Any_Motor_Scales_No_Complications': 'Motor scales',
    'Validation_Clinical - Motor Complications (dyskinesia, OFF periods, dystonia, motor fluctuations)': 'Motor complications',
        'Validation_Clinical_Phonation': 'Phonation',
    'Validation_Clinical_Motor_Tests': 'Motor tests',
    'Validation_Any_Demographic': 'Demographic data',
    'Validation_Any_Time_or_Staging_Measurements':'Disease staging',
    'Validation_Clinical_Vital_or_Neurologic_Examination': 'Neurologic examination findings'

}

data.rename(columns=column_renames, inplace=True)

# Showing
calculate_percentage_distribution(data, column_renames.values())

In [ ]:
# Configuration
figurename = 'Validation feature types according to year - clinical motor detailed'

# Grouping by 'Year of publication Quantile Division' and calculating mean for plotting
df_grouped = data.groupby('Year of publication Quantile Division').mean()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
for feature in column_renames.values():
  if feature != 'Year of publication Quantile Division':
    ax.plot(df_grouped.index, df_grouped[feature], marker='o', label=feature)

# Adjusting labels, ticks, and legend
ax.set_ylabel('Proportion of utilization (%)', fontsize=14)
ax.set_xlabel('Year range', fontsize=14)
ax.set_xticks(np.arange(len(df_grouped)))
ax.set_xticklabels(df_grouped.index, rotation=0)
ax.set_yticks(np.arange(0, 1.1, 0.1))  # Adjusting y-ticks to every 0.1
ax.set_yticklabels(['{:.0f}%'.format(x * 100) for x in np.arange(0, 1.1, 0.1)])  # Converting to percentage
ax.tick_params(axis='x', which='major', labelsize=12)  # Increase font size for x-axis ticks
ax.tick_params(axis='y', which='major', labelsize=12)  # Increase font size for x-axis ticks

# Customizing the legend
legend = ax.legend(title='', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=11)

# Set the y-axis limits to ensure consistent spacing
ax.set_ylim(-0.05, 1.05)

# Aesthetic adjustments
ax.set_facecolor('white')  # Set background to white
ax.grid(False)  # Turn off the grid
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Showing
plt.show()

### Neuroimaging

In [ ]:
featuretypes = ['Year of publication Quantile Division',
                'Validation_Neuroimaging - Structural or Volumetric MRI',
 'Validation_Neuroimaging - Diffusion Imaging MRI',
 'Validation_Neuroimaging - DaTSCAN',
 'Validation_Neuroimaging - Radiomics',
 'Validation_Neuroimaging - Functional MRI',
 'Validation_Neuroimaging - Other Methods (e.g. neuromelanin-sensitive MRI)']

# Creating temporary dataset to work with
data = df[featuretypes]

# Renaming columns
column_renames = {
    'Validation_Neuroimaging - Structural or Volumetric MRI': 'Structural or Volumetric MRI',
 'Validation_Neuroimaging - Diffusion Imaging MRI': 'Diffusion Imaging MRI',
 'Validation_Neuroimaging - DaTSCAN': 'DaTSCAN',
 'Validation_Neuroimaging - Radiomics': 'Radiomics',
 'Validation_Neuroimaging - Functional MRI': 'Functional MRI',
 'Validation_Neuroimaging - Other Methods (e.g. neuromelanin-sensitive MRI)': 'Other Methods'
}

data.rename(columns=column_renames, inplace=True)

# Showing
calculate_percentage_distribution(data, column_renames.values())

In [ ]:
# Configuration
figurename = 'Validation feature types according to year - neuroimaging'

# Grouping by 'Year of publication Quantile Division' and calculating mean for plotting
df_grouped = data.groupby('Year of publication Quantile Division').mean()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
for feature in column_renames.values():
  if feature != 'Year of publication Quantile Division':
    ax.plot(df_grouped.index, df_grouped[feature], marker='o', label=feature)

# Adjusting labels, ticks, and legend
ax.set_ylabel('Proportion of utilization (%)', fontsize=14)
ax.set_xlabel('Year range', fontsize=14)
ax.set_xticks(np.arange(len(df_grouped)))
ax.set_xticklabels(df_grouped.index, rotation=0)
ax.set_yticks(np.arange(0, 0.55, 0.05))  # Adjusting y-ticks to every 0.1
ax.set_yticklabels(['{:.0f}%'.format(x * 100) for x in np.arange(0, 0.55, 0.05)])  # Converting to percentage
ax.tick_params(axis='x', which='major', labelsize=12)  # Increase font size for x-axis ticks
ax.tick_params(axis='y', which='major', labelsize=12)  # Increase font size for x-axis ticks

# Customizing the legend
legend = ax.legend(title='', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=11)

# Set the y-axis limits to ensure consistent spacing
ax.set_ylim(-0.05, 0.55)

# Aesthetic adjustments
ax.set_facecolor('white')  # Set background to white
ax.grid(False)  # Turn off the grid
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Showing
plt.show()

### Omics

In [ ]:
featuretypes = ['Year of publication Quantile Division',
                 'Validation_Omics - Proteomics',
 'Validation_Omics - Transcriptomics',
 'Validation_Omics - Genomics',
 'Validation_Omics - Lipidomics',
 'Validation_Omics - Other (e.g. metabolomics, metagenomics)']

# Creating temporary dataset to work with
data = df[featuretypes]

column_renames = {
    'Year of publication Quantile Division': 'Year of publication Quantile Division',
    'Validation_Omics - Proteomics': 'Proteomics',
    'Validation_Omics - Transcriptomics': 'Transcriptomics',
    'Validation_Omics - Genomics': 'Genomics',
    'Validation_Omics - Lipidomics': 'Lipidomics',
    'Validation_Omics - Other (e.g. metabolomics, metagenomics)': 'Other omics'
}

data.rename(columns=column_renames, inplace=True)

# Showing
calculate_percentage_distribution(data, list(column_renames.values())[1:])

In [ ]:
# Configuration
figurename = 'Validation feature types according to year - omics'

# Grouping by 'Year of publication Quantile Division' and calculating mean for plotting
df_grouped = data.groupby('Year of publication Quantile Division').mean()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
for feature in column_renames.values():
  if feature != 'Year of publication Quantile Division':
    ax.plot(df_grouped.index, df_grouped[feature], marker='o', label=feature)

# Adjusting labels, ticks, and legend
ax.set_ylabel('Proportion of utilization (%)', fontsize=14)
ax.set_xlabel('Year range', fontsize=14)
ax.set_xticks(np.arange(len(df_grouped)))
ax.set_xticklabels(df_grouped.index, rotation=0)
ax.set_yticks(np.arange(0, 0.55, 0.05))  # Adjusting y-ticks to every 0.1
ax.set_yticklabels(['{:.0f}%'.format(x * 100) for x in np.arange(0, 0.55, 0.05)])  # Converting to percentage
ax.tick_params(axis='x', which='major', labelsize=12)  # Increase font size for x-axis ticks
ax.tick_params(axis='y', which='major', labelsize=12)  # Increase font size for x-axis ticks

# Customizing the legend
legend = ax.legend(title='', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=11)

# Set the y-axis limits to ensure consistent spacing
ax.set_ylim(-0.05, 0.55)

# Aesthetic adjustments
ax.set_facecolor('white')  # Set background to white
ax.grid(False)  # Turn off the grid
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Showing
plt.show()

## Feature types according to year (clustering or validation)

Not used in the manuscript

### All features

In [ ]:
featuretypes = ['Year of publication Quantile Division',
                'Utilized_Motor_Scales_no_Complications_Either_in_Clustering_or_Validation',
                'Utilized_Motor_Complications_Either_in_Clustering_or_Validation',
                'Utilized_Cognition_Either_in_Clustering_or_Validation',
                'Utilized_Non_Motor_Excluding_Cognition_QoL_Other_Either_in_Clustering_or_Validation',
                'Utilized_Any_Biomarkers_Either_in_Clustering_or_Validation',
                'Utilized_Any_Neurophisiology_Either_in_Clustering_or_Validation',
                'Utilized_Any_Neuroimaging_Either_in_Clustering_or_Validation',
                'Utilized_Any_Omics_Either_in_Clustering_or_Validation']

# Creating temporary dataset to work with
try:
    data = df[featuretypes]
except KeyError as e:
    print(f"KeyError: {e}")
    print("Check the following column names:", featuretypes)
    print("Existing columns in df:", df.columns)

# Renaming columns
column_renames = {
    'Year of publication Quantile Division': 'Year of publication Quantile Division',
    'Utilized_Motor_Scales_no_Complications_Either_in_Clustering_or_Validation': 'Motor scales',
    'Utilized_Motor_Complications_Either_in_Clustering_or_Validation': 'Motor complications',
    'Utilized_Cognition_Either_in_Clustering_or_Validation': 'Cognition',
    'Utilized_Non_Motor_Excluding_Cognition_QoL_Other_Either_in_Clustering_or_Validation': 'Other non-motor',
    'Utilized_Any_Biomarkers_Either_in_Clustering_or_Validation': 'Biomarkers',
    'Utilized_Any_Neurophisiology_Either_in_Clustering_or_Validation': 'Neurophysiology',
    'Utilized_Any_Neuroimaging_Either_in_Clustering_or_Validation': 'Neuroimaging',
    'Utilized_Any_Omics_Either_in_Clustering_or_Validation': 'Omics'
}

data.rename(columns=column_renames, inplace=True)

# Showing
calculate_percentage_distribution(data, list(column_renames.values())[1:])

In [ ]:
# Configuration
figurename = 'Clustering or validation feature types according to year - all'

# Grouping by 'Year of publication Quantile Division' and calculating mean for plotting
df_grouped = data.groupby('Year of publication Quantile Division').mean()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
for feature in column_renames.values():
  if feature != 'Year of publication Quantile Division':
    ax.plot(df_grouped.index, df_grouped[feature], marker='o', label=feature)

# Adjusting labels, ticks, and legend
ax.set_ylabel('Proportion of utilization (%)', fontsize=14)
ax.set_xlabel('Year range', fontsize=14)
ax.set_xticks(np.arange(len(df_grouped)))
ax.set_xticklabels(df_grouped.index, rotation=0)
ax.set_yticks(np.arange(0, 1.1, 0.1))  # Adjusting y-ticks to every 0.1
ax.set_yticklabels(['{:.0f}%'.format(x * 100) for x in np.arange(0, 1.1, 0.1)])  # Converting to percentage
ax.tick_params(axis='x', which='major', labelsize=12)  # Increase font size for x-axis ticks
ax.tick_params(axis='y', which='major', labelsize=12)  # Increase font size for x-axis ticks

# Set the y-axis limits to ensure consistent spacing
ax.set_ylim(-0.05, 1.05)

# Customizing the legend
legend = ax.legend(title='', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=11)

# Aesthetic adjustments
ax.set_facecolor('white')  # Set background to white
ax.grid(False)  # Turn off the grid
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Showing
plt.show()

### Clinical special

In [ ]:
featuretypes = ['Year of publication Quantile Division',
                'Clinical - Neuropsychiatric (anxiety, depression, impulsivity, delusions, hallucinations etc)_Either_in_Clustering_or_Validation',
 'Clinical - Sleep (REM sleep behavior disorder, insomnia, daytime sleepiness)_Either_in_Clustering_or_Validation',
 'Clinical - Anosmia or Hyposmia (absence or loss of the capability to smell)_Either_in_Clustering_or_Validation',
 'Clinical - Autonomic (gastroparesis, constipation, orthostatic hypotension, urinary incontinence, sexual difficulties)_Either_in_Clustering_or_Validation',
 'Clinical - Quality of Life_Either_in_Clustering_or_Validation',
 'Clinical - Other (pain, fatigue etc)_Either_in_Clustering_or_Validation',
 'Utilized_Demographic_Either_in_Clustering_or_Validation',
 'Utilized_Time_or_Staging_Measurements_Either_in_Clustering_or_Validation',
 'Clinical - Vital Signs or Neurologic Examination Findings_Either_in_Clustering_or_Validation',
 'Clinical - Phonation_Either_in_Clustering_or_Validation',
 'Clinical - Motor Tests (example: Timed-Up & Go)_Either_in_Clustering_or_Validation']

# Creating temporary dataset to work with
data = df[featuretypes]

column_renames = {
    'Clinical - Neuropsychiatric (anxiety, depression, impulsivity, delusions, hallucinations etc)_Either_in_Clustering_or_Validation': 'Neuropsychiatric symptoms',
    'Clinical - Sleep (REM sleep behavior disorder, insomnia, daytime sleepiness)_Either_in_Clustering_or_Validation': 'Sleep symptoms',
    'Clinical - Anosmia or Hyposmia (absence or loss of the capability to smell)_Either_in_Clustering_or_Validation': 'Anosmia or hyposmia',
    'Clinical - Autonomic (gastroparesis, constipation, orthostatic hypotension, urinary incontinence, sexual difficulties)_Either_in_Clustering_or_Validation': 'Autonomic symptoms',
    'Clinical - Quality of Life_Either_in_Clustering_or_Validation': 'Quality of life',
    'Clinical - Other (pain, fatigue etc)_Either_in_Clustering_or_Validation': 'Other symptoms',
    'Utilized_Demographic_Either_in_Clustering_or_Validation': 'Demographic data',
    'Utilized_Time_or_Staging_Measurements_Either_in_Clustering_or_Validation':'Disease staging',
    'Clinical - Vital Signs or Neurologic Examination Findings_Either_in_Clustering_or_Validation': 'Neurologic examination findings',
    'Clinical - Phonation_Either_in_Clustering_or_Validation': 'Phonation',
    'Clinical - Motor Tests (example: Timed-Up & Go)_Either_in_Clustering_or_Validation': 'Motor tests'
}

data.rename(columns=column_renames, inplace=True)

# Showing
calculate_percentage_distribution(data, column_renames.values())

In [ ]:
# Configuration
figurename = 'Clustering or validation feature types according to year - clinical detailed'

# Grouping by 'Year of publication Quantile Division' and calculating mean for plotting
df_grouped = data.groupby('Year of publication Quantile Division').mean()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
for feature in column_renames.values():
  if feature != 'Year of publication Quantile Division':
    ax.plot(df_grouped.index, df_grouped[feature], marker='o', label=feature)

# Adjusting labels, ticks, and legend
ax.set_ylabel('Proportion of utilization (%)', fontsize=14)
ax.set_xlabel('Year range', fontsize=14)
ax.set_xticks(np.arange(len(df_grouped)))
ax.set_xticklabels(df_grouped.index, rotation=0)
ax.set_yticks(np.arange(0, 1.1, 0.1))  # Adjusting y-ticks to every 0.1
ax.set_yticklabels(['{:.0f}%'.format(x * 100) for x in np.arange(0, 1.1, 0.1)])  # Converting to percentage
ax.tick_params(axis='x', which='major', labelsize=12)  # Increase font size for x-axis ticks
ax.tick_params(axis='y', which='major', labelsize=12)  # Increase font size for x-axis ticks

# Customizing the legend
legend = ax.legend(title='', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=11)

# Set the y-axis limits to ensure consistent spacing
ax.set_ylim(-0.05, 1.05)

# Aesthetic adjustments
ax.set_facecolor('white')  # Set background to white
ax.grid(False)  # Turn off the grid
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Showing
plt.show()

### Neuroimaging

In [ ]:
featuretypes = ['Year of publication Quantile Division',
                'Neuroimaging - Structural or Volumetric MRI_Either_in_Clustering_or_Validation',
 'Neuroimaging - Diffusion Imaging MRI_Either_in_Clustering_or_Validation',
 'Neuroimaging - DaTSCAN_Either_in_Clustering_or_Validation',
 'Neuroimaging - Radiomics_Either_in_Clustering_or_Validation',
 'Neuroimaging - Functional MRI_Either_in_Clustering_or_Validation',
 'Neuroimaging - Other Methods (e.g. neuromelanin-sensitive MRI)_Either_in_Clustering_or_Validation']

# Creating temporary dataset to work with
data = df[featuretypes]

# Renaming columns
column_renames = {
    'Neuroimaging - Structural or Volumetric MRI_Either_in_Clustering_or_Validation': 'Structural or Volumetric MRI',
 'Neuroimaging - Diffusion Imaging MRI_Either_in_Clustering_or_Validation': 'Diffusion Imaging MRI',
 'Neuroimaging - DaTSCAN_Either_in_Clustering_or_Validation': 'DaTSCAN',
 'Neuroimaging - Radiomics_Either_in_Clustering_or_Validation': 'Radiomics',
 'Neuroimaging - Functional MRI_Either_in_Clustering_or_Validation': 'Functional MRI',
 'Neuroimaging - Other Methods (e.g. neuromelanin-sensitive MRI)_Either_in_Clustering_or_Validation': 'Other Methods'
}

data.rename(columns=column_renames, inplace=True)

# Showing
calculate_percentage_distribution(data, column_renames.values())

In [ ]:
# Configuration
figurename = 'Clustering or validation feature types according to year - neuroimaging'

# Grouping by 'Year of publication Quantile Division' and calculating mean for plotting
df_grouped = data.groupby('Year of publication Quantile Division').mean()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
for feature in column_renames.values():
  if feature != 'Year of publication Quantile Division':
    ax.plot(df_grouped.index, df_grouped[feature], marker='o', label=feature)

# Adjusting labels, ticks, and legend
ax.set_ylabel('Proportion of utilization (%)', fontsize=14)
ax.set_xlabel('Year range', fontsize=14)
ax.set_xticks(np.arange(len(df_grouped)))
ax.set_xticklabels(df_grouped.index, rotation=0)
ax.set_yticks(np.arange(0, 0.55, 0.05))  # Adjusting y-ticks to every 0.1
ax.set_yticklabels(['{:.0f}%'.format(x * 100) for x in np.arange(0, 0.55, 0.05)])  # Converting to percentage
ax.tick_params(axis='x', which='major', labelsize=12)  # Increase font size for x-axis ticks
ax.tick_params(axis='y', which='major', labelsize=12)  # Increase font size for x-axis ticks

# Customizing the legend
legend = ax.legend(title='', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=11)

# Set the y-axis limits to ensure consistent spacing
ax.set_ylim(-0.05, 0.55)

# Aesthetic adjustments
ax.set_facecolor('white')  # Set background to white
ax.grid(False)  # Turn off the grid
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Showing
plt.show()

### Omics

In [ ]:
featuretypes = ['Year of publication Quantile Division',
                 'Omics - Proteomics_Either_in_Clustering_or_Validation',
 'Omics - Transcriptomics_Either_in_Clustering_or_Validation',
 'Omics - Genomics_Either_in_Clustering_or_Validation',
 'Omics - Lipidomics_Either_in_Clustering_or_Validation',
 'Omics - Other (e.g. metabolomics, metagenomics)_Either_in_Clustering_or_Validation']

# Creating temporary dataset to work with
data = df[featuretypes]

column_renames = {
    'Year of publication Quantile Division': 'Year of publication Quantile Division',
    'Omics - Proteomics_Either_in_Clustering_or_Validation': 'Proteomics',
    'Omics - Transcriptomics_Either_in_Clustering_or_Validation': 'Transcriptomics',
    'Omics - Genomics_Either_in_Clustering_or_Validation': 'Genomics',
    'Omics - Lipidomics_Either_in_Clustering_or_Validation': 'Lipidomics',
    'Omics - Other (e.g. metabolomics, metagenomics)_Either_in_Clustering_or_Validation': 'Other omics'
}

data.rename(columns=column_renames, inplace=True)

# Showing
calculate_percentage_distribution(data, list(column_renames.values())[1:])

In [ ]:
# Configuration
figurename = 'Clustering or validation feature types according to year - omics'

# Grouping by 'Year of publication Quantile Division' and calculating mean for plotting
df_grouped = data.groupby('Year of publication Quantile Division').mean()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
for feature in column_renames.values():
  if feature != 'Year of publication Quantile Division':
    ax.plot(df_grouped.index, df_grouped[feature], marker='o', label=feature)

# Adjusting labels, ticks, and legend
ax.set_ylabel('Proportion of utilization (%)', fontsize=14)
ax.set_xlabel('Year range', fontsize=14)
ax.set_xticks(np.arange(len(df_grouped)))
ax.set_xticklabels(df_grouped.index, rotation=0)
ax.set_yticks(np.arange(0, 0.55, 0.05))  # Adjusting y-ticks to every 0.1
ax.set_yticklabels(['{:.0f}%'.format(x * 100) for x in np.arange(0, 0.55, 0.05)])  # Converting to percentage
ax.tick_params(axis='x', which='major', labelsize=12)  # Increase font size for x-axis ticks
ax.tick_params(axis='y', which='major', labelsize=12)  # Increase font size for x-axis ticks

# Customizing the legend
legend = ax.legend(title='', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=11)

# Set the y-axis limits to ensure consistent spacing
ax.set_ylim(-0.05, 0.55)

# Aesthetic adjustments
ax.set_facecolor('white')  # Set background to white
ax.grid(False)  # Turn off the grid
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Showing
plt.show()

## Cohorts used

In [ ]:
featuretypes = ['Year of publication Quantile Division',
                'Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [AMP-PD]',
 'Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [BioFIND]',
 'Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Fox Insight]',
 'Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [GP2 Dataset]',
 'Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [LRRK2 Cohort Consortium]',
 'Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Parkinson Progression Marker Initiative (PPMI)]',
 "Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Parkinson's Disease Biomarker Program (PDBP)]",
 'Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [UK Biobank]',
 'Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Oxford Parkinson Disease Center Discovery Cohort]',
 'Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Other specific or local datasets]']

# Creating temporary dataset to work with
data = df[featuretypes]

column_renames = column_renames = {'Year of publication Quantile Division':'Year of publication Quantile Division',
    'Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [AMP-PD]': 'AMP-PD',
    'Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [BioFIND]': 'BioFIND',
    'Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Fox Insight]': 'Fox Insight',
    'Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [GP2 Dataset]': 'GP2',
    'Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [LRRK2 Cohort Consortium]': 'LRRK2 Cohort Consortium',
    'Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Parkinson Progression Marker Initiative (PPMI)]': 'PPMI',
    "Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Parkinson's Disease Biomarker Program (PDBP)]": "PDBP",
    'Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [UK Biobank]': 'UK Biobank',
    'Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Oxford Parkinson Disease Center Discovery Cohort]' : 'OPDC Discovery',
    'Dataset_Used Which datasets were utilized in the creation and/or validation of the subtyping algorithm? [Other specific or local datasets]': 'Other specific or local datasets'
}

data.rename(columns=column_renames, inplace=True)

# Showing
calculate_percentage_distribution(data, list(column_renames.values())[1:])

In [ ]:
# Droping columns with 0 values but also the specific datasets
# List of columns to remove
cols_to_remove = ['AMP-PD', 'Fox Insight', 'GP2',  'LRRK2 Cohort Consortium', 'UK Biobank', 'Other specific or local datasets']

# Drop the columns
try:
  data = data.drop(cols_to_remove, axis=1)
except Exception as e:
  print(e)

In [ ]:
# Configuration
figurename = 'Cohorts used during the years'

# Grouping by 'Year of publication Quantile Division' and calculating mean for plotting
df_grouped = data.groupby('Year of publication Quantile Division').mean()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
for feature in data.columns:
  if feature != 'Year of publication Quantile Division':
    ax.plot(df_grouped.index, df_grouped[feature], marker='o', label=feature)

# Adjusting labels, ticks, and legend
ax.set_ylabel('Proportion of utilization (%)', fontsize=14)
ax.set_xlabel('Year range', fontsize=14)
ax.set_xticks(np.arange(len(df_grouped)))
ax.set_xticklabels(df_grouped.index, rotation=0)
ax.set_yticks(np.arange(0, 0.6, 0.1))  # Adjusting y-ticks to every 0.1
ax.set_yticklabels(['{:.0f}%'.format(x * 100) for x in np.arange(0, 0.6, 0.1)])  # Converting to percentage
ax.tick_params(axis='x', which='major', labelsize=12)  # Increase font size for x-axis ticks
ax.tick_params(axis='y', which='major', labelsize=12)  # Increase font size for x-axis ticks

# Customizing the legend
legend = ax.legend(title='', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=11)

# Aesthetic adjustments
ax.set_facecolor('white')  # Set background to white
ax.grid(False)  # Turn off the grid
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

# Showing
plt.show()

## Scales utilized

Radar chart

In [ ]:
# Removed (no interest) - 'Income', 'Vital Signs (blood pressure, cardiac frequency)', 'Physical Attributes (weight, height etc)'

descriptivedata = ['Before_2020_After',
                   'UPDRS motor scores','UPDRS non-motor scores',
                   'MDS-UPDRS Part I', 'MDS-UPDRS Part II', 'MDS-UPDRS Part III', 'MDS-UPDRS Part IV',
                   'The Schwab and England ADL (Activities of Daily Living) scale',
                   'Hoehn and Yahr Scale or Modified Hoehn and Yahr Scale',
                   'University of Pennsylvania Smell Identification Test (UPSIT)',
                   'Non-Motor Symptoms Scale (NNMS)',
                   'Non-Motor Symptoms Questionnaire (NMSQ)',
                   'Montreal Cognitive Assessment (MoCA)',
                   'Mini-Mental State Examination (MMSE)',
                   'SCOPA-AUT',
                   'Geriatric Depression Scale (GDS)', 'Beck Depression Inventory (BDI)',
                   'Hospital Anxiety and Depression Scale (HADS)', 'State-Trait Anxiety Inventory for Adults (STAI)',
                   'Questionnaire for Impulsive-Compulsive Disorders in Parkinson’s Disease (QUIP)',
                   'REM-Sleep Behavior Disorder Screening Questionnaire (RBDSQ)', 'Epworth Sleepiness Scale (ESS)']

In [ ]:
data = df[descriptivedata]

# Renaming columns
column_renames ={
    'Before_2020_After':'Year Category',
    'UPDRS motor scores': 'Motor UPDRS',
    'UPDRS non-motor scores': 'Non-motor UPDRS',
    'MDS-UPDRS Part I': 'MDS-UPDRS I',
    'MDS-UPDRS Part II': 'MDS-UPDRS II',
    'MDS-UPDRS Part III': 'MDS-UPDRS III',
    'MDS-UPDRS Part IV': 'MDS-UPDRS IV',
    'The Schwab and England ADL (Activities of Daily Living) scale': 'Schwab & England ADL',
    'Hoehn and Yahr Scale or Modified Hoehn and Yahr Scale': 'Hoehn & Yahr Scale',
    'University of Pennsylvania Smell Identification Test (UPSIT)': 'UPSIT',
    'Non-Motor Symptoms Scale (NNMS)': 'NNMS',
    'Non-Motor Symptoms Questionnaire (NMSQ)': 'NMSQ',
    'Montreal Cognitive Assessment (MoCA)': 'MoCA',
    'Mini-Mental State Examination (MMSE)': 'MMSE',
    'SCOPA-AUT': 'SCOPA-AUT',
    'Geriatric Depression Scale (GDS)': 'GDS',
    'Beck Depression Inventory (BDI)': 'BDI',
    'Hospital Anxiety and Depression Scale (HADS)': 'HADS',
    'State-Trait Anxiety Inventory for Adults (STAI)': 'STAI',
    'Questionnaire for Impulsive-Compulsive Disorders in Parkinson’s Disease (QUIP)': 'QUIP',
    'REM-Sleep Behavior Disorder Screening Questionnaire (RBDSQ)': 'RBDSQ',
    'Epworth Sleepiness Scale (ESS)': 'ESS'
}

data.rename(columns=column_renames, inplace=True)

# Calculate percentages
percentages = (data[list(column_renames.values())[1:]].mean() * 100).round()
top_percentages = percentages.nlargest(15)  # Select top 15 values

# Showing
calculate_percentage_distribution(data, list(column_renames.values())[1:])

In [ ]:
!pip install kaleido
import kaleido

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Configuration
figurename = 'Clinical scales radar chart'

# Calculate the sum of each column, then compute the percentage relative to the total observations
total_observations = len(data)
column_percentages = (data[list(data.columns[1:])].sum() / total_observations * 100).round(0)

# Sort the percentages and select the top 15
top_15 = column_percentages.sort_values(ascending=False).head(15)

# Categories and values for the radar chart, closing the loop by appending the first element at the end
categories = list(top_15.index) + [top_15.index[0]]
values = list(top_15.values) + [top_15.values[0]]

# Create the radar chart
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=values,
    theta=categories,
    fill='toself',
    name='Top 15 Feature Percentages'
))

# Update layout to make it more visually appealing and simpler
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, max(values) + 10],  # Adjust the range to maximum observed value + 10
            gridcolor='#F0F8FF',  # Change grid line color to light blue
            gridwidth=2,  # Increase grid line width to 1
            color='black',  # Standard color for text
            tickvals=list(range(0, int(max(values) + 10), 10)),  # Setting ticks every 10%
            tickfont=dict(size=15, color='black'),  # Font size and color for tick labels
            linecolor='black'  # Color of the radial axis lines
        ),
        angularaxis=dict(  # Customize angular axis
            tickfont=dict(size=17, color='black'),  # Font size and color for category labels
            rotation=90,  # Rotate labels for better layout
            direction='clockwise'  # Direction of categories
        )
    ),
    title_font=dict(size=14, color='black'),  # Subdued title font size and color
    showlegend=False,
    margin=dict(
        b=40,
        r=45,
        l=45,
        t=40
    )
)

# Show the plot
fig.show()

# If you need to save the plot to a file, uncomment and adapt the line below
import plotly.io as pio
pio.write_image(fig, 'Plots/SVG/' + figurename + '.svg', format='svg')
pio.write_image(fig, 'Plots/PNG/' + figurename + '.png', format='png')

## Bubble Plot

**X** = Years | **Y** = Number of clustering features | **Bubble size** = patients in cluster | **Bubble color** = use of non-clinical feature for clustering | **Bubble opacity** = quality scores

In [ ]:
data = df[['Year of publication', 'Clustering_Number_of_Specific_Domains', 'Number of PD patients utilized for clustering purposes for bubble', 'Clustering_Any_Non_Clinical','Quality_Index_Mestre']].astype(int)

for column in data:
  data[column] = pd.to_numeric(data[column], errors='coerce')

data.describe()

In [ ]:
df['Number of PD patients utilized for clustering purposes for bubble'].astype(int).describe()

In [ ]:
df['Number of PD patients utilized for clustering purposes for bubble'].astype(int).value_counts(dropna=False)

In [ ]:
max(df['Clustering_Number_of_Specific_Domains'])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

figurename = 'Bubble plot of number of features per year according to usage of non-clinical data and quality index'

plt.figure(figsize=(10, 6))

# Adjust transparency to range from 40% to 100%
transparency = 0.2 + 0.8 * (data['Quality_Index_Mestre'] / 15)

# Using the original color palette but ensuring it's for binary data
cmap = plt.get_cmap('coolwarm', 2)  # 2 distinct colors for binary data

# Further reduced bubble size
size_scale = data['Number of PD patients utilized for clustering purposes for bubble'] * 0.8  # Reduced size scale

bubble = plt.scatter(
    data['Year of publication'],
    data['Clustering_Number_of_Specific_Domains'],
    s=size_scale,  # Bubble size scaling
    c=data['Clustering_Any_Non_Clinical'],  # Using the non-clinical feature for colors
    cmap=cmap,  # Updated color map for binary data
    alpha=transparency,  # Transparency determined by 'Quality_Index_Mestre'
)

# Adding titles and labels
plt.xlabel('Year of publication', fontsize=14)
plt.ylabel('Number of input cluster features', fontsize=14)

# Adjusting y-axis limits to avoid touching x-axis
plt.ylim(bottom=np.min(data['Clustering_Number_of_Specific_Domains']) - 1)

# Set y-ticks to cover from 1 to the max value of 'Clustering_Number_of_Specific_Domains', inclusive
yticks = np.arange(1, data['Clustering_Number_of_Specific_Domains'].max() + 1)
plt.yticks(yticks, fontsize=14)

# Modify x-axis to display ticks every four years
years = np.arange(data['Year of publication'].min(), data['Year of publication'].max() + 1, 3)
plt.xticks(years, fontsize=14)

# Manually set and enlarge the legend with accurate colors
colors = [cmap(0), cmap(1)]  # Extract colors from the colormap
handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=colors[0], markersize=14, label="Only utilizes clinical features"),
           plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=colors[1], markersize=14, label='Also utilizes a non-clinical features')]

legend = plt.legend(handles=handles, title="", frameon=False, loc='upper left', fontsize=12, title_fontsize=14)

plt.grid(False)
plt.tight_layout()

# Saving
plt.savefig('Plots/SVG/'+figurename+'.svg', format='svg', dpi=300)  # Export as SVG
plt.savefig('Plots/PNG/'+figurename+'.png', format='png', dpi=300)  # Export as PNG

plt.show()
